In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jul 19 16:14:43 2016

@author: nmvenuti
"""

# -*- coding: utf-8 -*-
"""
Created on Thu Jun  2 15:23:11 2016

@author: nmvenuti
"""
import time
start=time.time()
import sys, os

os.chdir('/Users/Seth/Documents/DSI/Capstone/DSI-Religion-2017')
#from joblib import Parallel, delayed
#import multiprocessing as mp
import os.path
import numpy as np
import pandas as pd
from datetime import datetime
sys.path.append('./prototype_python/')
import lingual as la
import nltk
nltk.download('punkt')
nltk.download('maxent_treebank_pos_tagger')
nltk.download('averaged_perceptron_tagger')


end=time.time()
#sys.stdout = open("output.txt", "a")
print(str(datetime.now()))
print('finished loading packages after '+str(end-start)+' seconds')
sys.stdout.flush()

#
import getNewDocs as gnd

stemmer = nltk.stem.snowball.EnglishStemmer()


[nltk_data] Downloading package punkt to /Users/Seth/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /Users/Seth/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Seth/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
2016-11-12 17:28:10.198313
finished loading packages after 2.19654512405 seconds


In [2]:
fileDF=gnd.newDocsToDF('./data_dsicap/', bin=5) ################################### WHERE THE NEW FILES ARE

fileList=fileDF.values.tolist()

fileList=[[fileList[i][0],fileList[i][1],fileList[i][2]] for i in range(len(fileList))]


#Get set of subgroups
subgroupList=[ list(y) for y in set((x[0],x[2]) for x in fileList) ]
print(subgroupList)
print('%%%%%\nlength of subgroupList is ' + str(len(subgroupList)))

50
41
59
20
53
[['DorothyDay', 'test0'], ['IntegralYoga', 'test11'], ['SeaShepherds', 'test1'], ['IntegralYoga', 'test1'], ['WBC', 'test3'], ['DorothyDay', 'test4'], ['Bahai', 'test7'], ['IntegralYoga', 'test8'], ['WBC', 'test8'], ['WBC', 'test7'], ['DorothyDay', 'test8'], ['Bahai', 'test3'], ['IntegralYoga', 'test4'], ['DorothyDay', 'test1'], ['IntegralYoga', 'test10'], ['SeaShepherds', 'test2'], ['IntegralYoga', 'test0'], ['WBC', 'test0'], ['DorothyDay', 'test5'], ['Bahai', 'test6'], ['WBC', 'test9'], ['WBC', 'test4'], ['Bahai', 'test2'], ['IntegralYoga', 'test7'], ['Bahai', 'test9'], ['WBC', 'test10'], ['SeaShepherds', 'test3'], ['IntegralYoga', 'test3'], ['WBC', 'test1'], ['DorothyDay', 'test2'], ['Bahai', 'test5'], ['WBC', 'test5'], ['DorothyDay', 'test6'], ['Bahai', 'test1'], ['IntegralYoga', 'test6'], ['Bahai', 'test8'], ['IntegralYoga', 'test2'], ['DorothyDay', 'test3'], ['Bahai', 'test4'], ['SeaShepherds', 'test0'], ['WBC', 'test2'], ['DorothyDay', 'test7'], ['Bahai', 'test0']

In [3]:
rawPath = './data_dsicap/' ###############change this eventually
runDirectory='./modelOutput/'
#groupList=['DorothyDay','JohnPiper','MehrBaba','NaumanKhan','PastorAnderson',
#   'Rabbinic','Shepherd','Unitarian','WBC']
#groupList=['DorothyDay','NaumanKhan','Rabbinic','NawDawg','SeaShepherds','IntegralYoga','Bahai']
#cocoWindow=int(sys.argv[1])
#cvWindow=int(sys.argv[2])
#startCount=int(sys.argv[3])
#netAngle=int(sys.argv[4])
cocoWindow=3
cvWindow=3
startCount=0
netAngle=30
groupSize=10
#testSplit=.3
targetWordCount=10
svdInt=50
simCount=1000

In [4]:
#Create paramList
paramList=[[x,fileList,targetWordCount,cocoWindow,svdInt,cvWindow,simCount,startCount,netAngle] for x in subgroupList]
print(len(paramList))

46


# PICK THE GROUP

In [5]:
#Run calculation 
#masterOutput=[textAnalysis(x) for x in paramList]  ### INSTEAD OF THIS, WE MAKE THE OBJECT
paramPick = paramList[0] ### instead of looping through, we just pick one ############################### PICK GROUP###
#print(paramPick)

In [6]:
#def textAnalysis(paramPick):
startTime=time.time()
groupId=paramPick[0]
fileList=paramPick[1]
targetWordCount=paramPick[2]
cocoWindow=paramPick[3]
svdInt=paramPick[4]
cvWindow=paramPick[5]
simCount=paramPick[6]
startCount=paramPick[7]
netAngle=paramPick[8]    

print(groupId)

['DorothyDay', 'test0']


In [7]:
#Get list of subfiles
subFileList=[x[1] for x in fileList if x[0]==groupId[0] and x[2]==groupId[1]]
print(len(subFileList))
print(subFileList)

5
['./data_dsicap/DorothyDay/raw/149.html.txt', './data_dsicap/DorothyDay/raw/150.html.txt', './data_dsicap/DorothyDay/raw/165.html.txt', './data_dsicap/DorothyDay/raw/172.html.txt', './data_dsicap/DorothyDay/raw/176.html.txt']


In [8]:
############################
#Create lingual object
loTest=la.lingualObject(subFileList)
############################

In [9]:
##### LOOK AT LIST OF TOKENS FOR FILE
#print(subFileList[0])
#loTest.tokens[subFileList[0]]
print(loTest.group)

DorothyDay


In [10]:
#Get coco
loTest.getCoco(cocoWindow)

#        self.cocoWindow=k
#        self.cocoDict={}
#        self.TF={}
#        self.docTF={}

print('%%%%\ncocoDict, length ' + str(len(loTest.cocoDict)))
print('---the first ten keys:')
print(loTest.cocoDict.keys()[:10])
print('---the first entry:')
key1 = loTest.cocoDict.keys()[0]
print(key1)
print(loTest.cocoDict[key1])

%%%%
cocoDict, length 1803
---the first ten keys:
['', u'hatr', u'yellow', u'four', u'hath', u'sleep', u'oldest', u'hate', u'consider', u'whose']
---the first entry:

{u'toknow': 1, u'they': 3, u'secur': 1, u'began': 1, u'each': 1, u'one': 1, u'write': 1, u'other': 1, u'through': 1, u'can': 1, u'have': 2, u'reader': 1, u'not': 1, u'yes': 1, u'believ': 1}


In [11]:
#Get DSM
loTest.getDSM(svdInt)
print('loTest.svdK: ' + str(loTest.svdK))

print('%%%%\nDSM, length ' + str(len(loTest.DSM)))
print('---the first ten keys:')
print(loTest.DSM.keys()[:10])
print('---the first entry:')
print(loTest.DSM[loTest.DSM.keys()[0]])

loTest.svdK: 50
%%%%
DSM, length 1803
---the first ten keys:
['', u'hatr', u'yellow', u'four', u'hath', u'sleep', u'oldest', u'hate', u'forget', u'whose']
---the first entry:
{0: 0.53963857353390854, 1: -0.061081258942444379, 2: 1.1959856434397802, 3: -1.1341449045547018, 4: -0.33640305789167546, 5: -0.16422258393407224, 6: -0.56233246232165435, 7: -0.83093163793420977, 8: -0.12921832622537738, 9: 1.2299261630099823, 10: 0.21995008616917899, 11: 0.064278892109024113, 12: 0.20911847487211777, 13: -0.1636763461009226, 14: -0.770192542023036, 15: -0.74246770776163995, 16: 0.58943345418835014, 17: 0.94287785482494191, 18: -0.084066302449313526, 19: 0.78469933140521542, 20: 0.61535883222871812, 21: 0.68710183657979229, 22: -0.35349769535545306, 23: -0.18827179786823953, 24: -0.51744914386064422, 25: -0.79499444935180086, 26: -1.0349860729402356, 27: -0.42271651858822962, 28: -0.40240737999241566, 29: 0.0038449973681982793, 30: 0.12919682990662801, 31: 0.052296552795399305, 32: -0.2698743012

# the old way
##### adjadv

In [12]:
#Set keywords
loTest.setKeywords('adjAdv',targetWordCount,startCount)
print(loTest.keywords)

[u'not', 'so', u'poor', u'when', u'onli', 'as', u'more', u'much', u'also', u'manual']


./prototype_python/lingual.py:444: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  targetDF.sort(['count'],inplace=True,ascending=False)


# the Manual way
##### manual

In [13]:
#Set keywords
loTest.setKeywords('manual',targetWordCount,startCount)
print(loTest.keywords)

['catholic', 'mass', 'god', 'communist', 'hope', 'peace']


# the new way
#### tfidf

In [14]:
loTest.setKeywords('tfidf',targetWordCount,startCount)
#print(loTest.keywords)
#keys = [word for word in loTest.keywords if not 'codecerror']
#keys = []
#for word in loTest.keywords:
#    if word != 'codecerror':
#        keys = keys + [word]
#print(keys)
#loTest.keywords = keys
print(loTest.keywords)

[u'you', 'we', u'your', u'fruit', u'love', u'our', u'mind', u'tree', 'me']


### the guts of the new way

In [137]:
idfFile = 'wiki-test-5-IDF.csv'
idf = pd.read_csv('./wiki-IDF/' + idfFile)
idf.head()

,term,freq,idf,logidf
0,four,91,9.164835,2.215374
1,cyprus,8,104.250000,4.646792
2,lord,15,55.600000,4.018183
3,digit,9,92.666667,4.529009
4,prize,18,46.333333,3.835862


In [138]:
idf.describe()

,freq,idf,logidf
count,2612.000000,2612.000000,2612.000000
mean,36.557427,65.961852,3.847418
std,83.011571,43.172716,0.989037
min,6.000000,1.000000,0.000000
25%,8.000000,27.800000,3.325036
50%,14.000000,59.571429,4.087176
75%,30.000000,104.250000,4.646792
max,834.000000,139.000000,4.934474


In [139]:
#idf.ix[loTest.keywords[0]]
#loTest.keywords[0]
idf = idf.set_index('term')

In [140]:
idf.sort_values(by='idf', ascending=False).head(10)


,freq,idf,logidf
term,,,
threat,6,139.0,4.934474
senat,6,139.0,4.934474
abolish,6,139.0,4.934474
mph,6,139.0,4.934474
milk,6,139.0,4.934474
excess,6,139.0,4.934474
gordon,6,139.0,4.934474
gone,6,139.0,4.934474
minnesota,6,139.0,4.934474


In [141]:
idf.sort_values(by='idf', ascending=True).head(10)

,freq,idf,logidf
term,,,
of,834,1.000000,0.000000
",",834,1.000000,0.000000
:,834,1.000000,0.000000
linknum:num,834,1.000000,0.000000
.,834,1.000000,0.000000
the,834,1.000000,0.000000
and,833,1.001200,0.001200
in,831,1.003610,0.003604
a,815,1.023313,0.023045


In [142]:
#idf.head()
idf.ix['milk']

freq        6.000000
idf       139.000000
logidf      4.934474
Name: milk, dtype: float64

In [143]:
print(loTest.keywords[0])
idf.ix[loTest.keywords[0]]


thi


KeyError: u'thi'

In [144]:
idf.ix[loTest.keywords[0]].tolist()
idf.ix[loTest.keywords[0]][2]

KeyError: u'thi'

In [145]:

for keyword in loTest.keywords:
    try:
        print(keyword)
        print(idf.ix[keyword])
    except:
        print(keyword)
        print('NOT FOUND')
        

thi
thi
NOT FOUND
thou
thou
NOT FOUND
my
freq        7.000000
idf       119.142857
logidf      4.780323
Name: my, dtype: float64
thee
thee
NOT FOUND
prophet
freq        6.000000
idf       139.000000
logidf      4.934474
Name: prophet, dtype: float64
me
me
NOT FOUND
we
freq      11.000000
idf       75.818182
logidf     4.328338
Name: we, dtype: float64
uniti
uniti
NOT FOUND
fate
fate
NOT FOUND


In [146]:
#loTest.rawText

In [147]:
all_words = []
#[all_words + toke for toke in loTest.tokens.values()]
for toke in loTest.tokens.values():
    all_words = all_words + toke

print(len(all_words))
print(all_words[:20])

4217
['o', u'thou', u'whose', u'face', 'is', u'the', u'object', 'of', 'my', u'ador', u'whose', u'beauti', 'is', 'my', u'sanctuari', u'whose', u'habit', 'is', 'my', u'goal']


In [148]:
from nltk.probability import FreqDist
freq = FreqDist(all_words) ## create FreqDist object with word frequencies
#
columns_obj = ["term", "freq"]
freqDF = pd.DataFrame(freq.items(), columns=columns_obj) # convert it to a data frame
freqDF = freqDF.set_index('term')
print('freqDF ' + str(freqDF.shape))
freqDF.head(5)

freqDF (1195, 1)


,freq
term,
all,41
concept,1
crusad,1
founder,1
sanctiti,1


In [149]:
#freqit = pd.concat([freqDF, idf])
freqit = freqDF.join(idf[['idf', 'logidf']])
# replace null values with max
maxidf = max(freqit['idf'].dropna())
maxlogidf = max(freqit['logidf'].dropna())
freqit.loc[pd.isnull(freqit['idf']), 'idf'] = maxidf
freqit.loc[pd.isnull(freqit['logidf']), 'logidf'] = maxlogidf


print('freqit ' + str(freqit.shape))
freqit.head()

freqit (1195, 3)


,freq,idf,logidf
term,,,
all,41,4.343750,1.468738
concept,1,28.758621,3.358938
crusad,1,139.000000,4.934474
founder,1,46.333333,3.835862
sanctiti,1,139.000000,4.934474


In [150]:
# should be empty, if the above cell has worked correctly
print(freqit[pd.isnull(freqit['idf'])])

Empty DataFrame
Columns: [freq, idf, logidf]
Index: []


In [151]:
freqit['tfidf'] = freqit['freq'] * freqit['idf']
freqit['logtfidf'] = freqit['freq'] * freqit['logidf']

In [152]:
#freqit = freqit.dropna(subset=['freq'], how='all')
freqit.shape

(1195, 5)

# THE RANKED TERMS

In [153]:
freqit = freqit.sort_values(by='tfidf', ascending=False) 
freqit

,freq,idf,logidf,tfidf,logtfidf
term,,,,,
codecerror,98,139.000000,4.934474,13622.000000,483.578445
thi,42,139.000000,4.934474,5838.000000,207.247905
thou,34,139.000000,4.934474,4726.000000,167.772114
my,21,119.142857,4.780323,2502.000000,100.386788
thee,18,139.000000,4.934474,2502.000000,88.820531
prophet,12,139.000000,4.934474,1668.000000,59.213687
me,12,139.000000,4.934474,1668.000000,59.213687
we,21,75.818182,4.328338,1592.181818,90.895101
uniti,9,139.000000,4.934474,1251.000000,44.410265


In [43]:
startCount=0
wordCount=10
freqit.iloc[startCount:wordCount+startCount].index.tolist()

[u'codecerror',
 u'you',
 'we',
 u'yoga',
 u'your',
 u'mind',
 u'our',
 u'self',
 u'sutra',
 u'conscious']

## doing a little exploring

In [83]:
for word in loTest.keywords:
    print(word + ': ' + str(freq[word]))

not: 94
when: 67
so: 58
onli: 28
where: 28
mani: 23
then: 23
good: 24
more: 23
back: 22


In [87]:
for word in loTest.keywords:
    try:
        print('%%%%%%%%\n' + word + ': ' + str(idf.ix[word]) + '\n%%%%%%%%')
    except:
        print('%%%%%%%%\n' + word + ' not found in IDF\n%%%%%%%%')

%%%%%%%%
not: freq      262.000000
idf       803.729008
logidf      6.689262
Name: not, dtype: float64
%%%%%%%%
%%%%%%%%
when not found in IDF
%%%%%%%%
%%%%%%%%
so not found in IDF
%%%%%%%%
%%%%%%%%
onli not found in IDF
%%%%%%%%
%%%%%%%%
where: freq      390.000000
idf       539.941026
logidf      6.291460
Name: where, dtype: float64
%%%%%%%%
%%%%%%%%
mani: freq      103808.000000
idf            2.028524
logidf         0.707308
Name: mani, dtype: float64
%%%%%%%%
%%%%%%%%
then not found in IDF
%%%%%%%%
%%%%%%%%
good: freq      43387.000000
idf           4.853458
logidf        1.579692
Name: good, dtype: float64
%%%%%%%%
%%%%%%%%
more: freq      458.000000
idf       459.775109
logidf      6.130737
Name: more, dtype: float64
%%%%%%%%
%%%%%%%%
back: freq      61096.000000
idf           3.446658
logidf        1.237405
Name: back, dtype: float64
%%%%%%%%


,term,freq
0,,5
1,vermin,2
2,four,7
3,protest,1
4,sleep,5
5,mansion,2
6,oldest,1
7,hate,3
8,whose,4
9,accus,4


In [ ]:
    # term_frequency is a dict which structure is like:
    # {
    #     'path_to_file': 
    #         {'term': 13.4, 'another_term': 15}, 
    #     'another_file': 
    #         {'term2': 12, 'foo': 15}
    #  } 
    for term in freq.keys():
        if isintance(term_frequency[text], dict):
            term_frequency[text][term] = freq[term]/numbers_of_words
        else:
            term_frequency[text] = {term: freq[term]/numbers_of_words}

In [13]:
#import nltk
#stemmer = nltk.stem.snowball.EnglishStemmer()
#stemkeywords=[stemmer.stem(word) for word in loTest.keywords]
#print(stemkeywords)

[u'not', u'good', u'then', u'even', u'when', 'so', u'here', u'other', u'also', u'differ']


In [14]:
for key in loTest.keywords: ### get context vectors for keywords...
    print('%%%%%\n' + key)
    print(loTest.cocoDict[key]) # fails on 'only' and then again on 'spiritual'

#print(loTest.cocoDict['spiritual'])
print(loTest.cocoDict['spirit'])

%%%%%
not
{u'all': 2, u'distanc': 1, u'they': 2, u'just': 2, u'queen': 1, u'sage': 1, u'becaus': 3, u'violent': 1, u'go': 1, u'chair': 1, u'still': 1, u'find': 1, u'help': 1, u'fit': 1, u'readabl': 1, u'should': 2, 'to': 11, u'book': 1, u'factor': 1, u'sens': 2, u'delici': 1, u'has': 1, u'might': 1, u'real': 1, 'do': 3, u'good': 1, u'everi': 1, u'read': 2, u'dessert': 1, u'somebodi': 1, u'know': 1, u'codecerror': 11, u'veri': 1, u'one': 1, u'school': 1, u'anyth': 5, u'smell': 1, u'drop': 1, u'these': 1, u'bad': 2, u'contain': 1, u'plank': 1, u'enjoy': 1, u'the': 23, u'right': 2, u'mental': 1, u'natur': 1, u'mind': 3, u'realli': 1, u'see': 1, u'are': 16, u'seeabl': 1, u'cold': 1, u'infinit': 1, u'serv': 1, u'what': 1, u'said': 1, u'for': 3, u'bhagavad': 1, u'someth': 1, u'abl': 2, u'uniform': 1, u'doer': 1, u'experi': 1, 'be': 6, 'we': 3, u'who': 1, u'led': 1, u'isol': 1, u'here': 1, u'bodi': 3, u'ask': 1, u'eye': 1, u'come': 1, 'by': 2, u'faith': 2, 'of': 2, u'could': 1, u'vidyalayam':

In [15]:
#for key in stemkeywords: ### get context vectors for keywords...
#    print('%%%%%\n' + key)
#    print(loTest.cocoDict[key]) # fails on 'only' and then again on 'spiritual'



In [16]:
## keywords in each file
for thisfile in subFileList:
    print(thisfile)
    filetokens = loTest.tokens[thisfile]
    for keyword in loTest.keywords:
         if keyword in filetokens:
                print(keyword)
#    if 'spirtual' in filetokens:
#        print("UAL!")
#    if 'spirit' in filetokens:
#        print("SPIRIT!")

./data_dsicap/IntegralYoga/raw/YV05.txt
not
good
then
even
when
so
here
other
also
./data_dsicap/IntegralYoga/raw/YV48.txt
not
then
even
when
so
other
also
differ
./data_dsicap/IntegralYoga/raw/YV54.txt
not
even
when
so
other
./data_dsicap/IntegralYoga/raw/YV58.txt
not
then
even
when
so
other
also
differ


In [17]:
## STEM keywords in each file
for thisfile in subFileList:
    print(thisfile)
    filetokens = loTest.tokens[thisfile]
    for keyword in stemkeywords:
         if keyword in filetokens:
                print(keyword)
    

./data_dsicap/IntegralYoga/raw/YV05.txt
not
good
then
even
when
so
here
other
also
./data_dsicap/IntegralYoga/raw/YV48.txt
not
then
even
when
so
other
also
differ
./data_dsicap/IntegralYoga/raw/YV54.txt
not
even
when
so
other
./data_dsicap/IntegralYoga/raw/YV58.txt
not
then
even
when
so
other
also
differ


In [18]:
#######################            
###Semantic analysis###
#######################

#Get context vectors
loTest.getContextVectors(cvWindow)
print('loTest.cvWindow: ' + str(loTest.cvWindow))

#        self.cvWindow=k
#        self.cvDict={}

print('%%%%\ncvDict, length ' + str(len(loTest.cvDict)))
print('---the first ten keys:')
print(loTest.cvDict.keys()[:10])
print('---the keywords each file:')
for key in loTest.cvDict.keys():
    print(key + ': ' + str(loTest.cvDict[key].keys()))
print('%%%%%%%')
#file1 = loTest.cvDict.keys()[0] #'./data_dsicap/IntegralYoga/raw/YV38.txt'
    
for file1 in loTest.cvDict.keys():    
    first = loTest.cvDict[file1]
    #print('---the first entry (length ' + str(len(first[first.keys()[0]])) + '):')
    print('%%%%%\n' + file1 + ' (' + str(len(first.keys())) + ' keys):')
    print('---keys: ' + str(first.keys()))
    for wordkey in first.keys():
        print('------thisfile[' + wordkey + ']:')
        tftk = first[wordkey]
        print('-------length: "' + wordkey + '" appears ' + str(len(tftk.keys())) + ' times in this file.')
        for numkey in tftk.keys():
            print('---------length of thisfile[' + wordkey +'][' + str(numkey) + ']: ' + str(len(tftk[numkey])))
        

loTest.cvWindow: 3
%%%%
cvDict, length 4
---the first ten keys:
['./data_dsicap/IntegralYoga/raw/YV58.txt', './data_dsicap/IntegralYoga/raw/YV48.txt', './data_dsicap/IntegralYoga/raw/YV54.txt', './data_dsicap/IntegralYoga/raw/YV05.txt']
---the keywords each file:
./data_dsicap/IntegralYoga/raw/YV58.txt: [u'even', u'then', u'differ', u'when', u'also', u'other', 'so', u'not']
./data_dsicap/IntegralYoga/raw/YV48.txt: [u'even', u'then', u'differ', u'when', u'also', u'other', 'so', u'not']
./data_dsicap/IntegralYoga/raw/YV54.txt: [u'not', u'even', u'when', u'other', 'so']
./data_dsicap/IntegralYoga/raw/YV05.txt: [u'even', u'then', u'good', u'when', u'here', u'also', u'other', 'so', u'not']
%%%%%%%
%%%%%
./data_dsicap/IntegralYoga/raw/YV58.txt (8 keys):
---keys: [u'even', u'then', u'differ', u'when', u'also', u'other', 'so', u'not']
------thisfile[even]:
-------length: "even" appears 5 times in this file.
---------length of thisfile[even][1]: 50
---------length of thisfile[even][2]: 50
-----

In [19]:
#Get average semantic density
#avgSD=np.mean([x[1] for x in loTest.getSD(simCount)])
#print(avgSD)

SD = [x[1] for x in loTest.getSD(simCount)]
print(len(SD))
print(loTest.keywords)
print(SD)

./data_dsicap/IntegralYoga/raw/YV05.txt
[u'even', u'then', u'good', u'when', u'here', u'also', u'other', 'so', u'not']
./data_dsicap/IntegralYoga/raw/YV48.txt
[u'even', u'then', u'differ', u'when', u'also', u'other', 'so', u'not']
./data_dsicap/IntegralYoga/raw/YV54.txt
[u'not', u'even', u'when', u'other', 'so']
./data_dsicap/IntegralYoga/raw/YV58.txt
[u'even', u'then', u'differ', u'when', u'also', u'other', 'so', u'not']
10
[u'not', u'good', u'then', u'even', u'when', 'so', u'here', u'other', u'also', u'differ']
[0.72404735956380151, 0.70700627327625354, 0.71615225785121706, 0.69572043045266363, 0.75862725558225175, 0.72200533701172254, 0.75878819429898337, 0.67993472478851136, 0.77246014024129339, 0.76795665602778718]


In [ ]:
########################################
###POS Tagging and Judgement Analysis###
########################################
judgementAvg=list(np.mean(np.array([[x[1],x[2]] for x in loTest.getJudgements()]),axis=0))

########################
###Sentiment Analysis###
########################

sentimentList=loTest.sentimentLookup()

############################
###Network Quantification###
############################
loTest.setNetwork(netAngle)

avgEVC=loTest.evc()

endTime=time.time()
timeRun=endTime-startTime
print('finished running'+'_'.join(groupId)+' in '+str(end-start)+' seconds')
sys.stdout.flush()

#Append outputs to masterOutput
return(['_'.join(groupId)]+[len(subFileList),timeRun]+sentimentList+judgementAvg+[avgSD]+[avgEVC])   

In [17]:
### practicing
naw = ['test', 'train', 'test']
paw = [1,2,3]

jaw = [naw[i] + str(paw[i]) for i in range(0,len(paw))]
print(jaw)

['test1', 'train2', 'test3']


In [27]:
import math
bincount = 21
print(int(math.ceil(bincount * 0.3)))
print(bincount * 0.3)



7
6.3


In [13]:

textList= ['May', 'you', 'I', 'help', 'me', 'with', 'our', 'problem']
print(textList)


['May', 'you', 'I', 'help', 'me', 'with', 'our', 'problem']


In [17]:
from nltk.tag.perceptron import PerceptronTagger
tagger = PerceptronTagger()

import nltk
from nltk.tag import pos_tag

def drop_pronouns(textList):
    tags = tagger.tag(textList)
    keep = []
    for i in range(1,len(textList)):
        if (tags[i][1] != 'PRP') & (tags[i][1] != 'PRP$'):
            #pronouns.append(tags[i][0])
            keep = keep + [i]
    return [textList[i] for i in keep]

textList= ['May', 'you', 'I', 'help', 'me', 'with', 'our', 'problem']
drop_pronouns(textList)
  


['help', 'with', 'problem']

In [ ]:
ddt = [u'the', u'cathol', u'worker', u'septemb', u'num', u'num', u'summari', u'anoth', u'chapter', u'from', u'her', u'unpublish', u'biographi', 'of', u'peter', u'maurin', u'describ', u'stfranci', 'as', u'the', u'great', u'personalist', u'and', u'goe', 'on', 'to', u'explic', 'a', u'philosophi', 'of', u'work', u'see', 'it', 'as', 'a', u'gift', 'a', u'vocat', u'that', u'one', u'should', u'find', u'what', u'hesh', u'doe', u'best', u'and', u'develop', 'it', u'encourag', u'scholar', 'to', u'becom', u'worker', u'and', u'worker', 'to', u'becom', u'scholar', 'in', u'order', u'that', u'more', u'understand', u'exit', u'between', u'the', u'two', u'defend', u'peter', u'from', u'the', u'critic', 'of', u'be', 'a', u'materialist', u'and', u'portray', u'him', 'as', 'an', u'apostl', 'to', u'the', u'world', u'not', 'of', u'the', u'world', u'ddlw', u'num', u'anoth', u'chapter', u'from', 'a', u'biographi', 'of', u'peter', u'maurin', u'lay', u'apostl', u'founder', 'of', u'the', u'cathol', u'worker', u'movement', u'peter', 'is', u'alway', u'get', u'back', 'to', u'saint', u'franci', 'of', u'assisi', u'who', u'was', u'most', u'truli', u'the', u'great', u'personalist', 'in', u'his', u'poverti', u'rich', 'in', u'renounc', u'all', u'possess', u'all', u'generous', u'give', u'out', 'of', u'his', u'heart', u'sow', u'generous', u'and', u'reap', u'generous', u'humbl', u'and', u'ask', u'when', 'in', u'need', u'possess', u'freedom', u'and', u'all', u'joy', u'without', u'doubt', u'peter', 'is', 'a', u'free', u'and', u'joyous', u'person', u'and', 'it', 'is', u'the', u'freedom', u'and', u'joyous', u'that', u'come', u'from', 'a', u'clear', u'heart', u'and', u'soul', u'there', u'are', u'those', u'who', u'might', u'say', 'it', u'come', u'becaus', 'of', u'his', u'anarchist', u'natur', u'his', u'refus', 'to', u'enter', u'into', u'polit', u'controversi', u'his', u'refus', 'to', u'use', u'world', u'mean', 'to', u'chang', u'the', u'social', u'order', 'he', u'doe', u'not', u'inde', u'refus', 'to', u'use', u'mystic', u'mean', u'physic', u'mean', u'secular', u'mean', u'the', u'mean', u'that', u'are', 'at', u'hand', u'but', u'the', u'mean', 'of', u'expedi', u'that', u'men', u'have', u'turn', 'to', u'for', 'so', u'mani', u'age', 'he', u'disdain', 'he', 'is', 'no', u'diplomat', 'he', 'is', 'no', u'politician', 'he', u'has', 'so', u'thorough', u'discourag', 'in', u'his', u'follow', u'the', u'use', 'of', u'polit', u'mean', u'that', 'he', u'has', u'been', u'term', 'an', u'anarchist', 'by', u'mani', u'especi', 'by', u'our', u'dear', u'jesuit', u'friend', u'father', u'dowl', u'who', u'often', u'has', u'often', u'come', 'to', 'us', u'and', u'talk', 'to', 'us', 'of', u'proport', u'represent', 'to', u'give', 'up', u'superflu', u'possess', u'peter', u'has', 'no', u'incom', 'so', u'doe', u'not', u'need', 'to', u'worri', u'about', u'incom', u'tax', 'he', u'doe', u'not', u'worri', u'about', u'ration', 'he', u'use', u'those', u'thing', 'he', u'need', 'in', u'the', u'way', 'of', u'cloth', u'and', u'food', 'as', u'though', 'he', u'use', u'them', u'not', 'he', u'has', 'no', u'worri', u'about', u'style', u'fit', u'fashion', 'he', u'eat', u'what', 'is', u'put', u'befor', u'him', u'and', 'if', 'he', u'prefer', u'anyth', 'he', u'prefer', u'veget', u'stew', 'to', u'meat', 'a', u'hot', u'drink', 'to', 'a', u'cold', u'oil', 'to', u'butter', 'he', u'doe', u'not', u'smoke', 'he', u'doe', u'not', u'drink', u'wine', u'onli', u'becaus', 'it', u'caus', u'his', u'brother', 'to', u'stumbl', u'otherwis', 'he', u'believ', 'in', u'feast', 'as', u'well', 'as', u'fast', u'and', u'there', u'are', u'after', u'all', u'mani', u'feast', u'day', u'day', 'of', u'rejoic', u'wed', u'baptism', u'feast', u'name', u'day', u'and', u'all', u'the', u'saint', u'day', u'saint', u'franci', u'desir', u'that', u'men', u'should', u'work', u'with', u'their', u'hand', u'peter', u'enjoy', u'manual', u'labor', 'he', u'use', 'to', u'tell', u'the', u'late', u'father', u'virgil', u'michel', u'that', 'if', u'benedictin', u'had', u'kept', 'to', u'their', u'earli', u'ideal', 'of', u'manual', u'labor', u'there', u'would', u'not', 'be', 'so', u'mani', u'breakdown', u'from', u'mental', u'overwork', 'we', u'must', u'use', u'the', u'whole', u'man', u'say', u'peter', 'so', u'that', 'we', u'may', 'be', u'holi', u'men', 'he', u'may', 'be', u'quot', 'it', u'sound', u'like', u'eric', u'gill', u'but', 'it', u'also', u'sound', u'like', u'peter', u'there', 'is', u'noth', 'he', u'like', u'better', u'than', u'build', u'fire', u'and', 'to', u'get', u'down', u'and', u'poke', 'a', u'grate', u'fire', u'until', 'it', 'is', u'all', u'but', u'out', u'and', u'poke', u'kindl', u'wood', 'in', u'under', u'the', u'coal', u'and', u'shake', 'it', u'down', u'and', u'final', u'dump', 'it', u'and', u'rebuild', 'it', u'altogeth', u'that', 'is', u'fun', u'then', 'to', u'labori', 'go', u'over', u'the', u'coal', 'we', u'have', 'no', u'sifter', u'get', u'out', u'the', u'piec', 'so', u'none', u'will', 'be', u'wast', u'and', 'to', u'empti', u'the', u'ash', u'and', u'usual', u'the', u'wind', u'blow', u'them', u'all', u'over', u'peter', u'his', u'grey', u'hair', u'his', u'iron', u'grey', u'suit', u'and', u'shoe', u'ive', u'seen', u'him', u'set', u'out', u'like', u'that', 'to', u'give', 'a', u'lectur', u'somewher', u'all', u'unbrush', u'and', u'uncomb', u'and', u'run', u'after', u'him', 'to', u'refurbish', u'him', 'a', u'bit', u'for', u'compani', 'it', 'is', u'for', u'the', u'sake', 'of', u'other', 'i', u'tell', u'him', u'but', u'peter', 'is', u'oblivi', 'of', u'appear', u'there', 'is', u'not', u'much', 'in', u'the', u'way', 'of', u'manual', u'labor', 'he', u'can', 'do', u'around', u'mott', u'street', u'except', 'to', u'help', u'keep', u'the', u'fire', u'go', u'and', 'to', u'mend', u'chair', 'we', u'are', u'alway', u'short', 'of', u'chair', 'so', u'each', u'one', 'is', 'a', u'treasur', u'sinc', u'peopl', u'live', u'out', 'of', u'door', 'a', u'good', u'part', 'of', u'the', u'winter', 'as', u'well', 'as', u'the', u'summer', u'the', u'women', 'in', u'the', u'tenement', 'on', u'either', u'side', 'of', 'us', u'back', u'and', u'front', u'come', u'down', 'on', u'the', u'sidewalk', u'when', u'their', u'work', 'in', u'the', u'hous', 'is', u'done', u'and', u'just', u'sit', u'usual', u'they', u'come', u'into', u'the', u'store', u'which', 'is', u'the', u'offic', u'and', u'take', u'the', u'chair', 'if', u'they', u'like', u'the', u'chair', u'they', u'bring', u'them', u'upstair', u'with', u'them', 'to', u'their', u'home', u'leav', 'us', u'the', u'old', u'brokendown', u'one', u'that', 'is', u'the', u'way', u'the', u'voluntarili', u'poor', u'are', u'treat', 'i', u'tell', u'peter', u'how', u'long', u'doe', 'it', u'take', u'christian', 'to', u'work', u'anyway', u'becaus', 'of', u'our', u'generos', 'in', u'let', u'them', u'borrow', u'our', u'chair', u'becaus', 'we', u'believ', u'that', u'when', u'someon', u'take', u'our', u'coat', 'we', u'should', u'offer', u'him', u'our', u'cloak', u'too', u'then', u'the', u'argument', 'is', u'they', 'do', u'not', u'appreci', u'good', u'thing', u'they', u'dont', u'valu', u'what', u'they', u'have', 'so', 'we', u'might', 'as', u'well', u'take', u'them', 'we', u'take', u'care', 'of', u'them', u'ive', u'seen', u'that', u'argument', u'work', 'in', u'peopl', u'mind', u'hundr', 'of', u'time', u'they', u'justifi', u'themselv', u'clever', u'the', u'poor', u'dont', u'know', u'the', u'differ', u'they', u'say', u'them', 'as', u'has', u'git', u'and', u'from', u'them', u'who', u'have', u'not', u'what', u'they', u'have', u'shall', 'be', u'taken', u'away', u'that', 'us', u'peter', u'doe', u'not', u'answer', u'but', u'take', u'the', u'brokendown', u'chair', 'or', u'those', u'too', u'heavi', 'to', 'be', u'move', u'which', u'they', u'have', u'left', 'us', u'and', u'mend', u'them', u'uncomplain', 'on', u'the', u'farm', u'there', 'is', u'plenti', 'of', u'work', u'for', u'all', u'anoth', u'reason', u'whi', u'peter', u'was', u'alway', u'extol', u'the', u'land', u'peopl', u'can', u'not', u'live', u'without', u'work', u'work', 'is', 'as', u'necessari', 'as', u'bread', u'but', u'what', 'is', u'need', 'is', 'a', u'philosophi', 'of', u'work', u'work', 'is', 'a', u'gift', 'a', u'vocat', u'befor', u'the', u'fall', u'adam', u'was', u'given', u'the', u'garden', 'to', u'cultiv', 'it', u'was', u'onli', u'after', u'the', u'fall', u'that', u'all', u'natur', u'travail', u'and', u'groan', 'so', u'that', u'man', u'has', 'to', u'work', u'with', u'the', u'sweat', 'of', u'his', u'brow', u'and', u'combat', u'earthquak', u'flood', u'drought', u'boll', u'weevil', u'japanes', u'beetl', u'fatigu', u'and', u'sloth', 'we', u'have', 'to', u'recogn', u'work', 'as', 'a', u'penanc', u'but', 'we', u'must', u'also', u'recogn', u'work', 'as', 'a', u'gift', u'man', u'has', u'talent', u'which', u'god', u'has', u'given', u'him', u'and', 'he', u'must', u'develop', u'these', u'talent', 'he', u'must', u'find', u'the', u'work', 'he', u'can', 'do', u'best', u'and', u'then', u'learn', 'to', 'do', 'it', u'well', u'for', u'his', u'own', u'sake', u'and', u'the', u'sake', 'of', u'his', u'fellow', u'peter', u'indoctrin', u'about', u'scholar', u'and', u'worker', u'has', u'this', u'practic', u'result', u'around', u'the', u'hous', 'of', u'hospit', u'when', u'the', u'scholar', u'start', u'scrub', u'and', u'clean', u'hous', u'the', u'cooper', u'from', u'the', u'worker', 'is', u'more', u'will', u'more', u'spontan', u'everyon', u'want', 'to', u'help', u'the', u'labor', u'and', u'exercis', u'tend', 'to', u'reliev', u'the', u'discourag', u'that', u'often', u'threaten', 'to', u'encompass', u'the', u'scholar', 'he', u'understand', u'better', u'after', 'a', u'bout', u'with', 'a', u'mop', u'the', u'discourag', 'of', u'the', u'poor', u'man', u'his', u'sloth', u'his', u'hopeless', 'he', u'begin', u'too', 'to', u'understand', u'what', u'christ', u'meant', u'when', 'he', u'said', 'he', u'came', 'to', u'minist', u'not', 'to', 'be', u'minist', u'unto', 'he', u'begin', 'to', u'understand', u'the', u'humili', 'of', u'the', u'veri', u'poor', u'and', 'by', u'seek', u'them', u'voluntarili', 'he', u'find', u'peac', u'and', u'rest', 'in', u'them', 'my', u'yoke', 'is', u'easi', u'and', 'my', u'burden', u'light', u'the', u'meek', u'shall', u'inherit', u'the', u'earth', u'but', u'these', u'thing', u'are', u'not', u'understood', u'until', u'practic', u'saint', u'franci', u'said', u'you', u'can', u'not', u'know', u'what', u'you', u'have', u'not', u'practic', 'a', u'jewish', u'convert', u'who', u'had', u'been', u'make', 'a', u'retreat', u'with', 'us', 'at', u'maryfarm', u'said', u'some', u'week', u'after', 'it', 'is', u'hard', 'to', u'live', 'in', u'the', u'upsidedown', u'world', 'of', u'the', u'gospel', u'truli', 'it', 'is', 'a', u'world', 'of', u'paradox', u'give', 'up', u'one', u'life', 'in', u'order', 'to', u'save', 'it', u'die', 'to', u'live', 'it', 'is', u'voluntari', u'poverti', u'strip', u'oneself', u'even', 'of', u'what', u'the', u'world', u'call', u'digniti', u'honor', u'human', u'respect', u'for', u'truli', 'it', u'must', 'be', u'admit', u'that', u'one', u'doe', u'not', u'alway', u'have', u'the', u'respect', 'of', u'the', u'poor', 'of', u'the', u'worker', u'there', u'was', u'smitti', u'who', u'gave', u'out', u'cloth', 'in', u'the', u'basement', u'store', u'room', u'everi', u'day', u'and', u'for', u'three', u'hour', 'he', u'took', u'the', u'abus', 'of', u'the', u'women', u'and', u'men', u'who', u'came', u'for', u'underwear', u'and', u'sock', u'and', u'sweater', u'and', u'coat', u'these', u'cloth', u'are', u'sent', 'in', u'here', u'for', 'us', u'your', u'hold', u'out', 'on', 'us', u'you', u'gave', 'it', 'to', u'him', u'yesterday', u'and', u'now', u'you', u'have', u'noth', u'for', 'me', u'your', u'sell', u'the', u'stuff', u'yourself', u'smitti', u'meek', u'accept', u'all', u'rebuk', 'he', u'was', 'in', u'rag', u'himself', u'down', 'at', u'the', u'heel', u'wear', u'his', u'cloth', u'until', 'we', u'beg', u'him', 'to', u'find', u'someth', u'els', 'in', u'order', 'to', u'wash', u'the', u'thing', 'he', u'had', 'on', 'he', u'was', 'so', u'poor', u'and', u'look', 'so', u'poor', u'that', u'the', u'miser', u'one', u'who', u'came', u'suspect', u'him', 'of', u'their', u'own', u'vice', 'he', u'must', u'drink', 'he', u'must', u'steal', 'it', 'is', u'beyond', u'reason', u'that', u'anyon', u'voluntarili', u'should', u'stay', u'down', 'in', u'that', u'dingi', u'ratridden', u'hole', u'under', u'the', u'fivestori', u'tenement', u'and', u'give', u'out', u'cloth', u'and', u'bit', 'of', u'literatur', u'and', u'keep', 'on', u'take', u'abus', 'as', u'meek', 'as', 'he', u'did', u'and', u'alreadi', 'he', u'had', u'been', 'up', u'sinc', u'five', u'oclock', 'in', u'the', u'morn', u'get', u'the', u'coffe', u'readi', u'for', u'the', u'breadlin', u'slice', u'the', u'bread', u'which', u'the', u'men', u'soak', 'in', u'their', u'coffe', u'there', u'were', u'over', 'a', u'hundr', u'men', u'everi', u'morn', u'there', u'use', 'to', 'be', u'mani', u'time', u'that', u'number', u'but', 'in', u'war', u'time', u'there', u'were', u'not', 'so', u'mani', u'unemploy', u'there', u'are', u'the', u'old', u'the', u'crippl', u'the', u'unemploy', 'in', u'the', u'neighborhood', u'they', u'are', u'serv', 'in', u'one', 'of', u'the', u'two', u'store', u'which', u'front', u'our', u'stjoseph', u'hous', 'of', u'hospit', u'the', u'store', 'is', u'long', u'and', u'narrow', u'and', u'there', 'is', u'not', u'enough', u'room', u'for', u'tabl', u'and', u'chair', u'the', u'men', u'must', u'come', 'in', 'on', 'a', u'line', 'be', u'serv', u'their', u'coffe', u'and', u'bread', 'at', 'a', u'counter', u'drink', 'it', u'and', u'eat', u'their', u'soakedup', u'bread', u'and', u'then', 'go', u'out', u'and', u'let', u'anoth', u'batch', 'of', u'thirti', 'or', 'so', u'come', 'in', u'they', u'can', 'go', u'back', 'on', u'the', u'line', u'again', u'and', u'take', u'their', u'turn', 'on', u'second', u'help', u'and', u'they', u'can', u'take', u'all', u'the', u'bread', u'they', u'wish', 'in', u'their', u'rag', u'pocket', u'smitti', u'has', u'helper', 'of', u'cours', u'alex', u'the', u'russian', u'who', u'was', u'torpedo', 'in', u'the', u'last', u'war', u'and', u'afloat', u'for', u'hour', u'there', 'is', 'a', u'littl', u'swiss', u'fellow', u'veri', u'critic', 'of', u'the', u'men', 'he', 'is', u'serv', u'peter', u'say', u'peopl', u'learn', u'the', u'art', 'of', u'human', u'contact', 'by', u'live', 'in', 'a', u'hous', 'of', u'hospit', u'mani', 'a', u'time', u'peter', u'make', u'what', 'he', u'call', u'point', u'but', 'i', 'do', u'not', u'understand', u'for', u'month', 'he', u'build', 'up', 'a', u'program', 'of', u'action', u'his', u'listen', u'conced', u'the', u'necess', 'of', u'work', u'out', u'such', 'a', u'program', u'and', u'then', 'he', u'expect', u'them', 'to', u'guid', u'their', u'live', u'therebi', u'readjust', u'themselv', 'to', u'these', u'new', u'idea', u'which', 'he', u'has', u'present', 'if', 'he', u'fail', 'to', u'influenc', u'other', 'as', 'he', u'has', u'hope', 'he', u'shrug', u'his', u'shoulder', u'and', u'goe', 'on', u'propound', u'social', u'doctrin', 'he', 'is', u'content', 'to', u'wait', u'until', u'circumst', u'aris', u'which', u'will', 'be', u'more', u'favor', u'for', u'the', u'work', u'out', 'of', u'his', u'idea', u'certain', u'through', u'the', u'fifti', u'thousand', u'reader', 'of', u'the', u'cathol', u'worker', u'that', 'is', u'it', u'present', u'circul', 'he', u'has', u'found', u'for', u'himself', u'mani', u'reader', u'mani', u'listen', 'he', u'had', u'invit', 'to', u'speak', 'at', u'colleg', u'seminari', u'and', u'group', u'throughout', u'the', u'countri', u'through', u'the', u'hous', 'of', u'hospit', u'which', u'have', u'been', u'establish', 'he', u'has', u'built', 'up', u'group', u'for', u'round', u'tabl', u'discuss', u'through', u'the', u'farm', u'commune', 'he', u'has', u'direct', u'attent', 'to', u'fundament', u'econom', u'idea', 'to', u'him', u'there', 'is', 'a', u'synthesi', u'about', u'all', u'his', u'idea', u'they', u'fit', u'togeth', 'as', u'blueprint', u'for', 'a', u'new', u'world', u'they', u'are', u'unsurpass', u'idyl', u'but', u'when', 'it', u'come', 'to', u'work', u'them', u'out', u'given', u'the', u'human', u'materi', u'the', u'lack', 'of', u'equip', u'the', u'vagari', 'of', u'human', u'natur', u'there', 'is', u'the', u'rub', 'do', u'they', u'work', u'doe', u'christian', u'work', 'if', 'it', u'fail', 'it', 'is', u'glorious', 'in', u'it', u'failur', u'the', u'failur', 'of', u'the', u'cross', 'i', 'do', u'know', u'thisthat', u'when', u'peopl', u'come', u'into', u'contact', u'with', u'peter', u'maurin', u'they', u'chang', u'they', u'awaken', u'they', u'begin', 'to', u'see', u'thing', u'becom', 'as', u'new', u'they', u'look', 'at', u'life', 'in', u'the', u'light', 'of', u'the', u'gospel', u'they', u'admit', u'the', u'truth', 'he', u'possess', u'and', u'live', 'by', u'and', u'though', u'they', u'themselv', u'fail', 'to', 'go', u'the', u'whole', u'way', u'their', u'face', u'are', u'turn', 'at', u'least', u'toward', u'the', u'light', u'and', u'peter', 'is', u'patient', u'look', 'at', u'thing', 'as', 'he', u'doe', 'in', u'the', u'light', 'of', u'histori', u'take', u'the', u'long', u'view', 'he', 'is', u'content', 'to', u'play', u'his', u'part', 'to', u'live', 'by', u'his', u'principl', u'and', 'to', u'wait', 'as', u'pascal', u'said', 'it', 'is', u'not', u'our', 'to', u'see', u'the', u'triumph', 'of', u'truth', u'but', 'to', u'fight', 'on', u'it', u'behalf', 'i', u'have', u'alway', u'thought', 'of', u'peter', 'as', 'an', u'apostl', 'to', u'the', u'world', 'in', u'the', u'essay', u'print', 'in', u'the', u'cathol', u'worker', u'mani', 'of', u'them', u'contain', 'an', u'outlin', 'of', u'histori', 'a', u'critic', 'of', u'histori', 'an', u'outlin', 'of', u'simpl', u'solut', u'they', u'all', u'have', 'to', 'do', u'with', u'the', u'world', u'this', u'life', u'which', 'we', u'know', u'and', u'love', u'with', u'the', u'need', 'of', u'our', u'bodi', u'for', u'food', u'cloth', u'and', u'shelter', u'his', u'philosophi', u'his', u'sociolog', u'his', u'econom', u'have', 'a', u'truli', u'religi', u'foundat', u'there', 'is', 'a', u'synthesi', 'in', u'his', u'instruct', 'to', 'us', u'all', u'and', 'it', 'is', u'not', u'just', 'to', u'use', u'the', u'catchi', u'phrase', u'that', 'he', u'list', u'his', u'quotat', u'under', u'the', u'head', u'cult', u'cultur', u'and', u'cultiv', u'father', u'furfey', 'of', u'the', u'cathol', u'univers', 'in', u'his', u'book', u'the', u'histori', 'of', u'social', u'thought', u'bring', u'out', 'in', u'his', u'first', u'chapter', u'how', u'long', u'the', u'histori', 'of', u'the', u'human', u'race', 'is', u'richarz', 'he', u'say', u'has', u'summar', u'the', u'evid', u'which', u'prove', u'that', u'num', u'year', 'is', u'the', u'absolut', u'minimum', u'and', u'then', 'he', u'goe', 'on', 'to', u'talk', u'about', u'earli', u'human', u'remain', 'of', u'the', u'pleistocen', u'period', u'which', u'began', u'from', u'num', 'to', 'a', u'million', u'year', u'ago', u'with', u'the', u'weight', 'of', u'opinion', u'inclin', u'more', u'and', u'more', 'to', u'the', u'larger', u'figur', u'thoma', u'mann', 'in', u'the', u'prelud', 'to', u'joseph', u'and', u'his', u'brother', u'say', u'that', u'expert', u'estim', u'the', u'age', 'of', u'the', u'human', u'speci', 'as', u'num', u'year', u'and', u'call', 'it', 'a', u'scant', u'reckon', 'by', u'the', u'side', 'of', u'these', u'figur', u'the', u'num', u'year', 'of', u'christian', u'seem', u'relat', 'an', u'instant', 'in', u'the', u'histori', 'of', u'the', u'world', u'with', u'this', u'fresh', u'point', 'of', u'view', u'peter', u'doe', u'not', u'find', 'it', 'at', u'all', u'extraordinari', 'to', u'expect', u'peopl', 'to', u'tri', 'to', u'begin', u'now', 'to', u'put', u'into', u'practic', u'some', 'of', u'the', u'social', u'idea', u'not', u'onli', 'of', u'the', u'new', u'testament', u'but', 'of', u'the', u'old', u'unless', 'we', u'tri', 'to', u'put', u'these', u'idea', u'into', u'practic', 'we', u'are', u'guilti', 'of', u'secular', 'so', u'ters', u'condemn', 'by', u'pope', u'pius', 'xi', u'unless', 'we', u'are', u'tri', 'to', u'put', u'the', u'social', u'idea', 'of', u'the', u'gospel', u'into', u'practic', 'we', u'are', u'not', u'show', u'our', u'love', u'for', u'our', u'neighbor', u'and', u'how', u'can', 'we', u'love', u'god', u'whom', 'we', u'have', u'not', u'seen', u'unless', 'we', u'love', u'our', u'brother', u'whom', 'we', 'do', u'see', 'as', u'saint', u'john', u'wrote', u'unless', 'we', u'are', u'put', u'these', u'social', u'idea', u'into', u'practic', u'recogn', u'the', u'correl', 'of', u'the', u'soul', u'and', u'bodi', 'we', u'are', u'use', u'religion', 'as', 'an', u'insur', u'polici', 'as', 'a', u'prop', 'as', 'a', u'comfort', 'in', u'afflict', u'and', u'not', u'onli', 'is', u'religion', u'then', u'truli', 'an', u'opiat', 'of', u'the', u'peopl', u'but', 'we', u'are', u'like', u'men', u'who', u'behold', u'our', u'face', 'in', 'a', u'glass', 'go', u'away', u'not', u'mind', 'of', u'what', u'manner', 'of', u'men', 'we', u'are', u'peter', u'doe', u'not', u'talk', u'subject', u'about', u'religion', 'he', u'bring', 'to', 'us', u'quotat', u'and', u'book', u'and', u'idea', u'that', 'by', u'stimul', u'the', u'mind', 'to', u'know', u'will', u'encourag', u'the', u'heart', 'to', u'love', u'three', u'quotat', u'from', u'the', u'first', u'letter', 'of', u'saint', u'john', u'epitom', u'peter', u'religi', u'attitud', u'for', 'me', 'no', u'man', u'hath', u'seen', u'god', 'at', u'ani', u'time', 'if', 'we', u'love', u'one', u'anoth', u'god', u'abideth', 'in', 'us', u'and', u'his', u'chariti', 'is', u'perfect', 'in', 'us', 'if', u'ani', u'man', u'say', 'i', u'love', u'god', u'and', u'hateth', u'his', u'brother', 'he', 'is', 'a', u'liar', u'for', 'he', u'that', u'loveth', u'not', u'his', u'brother', u'whom', 'he', u'seeth', u'how', u'can', 'he', u'love', u'god', u'whom', 'he', u'seeth', u'not', 'he', u'that', u'hath', u'the', u'substanc', 'of', u'this', u'world', u'and', u'shall', u'see', u'his', u'brother', 'in', u'need', u'and', u'shall', u'shut', 'up', u'his', u'heart', u'from', u'him', u'how', u'doth', u'the', u'chariti', 'of', u'god', u'abid', 'in', u'him', u'and', u'there', 'is', u'that', u'sentenc', 'of', u'saint', u'jame', 'if', 'a', u'brother', 'or', u'sister', 'be', u'nake', u'and', u'want', u'daili', u'food', u'and', u'one', 'of', u'you', u'say', 'to', u'them', 'go', 'in', u'peac', 'be', 'ye', u'warm', u'and', u'fill', u'yet', u'give', u'them', u'not', u'those', u'thing', u'that', u'are', u'necessari', u'for', u'the', u'bodi', u'what', u'shall', 'it', u'profit', u'and', 'of', u'cours', 'to', u'sum', 'it', u'all', 'up', u'there', u'are', u'those', u'nevertobeforgotten', u'word', 'of', u'our', u'lord', u'inasmuch', 'as', 'ye', u'have', u'done', 'it', u'unto', u'the', u'least', 'of', u'these', 'my', u'brethren', 'ye', u'have', u'done', 'it', u'unto', 'me', 'it', 'is', u'the', u'abov', u'quotat', u'that', u'point', u'the', u'reason', u'for', u'peter', u'preoccup', u'with', u'the', u'materi', u'detail', 'of', u'this', u'world', u'with', u'the', u'social', u'order', u'with', u'the', u'need', 'of', u'rebuild', 'as', 'he', u'say', u'within', u'the', u'shell', 'of', u'the', u'old', u'with', 'a', u'philosophi', u'which', 'is', 'so', u'old', u'that', 'it', u'look', u'like', u'new', u'and', u'this', u'preoccup', 'of', u'his', u'with', u'busi', u'with', u'econom', u'with', u'agricultur', u'with', u'labor', u'with', u'capit', u'with', u'credit', u'union', u'matern', u'guild', u'with', u'cooper', u'his', u'unceas', u'emphasi', 'on', u'the', u'fact', u'that', u'these', u'are', u'the', u'vital', u'concern', 'of', u'religion', u'have', u'led', u'peopl', 'to', u'think', 'of', u'him', 'as', 'a', u'materialist', u'lay', u'too', u'much', u'emphasi', 'on', u'the', u'materi', u'they', u'say', u'piousli', u'and', u'return', 'to', u'their', u'prayer', u'after', u'all', 'we', u'must', u'use', u'our', u'spiritu', u'weapon', 'we', u'must', u'devot', u'ourselv', 'to', u'religi', u'servic', u'and', u'all', u'these', u'thing', u'will', 'be', u'ad', u'unto', 'us', u'and', u'withdraw', u'themselv', u'keep', u'themselv', u'unspot', u'from', u'the', u'world', u'they', u'again', u'are', u'guilti', 'of', u'secular', 'of', u'use', u'religion', 'as', 'an', u'opiat', u'the', u'cathol', u'worker', u'septemb', u'num', u'num', u'summari', u'propos', 'a', u'new', u'attitud', u'toward', u'labor', u'which', u'need', 'to', 'be', u'achiev', u'through', u'the', u'educ', u'system', u'draw', u'from', u'pius', u'xii', u'and', u'peter', u'maurin', 'to', u'articul', 'a', u'mystic', 'of', u'labor', u'that', u'promot', 'a', u'whole', 'of', u'cult', u'cultur', u'and', u'cultiv', u'this', u'attitud', u'advoc', u'one', 'to', u'work', u'for', u'what', u'one', u'need', u'not', u'what', u'one', u'want', 'so', u'one', u'can', u'work', u'for', u'other', 'in', u'need', u'ddlw', u'num', 'a', u'veri', u'interest', u'articl', 'in', u'the', u'life', 'of', u'the', u'spirit', u'blackfriar', u'num', u'bloomsburi', 'rd', 'wc', u'num', u'london', 'on', u'work', u'intellig', u'parent', 'of', u'today', u'rather', u'than', u'leav', u'their', u'children', 'a', u'certain', u'sum', 'of', u'money', u'prefer', 'to', u'spend', u'their', u'substanc', 'in', u'give', u'them', 'an', u'educ', u'which', u'will', u'make', u'them', u'capabl', u'worker', u'whether', u'the', u'work', 'be', u'intellectu', 'or', u'manual', u'they', u'are', u'right', u'for', u'person', 'of', u'today', u'their', u'real', u'insur', u'consist', 'in', u'their', u'capac', u'for', u'work', u'the', u'time', u'will', u'come', u'for', u'nun', u'alsoand', 'in', 'a', u'certain', u'sens', 'it', u'has', u'alreadi', u'comewhen', u'the', u'most', u'secur', u'dowri', u'for', 'a', u'young', u'girl', u'wish', 'to', u'enter', 'a', u'convent', u'will', 'be', u'her', u'capac', u'for', u'work', u'nun', u'hand', u'are', u'alway', u'occupi', u'howev', u'for', u'the', u'most', u'part', u'their', u'work', u'has', u'been', u'almost', u'entir', u'direct', 'to', u'maintain', u'the', u'possess', 'of', u'the', u'communiti', 'in', u'good', u'order', u'poverti', u'has', u'been', u'practic', u'particular', 'by', u'mean', 'of', u'economi', 'it', u'has', u'been', u'sought', 'to', u'make', u'thing', u'last', u'darn', u'and', u'patch', u'have', u'therefor', u'been', u'multipli', u'time', u'be', u'taken', 'up', 'in', u'excess', 'of', u'what', 'a', u'more', u'ration', u'econom', u'organ', u'would', u'allow', u'those', u'who', u'desir', 'to', u'produc', u'must', u'take', u'account', 'of', u'the', u'time', u'factor', u'stteresa', 'of', u'avila', u'understood', u'this', u'when', u'she', u'said', u'that', u'she', u'want', u'her', u'nun', 'to', u'earn', u'their', u'live', 'by', u'their', u'hand', 'in', u'addit', 'to', u'live', 'on', u'the', u'alm', u'sent', u'them', 'to', u'return', 'to', 'an', u'econom', u'organ', 'of', u'convent', u'base', 'on', u'labor', 'is', u'onli', 'to', u'return', u'tostteresa', u'primit', u'plan', u'which', u'was', u'alway', u'her', u'favorit', '', u'while', u'ask', 'of', u'nun', u'this', u'effort', u'for', u'the', u'organ', 'of', u'product', u'work', u'holi', u'church', u'has', u'act', 'in', 'a', u'most', u'mother', u'way', u'be', u'anxious', u'not', u'onli', 'to', u'secur', u'their', u'materi', u'subsist', u'but', u'also', 'to', u'help', u'their', u'spiritu', u'life', 'in', u'the', u'best', u'way', u'possibl', u'therefor', u'befor', u'give', u'express', 'to', u'her', u'requir', 'as', u'set', u'forth', 'in', u'articl', u'eight', 'of', u'the', u'apostol', u'constitut', u'sponsa', u'christi', u'holi', u'church', 'in', u'the', u'explanatori', u'part', 'of', u'the', u'constitut', u'has', u'repres', 'to', u'nun', 'in', u'the', u'happiest', u'way', u'how', 'it', 'is', u'possibl', 'to', u'combin', u'this', u'more', u'intens', u'applic', 'to', u'work', u'with', u'that', u'search', u'for', u'union', u'with', u'god', u'which', 'is', u'characterist', 'of', u'their', u'contempl', u'life', u'these', u'teach', u'are', 'so', u'rich', u'and', u'may', 'be', u'foreseen', 'as', 'so', u'fruit', u'that', u'they', u'could', u'serv', 'as', 'a', u'basi', u'for', 'a', u'mystic', 'of', u'labor', u'these', u'teach', u'contain', 'an', u'appreci', 'of', u'work', u'both', u'moral', u'and', u'spiritu', u'this', u'commentari', u'show', u'not', u'onli', u'how', u'the', u'holi', u'father', 'in', u'his', u'new', u'direct', 'to', u'religi', u'order', 'is', u'meet', u'the', u'need', 'of', u'the', u'time', u'and', u'the', u'critic', 'of', u'the', u'time', u'but', u'also', u'how', u'they', u'are', u'be', u'receiv', u'there', 'is', u'meek', u'and', u'humil', u'both', 'on', u'the', u'part', 'of', u'the', u'vicar', 'of', u'christ', u'and', 'of', u'those', u'who', u'comment', 'on', u'his', u'pronounc', '', 'we', u'have', u'often', u'spoken', 'of', u'the', u'collect', u'wealth', 'of', u'order', u'and', u'the', u'individu', u'poverti', 'of', u'religi', u'who', u'often', u'have', u'not', u'enough', 'to', u'buy', u'stationeri', 'we', u'have', u'spoken', 'in', u'the', u'cathol', u'worker', 'on', u'how', u'order', 'in', u'the', u'church', u'have', u'been', 'so', u'plenti', u'suppli', u'with', u'properti', u'and', u'the', u'mean', 'of', u'improv', u'that', u'properti', 'of', u'the', u'alm', 'of', u'the', u'faith', u'and', u'how', u'littl', u'has', u'been', u'done', 'to', u'help', u'famili', u'each', u'one', 'of', u'which', 'is', 'a', u'littl', u'church', 'a', u'littl', u'communiti', u'and', u'how', u'when', u'that', u'help', 'is', u'given', u'the', u'world', u'sic', u'alm', u'and', u'pauper', u'carri', 'a', u'stigma', u'the', u'new', u'movement', 'in', u'the', u'church', u'toward', u'poverti', u'and', u'manual', u'labor', 'a', u'share', u'with', u'the', u'poor', u'and', u'the', u'destitut', u'their', u'hard', u'lot', 'in', u'life', u'and', u'side', 'by', u'side', u'with', u'this', u'the', u'persecut', 'of', u'the', u'church', u'and', u'it', u'clergi', u'and', u'religi', u'and', u'the', u'despoil', 'of', u'the', u'church', 'of', u'it', u'properti', u'bring', u'about', 'an', u'involuntari', u'precar', 'go', u'hand', 'in', u'hand', u'with', u'the', u'pronounc', 'of', u'pope', u'pius', u'xii', u'who', u'consol', u'his', u'children', u'and', 'at', u'the', u'same', u'time', u'urg', u'them', 'to', 'a', u'new', u'life', u'this', u'new', u'life', u'would', u'mean', 'an', u'intensif', 'of', u'the', u'interior', u'life', u'accent', 'on', u'communiti', u'among', u'the', u'secular', u'clergi', u'and', u'accent', 'on', u'work', u'product', u'work', 'to', u'earn', 'a', u'live', u'among', u'religi', u'accord', 'to', u'the', u'holi', u'father', u'recent', u'direct', '', u'and', u'all', u'this', u'renew', 'of', u'life', u'will', u'overflow', 'in', u'benefit', 'to', u'the', u'laiti', u'perhap', 'in', u'the', u'veri', u'teach', u'that', u'reach', u'the', u'children', 'in', u'all', u'our', u'parochi', u'school', u'the', u'teach', u'sister', u'are', u'sure', u'do', 'a', u'tremend', u'job', u'and', u'earn', u'their', u'food', 'by', u'the', u'sweat', 'of', u'their', u'brow', u'but', u'perhap', u'even', u'here', u'and', u'peter', u'maurin', u'talk', 'of', u'this', 'so', u'mani', u'time', u'there', 'is', u'need', 'a', u'great', u'chang', 'in', u'attitud', u'children', u'too', u'need', 'to', 'be', u'taught', 'a', u'mystic', 'of', u'labor', u'peter', u'maurin', u'use', 'to', u'call', 'it', 'a', u'philosophi', 'of', u'work', 'fr', u'jimmi', u'tompkin', u'use', 'to', u'say', u'that', u'all', u'work', u'should', 'be', u'consid', 'in', u'the', u'light', 'of', u'the', u'work', 'of', u'merci', 'is', u'our', u'work', u'that', 'we', u'are', u'prepar', 'to', 'do', 'in', u'life', u'help', 'to', u'feed', u'cloth', u'shelter', u'peopl', u'are', u'children', u'be', u'taught', 'a', u'rever', u'for', u'the', u'soil', u'out', 'of', u'which', u'all', u'thing', u'come', u'sinc', 'we', u'are', u'but', u'dust', u'the', u'tabl', 'we', u'work', 'at', u'the', u'food', 'we', u'eat', u'the', u'bed', 'we', u'lie', 'on', u'the', u'cover', 'on', 'it', u'all', u'come', u'from', u'the', u'soil', 'to', u'dig', 'to', u'sow', u'and', u'reap', 'to', u'build', u'and', u'construct', u'all', u'children', u'love', 'to', 'do', u'these', u'thing', 'at', u'first', u'but', 'in', u'school', u'literaci', u'take', u'first', u'place', u'and', u'read', 'is', 'no', u'longer', u'taught', u'begin', u'with', u'the', u'littl', u'offic', u'our', u'ladi', u'primer', 'i', u'rememb', u'one', 'of', 'my', u'young', u'niec', u'come', u'home', u'from', u'school', u'with', 'a', u'project', u'book', u'she', u'was', u'make', u'her', u'task', u'was', 'to', u'furnish', 'a', u'home', 'to', u'cut', u'out', u'all', u'the', u'thing', u'one', u'would', u'need', 'in', u'that', u'home', u'and', u'she', u'pore', u'over', u'magazin', u'and', u'cut', u'out', u'linoleum', u'furnitur', u'kitchen', u'set', u'parlor', u'set', u'gadget', u'and', u'had', 'a', u'love', u'time', u'do', 'it', u'and', u'all', u'the', u'while', u'standard', u'were', u'be', u'set', 'up', 'in', u'her', u'mind', u'desir', u'were', u'be', u'stimul', 'to', u'buy', u'what', u'the', u'advertis', u'present', u'and', 'to', u'get', u'the', u'job', 'to', u'get', u'the', u'money', 'to', u'buy', u'what', u'the', u'advertis', u'present', 'i', u'rememb', 'an', u'italian', 'on', u'mott', u'street', u'just', u'about', 'to', u'get', u'marri', u'his', u'wifetob', u'insist', 'on', u'throw', u'out', u'all', u'the', u'furnitur', u'that', 'he', u'had', u'from', u'his', u'mother', u'and', u'buy', 'a', u'new', u'set', u'anoth', u'neighbor', u'was', u'alway', u'discard', u'inde', u'our', u'friend', u'the', u'whelan', u'when', u'they', u'marri', u'were', u'abl', 'to', u'furnish', u'their', u'home', u'with', u'the', u'castoff', 'of', u'those', u'who', u'live', 'in', u'tenement', u'now', u'they', u'have', u'acreag', u'and', 'a', u'hous', u'built', u'with', u'their', u'own', u'hand', u'they', u'have', u'the', u'necess', 'of', u'life', u'and', u'not', u'the', u'luxuri', u'are', u'our', u'children', u'be', u'taught', u'not', u'onli', 'to', u'work', u'for', u'what', u'they', u'need', u'not', u'what', u'they', u'want', u'and', u'also', 'to', u'work', u'for', u'other', 'so', u'that', u'they', u'will', u'alway', u'have', 'a', u'surplus', 'to', u'give', u'away', u'are', u'they', u'taught', 'to', u'tith', u'themselv', 'to', u'give', u'even', u'onetenth', 'of', u'what', u'they', u'are', u'go', 'to', u'earn', 'to', u'the', u'poor', 'i', u'think', u'these', u'thing', 'as', 'i', u'see', 'my', u'grandchildren', u'go', u'back', 'to', u'school', 'at', u'home', u'becki', u'occasion', u'say', u'when', u'she', 'is', u'made', 'to', u'run', u'too', u'mani', u'errand', 'i', 'am', u'the', u'chief', u'slave', u'around', u'here', u'she', 'is', u'seven', u'and', u'the', u'eldest', 'of', u'six', u'her', u'mother', u'say', 'no', 'i', 'am', u'the', u'chief', u'slave', u'and', u'you', u'are', u'first', u'assist', u'becki', u'susi', u'eric', u'nicki', u'and', u'even', u'mari', u'elizabeth', 'at', u'the', u'age', 'of', u'two', u'tri', 'to', u'help', u'mari', u'will', u'pick', u'thing', 'up', u'off', u'the', u'floor', u'and', u'bring', u'them', 'to', u'you', u'she', u'has', u'not', u'far', 'to', u'bend', u'nicki', u'love', 'to', u'set', u'the', u'tabl', u'and', u'bring', u'out', u'basket', 'of', u'food', u'when', 'we', u'are', u'have', 'a', u'picnic', u'under', u'the', u'mulberri', u'tree', u'everybodi', u'help', u'with', u'the', u'prepar', 'of', u'food', u'how', u'they', u'all', u'love', 'to', 'go', 'to', u'the', u'garden', 'to', u'pick', u'squash', u'tomato', u'onion', u'cucumb', u'green', u'pepper', u'and', u'carrot', u'and', u'they', u'love', 'to', u'bring', u'out', u'the', u'remaind', 'to', u'the', u'num', u'rabbit', u'num', u'chicken', u'and', u'gees', 'it', 'is', u'the', u'clean', 'up', u'the', u'wash', 'of', u'dish', u'the', u'put', u'away', 'of', u'thing', u'that', u'are', u'hard', u'the', u'other', u'night', u'bob', u'reynold', u'editor', 'of', u'jubile', u'came', 'by', 'to', u'pay', 'a', u'call', 'on', u'david', 'to', u'get', u'some', u'belloc', u'pictur', u'tamar', u'and', 'i', u'had', u'forgotten', u'about', 'it', u'and', u'david', u'when', 'he', u'return', u'home', u'from', u'his', u'work', 'at', u'wolf', u'pond', u'park', 'at', u'fourthirti', u'remind', 'us', u'that', u'bob', u'was', u'expect', u'ani', u'minut', 'we', u'havent', 'a', u'thing', 'in', u'the', u'hous', u'the', u'usual', u'woman', u'reaction', u'but', u'half', 'an', u'hour', u'scurri', u'meant', 'a', u'feast', u'the', u'children', u'caught', 'a', u'chicken', u'david', u'kill', 'it', u'tamar', u'had', u'the', u'kettl', 'on', u'and', u'scald', 'it', 'so', u'the', u'feather', u'came', u'off', 'in', 'a', u'minut', u'the', u'children', u'love', 'to', 'do', u'this', 'it', u'was', u'clean', u'cut', 'up', u'dip', 'in', u'whole', u'wheat', u'flour', u'and', 'in', u'the', u'fri', u'pan', u'minut', u'after', u'becki', u'got', u'busi', 'at', u'the', u'cucumb', u'eric', u'brought', 'in', u'and', u'peel', u'and', u'slice', u'them', u'rather', u'thick', u'there', u'was', u'sweet', u'corn', u'and', u'french', u'fri', u'potato', u'take', u'onli', 'a', u'minut', u'there', u'was', u'whole', u'wheat', u'bread', u'which', u'betti', u'lou', u'had', u'made', 'at', u'peter', u'maurin', u'farm', u'the', u'children', u'and', 'i', u'had', u'our', u'feast', u'out', u'under', u'the', u'tree', u'and', u'dave', u'and', u'tamar', u'and', u'bob', u'ate', 'in', u'the', u'cool', u'and', u'quiet', u'hous', 'we', u'are', u'never', 'as', u'order', 'as', 'we', u'should', 'be', u'what', u'with', u'the', u'accumul', 'of', u'cloth', 'in', u'the', u'bedroom', 'if', u'wed', u'thought', 'of', u'compani', 'we', u'would', u'have', u'pick', 'up', u'instead', 'of', u'wash', u'blanket', u'the', u'woodwork', u'and', u'wall', u'need', u'paint', u'the', u'furnitur', 'is', u'broken', u'down', u'but', 'we', u'all', u'had', u'fun', u'the', u'children', u'includ', u'yesterday', u'eric', u'who', 'is', u'five', u'built', 'a', u'veri', u'credit', u'tabl', u'and', u'becki', u'fed', u'through', u'for', u'the', u'chicken', u'the', u'tabl', 'is', u'veri', u'weak', u'eric', u'said', u'with', u'frank', u'selfcritic', u'but', 'i', u'think', 'it', 'is', u'strong', u'enough', 'to', u'eat', u'ice', u'cream', 'on', u'how', 'to', u'work', 'to', u'build', 'up', u'the', u'habit', 'of', u'work', u'the', u'love', 'of', u'work', u'these', u'are', u'the', u'thing', u'children', u'need', 'to', 'be', u'taught', u'and', 'we', u'need', u'the', u'kind', 'of', u'school', u'where', 'it', 'is', u'be', u'taught', 'if', 'we', u'had', u'this', u'good', u'foundat', 'of', u'product', u'work', u'cultur', u'would', u'grow', u'from', 'it', u'folk', u'song', u'folk', u'art', u'folk', u'danc', u'are', u'express', 'of', u'the', u'exuber', u'and', u'joy', u'and', u'thank', u'for', u'life', u'cultiv', u'and', u'cultur', u'are', u'base', 'on', u'cult', u'which', 'is', u'our', u'holi', u'faith', 'we', u'have', u'had', u'two', u'and', u'three', u'mass', 'a', u'day', 'at', u'peter', u'maurin', u'farm', u'for', u'the', u'past', u'month', u'there', 'is', u'the', u'rosari', u'prime', u'complin', u'vesper', 'at', u'maryfarm', u'there', 'is', u'cultur', u'which', 'is', u'the', u'draw', u'which', u'littl', u'mac', u'smith', u'doe', u'and', u'all', u'the', u'children', u'love', 'to', 'do', u'and', u'the', u'read', u'and', u'the', u'listen', 'to', u'music', u'and', u'the', u'make', 'of', 'it', 'at', u'maryfarm', u'hector', u'black', u'gave', 'us', 'a', u'wonder', u'concert', u'this', u'summer', 'on', u'our', u'old', u'piano', u'which', 'he', u'tune', 'at', u'peter', u'maurin', u'farm', 'we', u'all', u'join', 'in', u'sing', u'michael', u'with', u'russian', u'song', 'fr', u'wencelaus', u'with', u'polish', u'stanley', u'with', u'lithuanian', 'fr', u'pinet', u'and', u'the', u'three', u'seminarian', u'who', u'were', u'visit', u'with', u'french', u'song', u'and', u'the', u'smith', u'children', u'with', u'calypso', u'cultiv', u'complet', u'the', u'synthesi', 'of', u'cult', u'cultur', u'and', u'cultiv', u'which', u'peter', u'maurin', u'talk', 'of', 'so', u'much', u'the', u'farm', 'at', u'newburgh', u'maryfarm', 'is', u'someth', 'to', u'boast', 'of', u'but', u'the', u'soil', 'at', u'peter', u'maurin', u'farm', 'is', u'veri', u'poor', 'fr', u'duffi', u'main', u'job', 'is', 'to', u'restor', u'it', u'fertil', u'meanwhil', u'the', u'field', u'begin', 'to', u'take', 'on', 'a', u'checkerboard', u'look', u'the', u'garden', 'is', u'yield', 'us', u'all', u'our', u'veget', u'the', u'hopi', u'indian', u'rais', u'their', u'food', u'from', u'more', u'obstin', u'land', u'and', u'they', u'induc', 'it', u'with', u'love', 'to', u'yield', u'their', u'necess', 'to', u'live', 'in', u'this', u'way', u'with', u'the', u'sacrament', 'of', u'the', u'church', u'and', u'with', 'a', u'sacrament', u'attitud', u'toward', u'the', u'materi', u'thing', 'of', u'life', u'this', 'is', u'inde', u'the', u'good', u'life', u'even', u'though', u'life', u'and', u'death', u'are', u'never', u'far', u'apart', u'they', u'sic', u'has', u'made', 'my', u'line', 'to', u'fall', 'in', u'good', u'place', 'o', 'my', u'god', u'the', u'cathol', u'worker', u'april', u'num', u'num', u'summari', u'repeat', 'p', u'maurin', u'fear', u'that', u'increas', u'state', u'regul', u'lead', 'to', u'fascism', u'and', u'undermin', u'person', u'respons', u'howev', u'agre', u'with', u'pius', 'xi', 'in', u'his', u'encycl', u'forti', u'year', u'after', u'that', u'the', u'state', u'may', u'interven', u'when', 'a', u'particular', u'group', 'is', u'threaten', u'and', 'no', u'other', u'mean', u'are', u'avail', 'to', u'them', u'ddlw', u'num', 'to', u'paraphras', 'on', u'the', u'titl', 'of', u'eugen', u'oneil', u'play', u'our', u'day', u'are', u'with', 'an', u'end', u'veri', u'much', 'so', u'and', 'it', 'is', 'to', u'explain', u'this', u'our', u'end', 'in', u'view', u'that', 'i', 'am', u'write', u'this', u'editori', u'peopl', u'come', 'to', u'the', u'worker', u'school', u'and', u'talk', u'and', u'hear', u'much', u'talk', u'about', u'the', u'encycl', u'forti', u'year', u'after', u'about', u'the', u'nra', u'and', u'how', u'far', 'it', u'goe', 'to', u'approach', u'the', u'pope', u'idea', u'about', u'intern', u'peac', u'and', u'intern', u'associ', 'of', u'work', u'men', u'and', u'industrialist', u'and', u'then', u'when', u'they', u'have', u'heard', u'all', u'this', u'from', u'the', u'lip', 'of', u'priest', u'professor', u'and', u'laymen', u'peter', u'maurin', u'rise', 'up', u'with', u'the', u'voic', 'of', u'one', u'cri', 'in', u'the', u'wilder', u'and', u'say', u'the', u'great', u'danger', 'of', u'the', u'present', u'day', 'is', u'fascism', u'and', u'the', u'tendenc', 'of', u'all', u'organ', 'is', 'to', u'lead', 'to', u'fascism', u'what', u'about', u'the', u'pope', u'and', u'quadragesimo', u'anno', u'our', u'listen', u'proclaim', u'and', u'peter', u'continu', u'his', u'warn', u'fascism', u'bewar', 'of', u'state', u'regul', u'becaus', 'it', u'lead', 'to', u'fascism', u'whether', 'it', 'is', u'the', u'state', u'regul', 'of', u'the', u'nra', 'or', u'the', u'state', u'regul', 'of', u'the', u'socialist', 'or', u'the', u'state', u'regul', u'recommend', 'by', u'pope', u'pius', 'xi', u'his', u'warn', 'is', u'the', u'same', u'bewar', 'of', u'fascism', u'peter', u'hour', 'at', u'the', u'school', 'is', u'from', u'seven', 'to', u'eight', u'the', u'speaker', u'take', u'the', u'floor', 'at', u'eight', u'and', u'the', u'listen', u'enter', u'into', u'discuss', u'with', u'the', u'speaker', u'from', u'nine', 'to', u'ten', 'or', u'tenthirti', 'or', u'eleven', u'often', 'it', 'is', u'hard', 'to', u'get', u'them', u'home', u'but', u'for', u'the', u'benefit', 'of', u'those', u'who', u'linger', u'over', u'their', u'supper', 'i', 'am', u'rehash', u'peter', u'preach', u'thus', 'in', u'view', 'of', u'the', u'general', u'disord', u'and', u'chaotic', u'condit', 'of', u'affair', u'pope', u'pius', u'issu', u'his', u'great', u'encycl', 'on', u'saint', u'franci', 'of', u'assisi', 'it', u'was', 'a', u'clarion', u'call', 'to', u'action', u'cathol', u'action', 'it', u'was', u'vital', 'of', u'tremend', u'import', u'for', u'the', u'right', 'of', u'the', u'world', u'ill', 'it', u'was', u'the', u'messag', 'of', u'christ', u'vicar', 'on', u'earth', u'the', u'messag', u'that', u'christ', u'himself', u'gave', 'in', u'the', u'sermon', 'on', u'the', u'mount', u'time', u'pass', u'and', u'the', u'condit', u'continu', 'in', u'fact', u'matter', 'go', u'from', u'bad', 'to', u'worst', 'so', u'that', 'a', u'world', u'wide', u'catastroph', u'threaten', u'the', u'result', u'was', u'the', u'encycl', u'forti', u'year', u'after', u'recal', u'pope', u'leo', u'xiii', u'encycl', 'on', u'the', u'condit', 'of', u'labor', u'which', u'was', u'also', u'disregard', u'for', u'forti', u'year', u'peter', u'messag', 'is', u'that', u'forti', u'year', u'after', u'doe', u'not', u'hold', 'up', u'the', u'ideal', 'of', u'person', u'respons', u'voic', 'by', u'the', u'encycl', 'on', u'stfranci', 'of', u'assisi', 'it', 'is', 'as', u'though', 'a', u'sad', u'and', u'weari', u'father', u'said', 'to', u'his', u'children', u'who', u'war', u'continu', 'on', u'one', u'anoth', u'veri', u'well', u'you', u'will', u'not', u'follow', u'the', u'ideal', u'for', u'the', u'sake', 'of', u'christ', 'i', u'will', u'present', 'to', u'you', u'then', u'still', u'anoth', u'program', 'of', u'action', u'organ', u'the', u'organ', 'of', u'some', 'so', u'that', u'other', u'may', 'be', u'coerc', u'therebi', u'you', u'will', u'not', u'voluntarili', u'reform', 'so', u'pressur', u'must', 'be', u'brought', 'to', u'bear', u'but', u'the', u'organ', u'held', 'up', 'as', u'best', 'by', u'pope', u'pius', 'xi', 'is', u'not', u'the', u'organ', 'of', u'the', u'labor', u'union', 'as', 'we', u'know', 'i', u'here', 'in', u'america', 'it', 'is', u'the', u'organ', 'of', u'cathol', u'workingmen', 'to', u'work', u'for', u'cathol', u'and', u'noncathol', u'alik', u'complet', u'and', u'widespread', u'organ', u'michael', u'gunn', u'ideal', 'of', u'the', u'labor', u'guild', u'approach', u'more', u'near', 'to', u'this', u'than', u'the', u'nra', u'but', u'mike', u'gunn', 'is', u'also', 'a', u'voic', 'of', u'one', u'cri', 'in', u'the', u'wilder', u'one', 'of', u'the', u'professor', u'who', u'lectur', 'us', u'was', u'point', u'out', u'how', u'utopia', u'would', u'never', u'work', u'but', u'alway', 'it', 'is', u'the', u'ideal', 'of', 'a', u'utopia', u'held', 'up', u'that', u'has', u'influenc', u'the', u'mass', u'there', 'is', u'alway', 'a', u'great', u'need', 'of', u'idealist', u'who', u'hold', 'up', u'the', u'ideal', u'rather', u'than', u'the', u'practic', u'without', u'them', u'men', u'would', u'not', u'strive', 'so', u'high', u'littl', 'by', u'littl', 'it', u'can', 'be', u'found', u'that', u'the', u'ideal', u'work', u'and', 'is', u'practic', u'and', u'then', u'peopl', u'are', u'surpris', u'the', u'perfect', u'state', 'it', 'is', 'a', u'thing', 'to', u'fight', u'for', u'christ', u'said', 'be', 'ye', u'therefor', u'perfect', u'even', 'as', u'your', u'father', 'in', u'heaven', 'is', u'perfect', 'go', 'ye', u'therefor', u'and', u'sell', u'what', u'you', u'have', u'and', u'give', 'to', u'the', u'poor', 'if', u'you', u'have', u'two', u'coat', u'take', u'one', u'and', u'give', 'it', 'to', u'your', u'brother', u'and', 'if', u'your', u'brother', u'ask', u'you', u'for', u'your', u'coat', u'give', u'him', u'your', u'cloak', u'too', u'peopl', 'do', u'not', u'scoff', 'at', u'these', u'word', u'becaus', u'they', u'are', u'the', u'word', 'of', u'christ', 'a', u'great', u'mani', u'regard', u'them', u'hopeless', u'and', u'fall', u'back', 'on', u'their', u'poor', u'human', u'they', u'admit', u'their', u'inabl', 'to', u'live', 'up', 'to', u'these', u'word', u'but', u'nevertheless', u'these', u'word', u'hard', u'word', 'go', u'down', u'through', u'the', u'age', u'and', u'through', u'them', u'mani', u'have', u'follow', u'the', u'precept', 'as', u'well', 'as', u'the', u'counsel', u'and', u'have', u'influenc', u'human', u'great', u'therebi', 'as', u'for', u'those', u'who', u'dont', u'god', u'know', u'that', 'we', u'are', u'but', u'dust', u'and', 'he', 'is', 'a', u'kind', u'and', u'tender', u'father', u'the', u'cathol', u'worker', u'stand', u'oppos', 'to', u'communism', u'social', u'and', u'fascism', u'the', u'cathol', u'worker', u'regard', u'the', u'exist', u'system', 'of', u'labor', u'union', 'as', 'a', u'poor', u'and', u'faulti', u'one', u'far', u'below', u'that', 'of', u'organ', u'describ', 'by', u'pope', u'pius', 'xi', 'in', u'his', u'encycl', u'forti', u'year', u'after', u'the', u'cathol', u'worker', u'fear', u'the', u'nra', u'inasmuch', 'as', 'it', u'may', u'lead', 'to', u'more', u'state', u'regul', u'and', u'bring', u'nearer', u'the', u'danger', 'of', u'fascism', u'the', u'cathol', u'worker', 'is', u'not', u'stand', u'for', u'the', u'cooper', u'commonwealth', u'feder', 'in', u'canada', u'for', u'that', u'veri', u'reason', u'more', u'state', u'regul', 'to', 'an', u'even', u'greater', u'degre', u'than', u'that', 'of', u'the', u'nra', 'we', u'admit', u'the', u'danger', 'of', u'these', u'palliat', 'to', u'the', u'exist', u'disord', u'but', 'we', u'see', u'their', u'danger', u'from', 'a', u'differ', u'standpoint', u'than', 'do', u'mani', u'other', u'who', u'oppos', u'them', 'as', u'be', u'the', u'offshoot', 'of', u'moscow', u'thought', 'we', u'believ', u'with', u'the', u'pope', u'that', u'whenev', u'the', u'general', u'interest', 'of', u'ani', u'particular', u'class', u'suffer', u'and', 'is', u'threaten', u'with', u'evil', u'which', u'can', 'in', 'no', u'other', u'way', 'be', u'met', u'the', u'public', u'author', u'the', u'state', u'must', u'step', 'in', 'to', u'meet', u'themif', u'within', u'the', u'wall', 'of', 'a', u'household', u'there', u'occur', u'grave', u'disturb', 'of', u'mutual', u'right', u'the', u'public', u'power', u'must', u'interfer', u'but', u'not', u'the', u'ital', u'which', u'can', 'in', 'no', u'other', u'way', 'be', u'met', u'peter', u'maurin', u'believ', u'that', u'the', u'pope', 'is', u'oppos', 'to', u'polit', u'action', u'that', 'he', u'welcom', u'the', u'dissolut', 'of', u'the', u'cathol', u'parti', 'in', u'itali', u'and', u'the', u'centrist', u'parti', 'in', u'germani', u'organ', 'of', u'cathol', u'workingmen', 'is', u'anoth', u'thing', u'entir', 'so', u'though', 'we', u'say', u'that', u'michael', u'gunn', u'more', u'near', u'approach', 'in', u'his', u'idea', u'the', u'teach', 'of', u'the', u'pope', u'than', u'doe', u'the', u'nra', 'we', u'continu', 'to', u'cling', 'to', u'the', u'ideal', 'as', u'held', 'up', 'in', u'the', u'gospel', u'and', 'in', u'the', u'encycl', 'on', u'stfranci', 'of', u'assisi', 'we', u'shall', u'not', u'reach', 'it', 'we', u'know', u'but', u'that', u'doe', u'not', u'mean', u'that', u'there', 'is', 'no', u'use', u'tri', u'the', u'cathol', u'worker', u'march', u'num', u'num', u'summari', u'discuss', u'the', u'differ', u'and', u'similar', 'of', u'the', u'worker', u'priest', u'and', u'the', u'littl', u'brother', u'explain', u'the', u'church', u'condemn', 'of', u'some', u'workerpriest', u'who', u'advoc', 'a', u'close', 'a', u'associ', u'between', u'marxism', u'and', u'catholic', u'but', 'is', u'distraught', 'at', u'the', u'church', u'inconsist', 'in', u'not', u'condemn', u'those', u'priest', u'who', u'are', u'close', u'link', 'to', u'capit', u'see', u'the', u'present', u'day', u'scandal', 'of', u'the', u'church', 'as', 'an', u'imbal', u'between', u'spiritu', u'and', u'materi', u'work', u'ddlw', u'num', u'the', u'worker', u'priest', 'of', u'franc', u'for', u'the', u'last', u'ten', u'year', u'have', u'left', u'the', u'side', 'of', u'the', u'faith', u'and', u'have', u'gone', u'after', u'the', u'lost', u'sheep', 'of', u'franc', u'they', u'have', u'been', u'do', u'what', u'jesus', u'christ', u'himself', u'told', u'them', 'to', 'do', 'in', u'their', u'great', u'love', 'of', u'god', u'and', 'of', u'their', u'brother', u'this', u'work', u'which', u'has', u'caught', u'the', u'attent', 'of', u'the', u'world', u'began', u'with', u'the', u'resist', u'movement', 'in', u'franc', u'when', u'both', u'communist', u'and', u'cathol', u'live', u'side', 'by', u'side', 'in', u'prison', u'and', u'concentr', u'camp', u'where', u'they', u'began', 'to', u'know', u'one', u'anoth', u'and', u'love', u'one', u'anoth', 'it', u'began', u'with', u'the', u'great', u'vision', 'of', 'a', u'great', u'churchman', u'cardin', u'suhard', u'whose', u'collect', u'write', u'can', 'be', u'obtain', u'from', u'the', u'fide', u'press', 'in', u'chicago', u'with', 'a', u'strong', u'cri', u'and', u'now', u'after', u'ten', u'year', u'trial', u'these', u'worker', u'priest', u'have', u'been', u'under', u'the', u'scrutini', u'and', u'critic', 'of', u'the', u'world', 'so', u'that', u'everi', u'day', u'there', 'is', u'some', u'mention', 'of', u'their', u'work', u'and', u'present', u'struggl', 'in', u'the', u'secular', u'press', 'as', u'well', 'as', u'the', u'cathol', u'the', u'critic', u'come', u'from', u'the', u'rich', u'and', u'power', u'whose', u'greed', u'and', u'wealth', u'make', u'them', u'sensit', 'to', u'the', u'critic', 'of', u'these', u'new', u'articul', u'poor', 'as', u'well', 'as', u'from', u'their', u'law', u'superior', u'the', u'present', u'cardin', 'of', u'franc', u'and', u'the', u'holi', u'see', 'in', u'rome', u'also', u'these', u'priest', u'have', u'answer', u'back', u'have', u'cri', u'out', u'with', 'a', u'strong', u'cri', u'and', u'tear', 'so', u'that', u'the', u'world', u'has', u'heard', u'and', 'is', u'wonder', 'if', u'this', 'is', u'go', 'to', 'be', u'anoth', u'case', 'of', u'the', u'crucifixion', 'of', u'the', u'good', u'the', u'failur', 'of', u'the', u'cross', 'or', 'of', u'disobedi', 'in', u'the', u'church', u'anoth', u'action', u'francais', u'condit', 'on', u'februari', u'num', u'the', u'cardin', 'of', u'pari', u'refus', u'the', u'appeal', 'of', u'two', u'third', 'of', u'the', u'worker', u'priest', u'and', u'insist', u'that', u'the', u'mission', u'among', u'the', u'worker', 'be', u'limit', 'by', u'the', u'rule', u'laid', u'down', 'by', u'rome', u'instead', 'of', u'work', u'eight', u'hour', 'a', u'day', u'they', u'were', u'permit', u'onli', 'to', u'work', u'four', u'instead', 'of', u'live', u'alon', u'they', u'were', 'to', u'live', u'with', u'their', u'fellow', u'they', u'were', u'not', 'to', u'commit', u'themselv', 'to', u'polit', u'action', 'or', 'to', u'membership', 'in', u'trade', u'union', u'littl', u'brother', u'there', u'has', u'been', 'no', u'mention', 'in', u'all', u'this', u'controversi', 'of', u'the', u'status', 'of', u'the', u'littl', u'brother', 'of', u'jesus', 'an', u'outgrowth', 'of', u'the', u'work', 'of', u'father', u'charl', 'de', u'foucauld', u'who', u'was', u'kill', 'by', u'the', u'arab', 'in', u'the', u'desert', 'in', u'num', u'and', u'who', u'had', 'no', u'postul', 'or', u'novic', u'for', u'his', u'propos', u'order', u'dure', u'all', u'his', u'life', u'the', u'littl', u'brother', u'now', u'number', u'num', u'two', u'third', u'will', 'be', u'priest', u'and', u'are', u'scatter', u'all', u'over', u'the', u'world', u'and', u'are', u'still', u'permit', 'to', u'work', 'a', u'full', u'eighthour', u'day', 'in', u'the', u'mine', 'on', u'the', u'dock', 'in', u'the', u'jungl', u'howev', u'they', u'live', 'in', u'communiti', 'of', u'two', 'or', u'three', u'and', u'have', u'the', u'bless', u'sacrament', 'in', u'their', u'hut', u'and', u'shanti', u'town', u'and', u'slum', u'dwell', u'wherev', u'they', u'are', u'there', u'are', u'also', u'num', u'littl', u'sister', u'who', u'earn', u'their', u'live', 'in', u'the', u'same', u'way', u'father', u'general', u'dure', u'the', u'last', u'month', 'we', u'were', u'veri', u'fortun', 'in', u'have', u'the', u'father', u'general', 'of', u'the', u'order', u'who', u'with', u'two', u'other', u'found', u'this', u'great', u'new', u'apostol', 'in', u'num', u'who', u'spoke', 'to', 'us', u'for', u'four', u'hour', u'about', u'their', u'work', u'and', u'show', 'us', u'slide', 'of', u'the', u'way', u'they', u'live', u'all', u'over', u'the', u'world', u'sinc', u'the', u'controversi', u'over', u'the', u'worker', u'priest', u'mission', 'in', u'pari', u'and', u'throughout', u'franc', u'has', u'start', u'the', u'littl', u'brother', 'of', u'jesus', u'have', u'receiv', u'word', u'from', u'rome', u'that', u'they', u'can', u'continu', 'to', u'work', u'eight', u'hour', 'a', u'day', u'and', u'live', 'as', u'they', u'are', u'do', u'there', u'are', u'even', u'two', 'of', u'their', u'number', u'live', u'and', u'work', 'on', 'a', u'collect', u'farm', 'in', u'palestin', u'their', u'mission', 'is', 'in', u'the', u'entir', u'world', u'not', u'just', 'in', u'franc', u'and', u'there', u'has', u'never', u'been', 'a', u'suggest', 'of', u'polit', u'activ', 'in', u'their', u'work', u'live', 'in', u'communiti', 'as', 'to', u'live', 'in', u'communiti', u'two', u'year', u'ago', 'an', u'encycl', u'men', u'nostra', u'was', u'publish', 'by', u'the', u'holi', u'see', u'and', u'encycl', u'come', u'direct', u'from', u'the', u'holi', u'father', u'show', u'the', u'mind', 'of', u'the', u'church', 'as', 'to', u'how', u'priest', u'should', u'live', u'and', u'work', u'togeth', u'there', u'has', u'been', u'littl', u'attent', u'paid', 'to', u'this', u'encycl', u'the', u'unfortun', u'aspect', 'of', u'the', u'controversi', u'over', u'the', u'priest', u'worker', 'is', u'that', 'it', 'is', u'be', u'use', 'by', u'the', u'enemi', 'of', u'the', u'church', 'to', u'belabor', u'the', u'church', u'blackfriar', u'the', u'best', u'resum', 'of', u'the', u'situat', 'is', u'made', 'by', u'father', u'john', u'fitzsimmon', 'in', u'the', u'februari', u'issu', 'of', u'blackfriar', u'still', u'our', u'favorit', u'dominican', u'public', u'despit', 'a', u'littl', u'acrimoni', u'disput', 'a', u'few', u'month', u'ago', u'this', u'scholar', u'articl', u'sum', 'up', u'the', u'exact', u'reason', u'whi', u'the', u'issu', 'of', u'the', u'priest', u'workmen', u'has', 'to', 'be', u'taken', 'up', u'cardin', u'salieg', u'has', u'said', u'there', u'was', 'a', u'yield', 'on', u'the', u'part', 'of', u'the', u'priest', 'to', u'the', u'temptat', 'to', u'believ', u'that', u'his', u'essenti', u'function', 'is', 'to', u'take', u'the', u'place', 'of', u'the', u'lay', u'leader', 'in', u'the', u'worker', u'fight', 'in', u'the', 'us', u'priest', u'are', 'in', u'mani', u'case', u'do', u'the', u'work', u'laymen', u'could', 'do', u'such', 'as', u'teach', u'the', u'articl', u'goe', 'on', 'to', u'state', u'perhap', u'the', u'most', u'disturb', u'factor', 'of', u'all', u'was', u'the', u'attempt', 'to', u'construct', 'an', u'ideolog', u'and', u'theolog', u'background', 'to', u'justifi', u'even', u'more', u'extrem', u'posit', u'be', u'taken', 'by', u'the', u'priest', u'worker', u'and', u'their', u'follow', u'this', u'group', u'led', 'by', 'fr', u'montuclard', u'has', u'been', u'the', u'spearhead', 'of', u'this', u'attempt', u'and', u'the', u'last', u'two', u'year', u'has', u'seen', 'a', u'progress', u'condemn', 'of', u'the', u'church', 'of', u'their', u'view', u'their', u'argument', u'can', 'be', u'reduc', 'to', u'three', u'head', u'num', u'the', u'onli', u'philosophi', u'which', 'is', u'imman', 'in', u'the', u'worker', u'movement', 'is', u'marxism', u'num', 'a', u'distinct', u'must', 'be', u'made', u'between', u'marxist', u'moral', u'and', u'it', u'atheismth', u'former', u'can', 'be', u'accept', u'even', u'the', u'latter', u'can', u'contribut', 'to', u'human', u'progress', u'num', 'a', u'clear', u'distinct', u'must', 'be', u'drawn', u'between', u'the', u'cathol', u'faith', u'and', u'tempor', u'action', u'marxism', 'is', u'the', u'scienc', 'of', u'the', u'liber', 'of', u'the', u'proletariat', u'and', 'as', u'such', u'doe', u'not', u'conflict', u'with', u'the', u'faith', u'this', u'view', u'was', u'set', u'out', 'in', 'a', u'public', 'of', u'jeuness', 'de', u'leglisewhich', u'contain', u'may', u'true', u'insight', u'but', u'upheld', u'this', u'two', u'phase', u'idea', 'of', u'social', u'reform', u'this', u'movement', u'was', u'condemn', 'by', u'the', u'assembl', 'of', u'cardin', u'and', u'archbishop', 'in', u'octob', u'num', u'while', 'it', u'would', 'be', u'erron', 'to', u'suggest', u'ani', u'close', u'associ', u'between', u'the', u'priest', u'worker', u'and', u'the', u'doctrin', u'error', 'of', u'jeuness', 'de', u'leglis', 'it', 'is', u'nevertheless', u'true', u'that', u'mani', 'of', u'their', u'defend', u'their', u'worst', u'enemi', u'their', u'friend', u'did', u'use', u'such', u'argument', 'as', u'these', 'it', u'will', 'be', u'seen', u'that', u'the', u'issu', 'is', u'not', u'too', u'clear', 'as', 'to', u'whether', 'or', u'not', u'ani', 'of', u'the', u'priest', u'worker', u'actual', u'did', u'embrac', u'marxism', 'as', 'a', u'solut', u'though', u'they', u'went', u'along', u'with', u'them', 'in', u'the', u'union', u'and', u'polit', u'action', u'and', u'their', u'work', u'for', u'peac', u'agreement', u'while', 'we', 'do', u'not', u'agre', u'with', u'their', u'emphasi', 'on', u'polit', u'action', u'and', u'peac', u'action', u'they', u'are', 'no', u'more', u'pacifist', u'than', u'the', u'communist', u'are', 'we', u'still', 'do', u'not', u'see', u'whi', u'there', u'should', 'be', u'such', 'a', u'furor', 'in', u'the', u'church', u'while', u'the', u'great', u'mass', 'of', u'priest', 'of', u'the', u'church', 'go', u'along', u'wholeheart', u'with', u'capit', u'which', u'count', 'de', 'la', u'torr', 'in', u'osservator', u'romano', u'call', 'a', u'cancer', 'on', u'the', u'social', u'bodi', 'of', u'the', u'church', u'and', u'wors', u'than', u'communism', 'we', u'hear', 'of', u'plenti', 'of', u'election', u'and', u'polit', u'action', u'amongst', u'priest', u'when', 'it', u'come', 'to', u'legisl', u'about', u'bingo', 'or', u'get', u'our', u'share', 'of', u'state', u'benefit', u'for', u'our', u'school', u'and', u'tax', u'exempt', u'but', u'where', u'are', u'the', u'priest', u'cri', u'out', u'for', u'the', u'worker', 'in', u'time', 'of', u'strike', u'cri', u'out', u'for', u'the', u'poor', u'live', u'among', u'them', 'in', u'their', u'tenement', u'wast', u'last', u'month', u'one', 'of', u'our', u'worker', u'wash', u'dish', 'in', 'a', u'monasteri', u'for', u'some', u'day', u'and', u'wit', u'the', u'thick', u'steak', u'chop', u'roast', u'which', u'were', u'serv', u'twice', 'a', u'day', 'to', u'the', u'fifteen', 'or', 'so', u'member', 'of', u'the', u'communiti', u'what', u'was', u'left', 'on', u'the', u'platter', 'as', u'well', 'as', 'on', u'the', u'plate', u'was', u'thrown', u'into', u'the', u'garbag', u'and', u'the', u'men', 'at', u'the', u'door', u'the', u'ambassador', 'of', u'christ', u'were', u'turn', u'away', u'harsh', u'first', 'by', u'the', u'housekeep', u'and', u'then', 'by', u'the', u'priest', 'at', u'the', u'front', u'door', u'who', u'then', u'came', u'out', 'to', u'the', u'kitchen', u'reproach', u'her', u'for', u'not', u'have', u'gotten', u'rid', 'of', u'the', u'bum', u'for', u'good', u'and', u'all', 'in', u'the', u'first', u'place', u'are', u'the', u'poor', u'the', u'first', u'children', 'of', u'the', u'church', 'or', u'the', u'rich', u'are', u'the', u'worker', u'more', u'like', u'jesus', 'of', u'nazareth', 'or', u'are', u'the', u'industrialist', u'the', u'absente', u'landlord', u'christ', u'came', u'for', u'the', u'sinner', 'he', u'told', 'us', 'so', u'that', u'must', 'go', u'for', u'boweri', u'sinner', u'too', u'clarif', 'as', u'far', 'as', 'we', 'of', u'the', u'cathol', u'worker', u'are', u'concern', u'this', u'controversi', 'is', u'all', 'to', u'the', u'good', 'in', u'that', 'it', u'will', u'perhap', u'through', u'much', u'discuss', u'and', u'suffer', u'bring', u'about', u'clarif', 'of', u'thought', 'on', u'the', u'subject', 'of', u'manual', u'labor', u'and', u'voluntari', u'poverti', 'as', u'essenti', u'mean', 'in', u'bring', u'forth', u'good', u'fruit', u'there', u'are', u'mani', u'aspect', 'of', u'the', u'ageold', u'marymartha', u'disput', 'in', u'this', u'turmoil', u'the', u'holi', u'see', 'is', u'wish', 'to', u'stress', u'the', u'primaci', 'of', u'the', u'spiritu', u'the', u'necess', 'of', u'emphasi', u'be', u'place', 'on', u'the', u'sacrament', 'as', u'mean', 'of', u'grace', u'rather', u'than', 'on', u'the', u'human', u'work', u'and', u'suffer', 'of', u'these', u'nobl', u'priest', u'the', u'great', u'scandal', 'of', u'the', u'age', 'is', u'that', u'those', u'without', u'the', u'sacrament', u'are', 'so', u'often', u'superior', 'in', u'chariti', u'courag', u'even', u'lay', u'down', u'their', u'live', u'for', u'their', u'brother', 'to', u'the', u'practic', u'cathol', u'who', u'partak', 'of', u'the', u'sacrament', 'of', u'penanc', u'and', u'holi', u'eucharist', u'and', u'then', u'stand', 'by', u'while', u'his', u'brother', 'is', u'exploit', u'starv', u'beaten', u'and', u'goe', 'on', u'live', u'his', u'bourgeoi', u'life', u'his', u'whole', u'work', u'be', 'to', u'maintain', u'his', u'standard', 'of', u'live', u'and', u'neglect', u'the', u'one', u'thing', u'need', u'love', 'of', u'god', u'and', u'brother', 'in', u'the', u'union', u'field', u'even', 'in', u'this', u'countri', 'it', u'has', u'been', u'the', u'union', u'organ', u'often', u'the', u'communist', u'who', u'has', u'risk', u'jail', u'and', u'beat', 'to', u'organ', u'textil', u'worker', u'and', u'migrant', u'worker', u'while', u'the', u'cathol', u'too', u'often', u'stand', 'by', u'and', u'accept', u'the', u'benefit', 'of', u'the', u'union', u'and', u'doe', u'not', u'earn', u'them', u'both', u'mauriac', u'and', u'maritain', u'have', u'said', u'that', 'he', u'who', u'love', u'his', u'brother', u'and', u'work', u'for', u'justic', 'is', u'work', u'for', u'christ', u'even', u'though', 'he', u'deni', u'him', u'that', 'is', u'deni', u'him', 'as', 'he', u'see', u'him', 'in', u'the', u'nomin', u'christian', u'perhap', u'they', u'accept', u'him', 'on', u'the', u'cross', u'where', 'he', u'took', u'our', u'sin', u'upon', u'himself', u'what', 'a', u'grace', 'to', u'recogn', u'him', 'so', u'with', u'the', u'infinit', u'varieti', 'of', u'work', 'in', u'the', u'church', 'to', u'some', u'one', 'of', u'which', u'each', u'one', 'of', 'us', u'has', u'his', u'vocat', u'though', 'we', u'may', u'turn', u'away', u'and', u'refus', 'it', u'there', 'is', 'of', u'cours', 'a', u'great', u'contrast', u'between', u'the', u'work', 'of', u'the', u'littl', u'brother', 'of', u'jesus', u'and', u'that', 'of', u'the', u'priest', u'worker', u'bless', u'them', u'all', u'both', u'are', u'do', u'tremend', u'work', 'we', u'who', u'have', u'work', u'for', u'the', u'same', u'length', 'of', u'time', 'as', u'the', u'littl', u'brother', u'sinc', u'num', u'can', u'see', u'that', 'no', u'point', u'four', u'program', 'no', u'technolog', u'advanc', 'no', u'pure', u'materi', u'mean', u'are', u'go', 'to', u'remedi', u'the', u'diseas', 'of', u'today', 'on', u'the', u'other', u'hand', u'see', u'too', 'as', 'we', 'do', u'the', u'slum', 'of', u'harlem', u'the', u'east', u'side', u'the', u'pervers', 'of', u'the', u'mass', 'by', u'poverti', u'and', u'propaganda', 'we', u'also', u'realiz', u'that', 'we', u'have', 'to', u'make', u'the', u'kind', 'of', u'social', u'order', u'where', 'it', 'is', u'easier', u'for', u'men', 'to', 'be', u'good', u'even', u'two', 'or', u'three', u'littl', u'sister', u'and', u'the', u'littl', u'brother', u'live', 'in', u'shanti', u'town', u'can', u'scarc', u'realiz', u'the', u'despair', 'of', u'the', u'mother', u'and', u'father', 'of', u'eight', 'or', u'nine', u'children', u'live', 'in', u'two', u'room', 'in', u'danger', 'of', u'dispossess', u'even', u'there', u'surround', 'by', u'the', u'filth', 'of', u'hall', u'and', u'areaway', u'and', u'alley', u'rat', u'and', u'vermin', u'seep', u'plumb', u'stench', u'and', u'damp', u'cold', u'and', u'diseas', 'of', u'mind', u'and', u'bodi', u'the', u'depth', 'of', u'the', u'suffer', 'of', u'the', u'world', 'is', u'measureless', 'a', u'bottomless', u'abyss', u'and', u'our', u'onli', u'approach', 'to', 'it', 'is', u'through', u'the', u'dark', u'night', 'of', u'the', u'soul', 'a', u'tast', 'of', u'which', u'the', u'priest', u'workmen', 'of', u'franc', u'are', u'now', u'have', u'god', u'bless', u'them', u'all', u'america', u'num', u'march', u'num', u'num', u'num', u'summari', u'describ', u'her', u'travel', u'with', u'the', u'sharecropp', u'and', u'the', u'situat', u'with', u'which', u'they', u'are', u'face', u'union', u'tri', 'to', u'organ', u'but', u'planter', u'violent', u'break', 'up', u'meet', u'and', u'evict', u'those', u'who', u'particip', u'depict', u'the', u'condit', 'of', u'tent', u'coloni', u'and', u'sick', u'that', u'exist', u'among', u'those', u'who', u'live', u'there', u'advoc', u'distribut', 'of', u'land', u'and', u'farm', u'cooper', u'ddlw', u'num', u'reprint', u'from', u'america', u'march', u'num', u'num', u'with', u'permiss', 'of', u'america', u'press', u'inc', u'cnum', u'for', u'subscript', u'inform', u'call', u'num', 'or', u'visit', u'http', u'wwwamericamagazineorg', 'it', u'was', u'seventeen', u'abov', u'zero', u'when', 'we', u'start', u'out', u'this', u'morn', u'with', 'a', u'carload', 'of', u'flour', u'meal', u'lard', u'sugar', 'i', u'coffe', u'and', u'soap', u'the', u'car', u'was', 'so', u'full', 'in', u'the', u'back', u'that', u'mari', u'pierc', u'the', u'color', u'girl', u'who', 'is', 'on', u'the', u'execut', u'committe', 'of', u'the', u'southern', u'tenant', u'farmer', u'union', u'and', 'i', u'were', u'cram', u'togeth', u'warm', 'on', u'the', u'back', u'seat', u'but', u'our', u'feet', u'were', u'ici', 'it', u'was', u'not', 'a', u'heat', u'car', u'and', u'when', 'we', u'stop', 'at', 'a', u'gas', u'station', u'where', u'there', u'was', u'also', 'a', u'dri', u'good', u'store', 'we', u'went', 'in', 'to', u'buy', 'an', u'extra', u'pair', 'of', u'sock', u'they', u'had', u'onli', u'cotton', u'one', u'but', u'they', u'help', 'i', u'bought', u'some', u'candi', u'and', u'mari', u'want', 'to', u'know', u'whether', 'i', u'had', 'an', u'extra', u'nickel', u'for', u'some', u'snuff', u'she', u'and', u'her', u'husband', u'were', u'sharecropp', u'and', u'they', u'had', u'been', u'evict', 'on', u'account', 'of', u'their', u'work', u'for', u'the', u'union', u'and', u'now', u'she', u'was', 'on', u'her', u'way', u'home', 'to', u'her', u'husband', u'peopl', u'where', u'there', u'were', u'three', u'other', u'evict', u'famili', u'live', 'in', u'the', u'two', u'room', u'cabin', u'but', u'they', u'were', u'comfort', u'off', u'peoplether', u'was', u'timber', 'on', u'the', u'littl', u'piec', 'of', u'land', u'they', u'rent', u'from', u'the', u'govern', u'and', u'they', u'could', u'keep', u'warm', 'we', u'had', u'set', u'out', u'from', u'the', u'headquart', 'of', u'the', u'union', u'which', u'was', u'just', u'across', u'the', u'river', 'in', u'memphi', u'where', 'it', u'was', u'safer', 'h', 'l', u'mitchel', u'execut', u'secretari', 'of', u'the', u'union', u'who', u'with', 'j', 'r', u'butler', u'had', u'start', u'the', u'work', 'of', u'organ', u'two', u'year', u'befor', u'had', u'live', 'in', u'tyronza', u'arkansa', u'but', u'his', u'life', u'wasnt', u'safe', u'there', u'what', u'with', u'meet', u'be', u'raid', u'union', u'member', u'shot', 'in', u'the', u'back', u'and', u'then', u'arrest', u'for', u'disturb', u'the', u'peac', 'it', u'wasnt', u'healthi', 'to', u'keep', u'the', u'union', u'headquart', 'in', u'arkansa', u'especi', u'when', u'the', u'union', u'membership', u'grew', u'from', u'seventeen', 'to', u'num', 'so', u'they', u'had', u'move', 'to', u'memphi', u'the', u'offic', 'of', u'the', u'union', 'on', u'the', u'outskirt', 'of', u'town', u'where', u'the', u'rent', 'is', u'cheap', u'compris', u'two', u'room', 'in', u'the', u'room', u'where', 'i', 'am', u'write', u'there', 'is', 'a', u'tabl', u'and', u'two', u'chair', u'and', u'shelv', u'pile', u'with', u'literatur', u'and', u'paper', u'for', u'they', u'are', u'publish', 'a', u'month', u'call', u'the', u'sharecropp', u'voicearound', u'the', u'wall', u'there', u'are', u'crude', u'paint', u'sign', u'woe', u'unto', u'them', u'that', u'join', u'hous', 'to', u'hous', u'that', u'lay', u'field', 'to', u'field', u'till', u'there', 'be', 'no', u'room', u'and', 'ye', 'be', u'made', 'to', u'dwell', u'alon', 'in', u'the', u'midst', 'of', u'the', u'land', u'what', u'mean', 'ye', u'that', 'ye', u'crush', 'my', u'peopl', u'and', u'grind', u'the', u'face', 'of', u'the', u'poor', '', u'land', u'for', u'the', u'landless', '', u'and', u'landless', u'inde', u'these', u'peopl', u'are', u'for', u'although', u'most', u'folk', u'now', u'realiz', u'that', u'there', u'are', u'eleven', u'million', u'unemploy', 'in', u'the', u'citi', u'they', 'do', u'not', u'know', u'that', u'there', u'are', u'eight', u'million', u'dispossess', u'through', u'the', u'south', u'accord', 'to', u'figur', u'publish', 'by', u'the', u'univers', 'of', u'north', u'carolina', u'press', u'mari', u'had', u'come', u'into', u'town', u'for', u'some', u'meet', u'and', 'on', u'the', u'way', u'out', 'to', u'bring', u'relief', 'to', u'dispossess', u'famili', 'we', u'were', u'go', 'to', u'drop', u'her', u'off', 'at', u'her', u'home', u'the', u'road', u'was', u'fright', u'and', 'it', u'was', u'onli', u'becaus', 'we', u'were', u'wedg', 'in', u'that', 'we', u'were', u'save', u'from', u'be', u'bruis', 'by', u'the', u'jolt', 'we', u'receiv', 'we', u'skid', u'through', u'and', u'over', u'the', u'frozen', u'rut', u'for', u'mile', u'and', u'after', 'a', u'while', 'we', u'cut', u'across', u'field', u'where', u'not', 'a', u'track', 'of', 'a', u'road', u'was', u'visibl', 'we', u'pass', u'some', u'negro', u'with', 'a', u'sack', 'of', u'chicken', u'over', u'their', u'shoulder', 'on', u'their', u'way', 'to', u'sell', u'them', u'they', u'get', u'ten', u'cent', 'a', u'pound', u'mari', u'explain', '', u'good', u'fri', u'chicken', u'too', u'theyr', u'all', u'sell', u'whatev', u'they', u'got', u'cant', u'afford', 'to', u'eat', u'them', u'themselv', u'lot', 'of', u'folk', u'havent', u'even', u'chicken', 'to', u'sell', '', u'mari', u'hous', u'was', u'right', 'on', u'the', u'edg', 'of', u'the', u'wood', u'and', 'we', u'stop', 'to', u'let', u'her', u'out', u'and', 'we', u'too', u'went', 'in', 'to', u'warm', u'ourselv', 'by', u'the', u'stove', u'the', u'room', u'was', u'line', u'with', u'newspap', u'there', u'was', u'one', u'high', u'bed', u'made', 'up', u'neat', u'and', 'a', u'few', u'chair', u'around', u'the', u'stove', 'it', u'was', 'no', u'better', u'than', 'a', u'shack', u'through', u'which', u'the', u'wind', u'tore', u'and', u'nag', 'at', u'the', u'loosehung', u'door', u'everyon', u'came', 'in', 'to', u'say', u'hello', u'and', u'one', 'of', u'the', u'evict', u'men', u'told', 'of', u'tri', 'to', u'get', u'relief', u'they', u'told', 'us', 'to', 'go', u'out', u'and', u'kill', u'rabbit', 'he', u'said', u'until', u'march', u'and', u'then', u'the', u'plantersd', u'take', u'care', 'of', 'us', 'so', u'they', u'said', u'hoover', u'hog', 'we', u'use', 'to', u'call', u'them', u'rabbit', u'they', u'aint', u'ani', u'now', u'show', u'themselv', u'when', u'roosevelt', u'come', 'in', u'they', u'said', 'it', u'was', u'against', u'the', u'law', 'to', u'kill', u'rabbit', u'just', u'ketch', 'em', u'milk', 'em', u'and', u'leav', 'em', u'for', u'the', u'next', u'fellow', u'and', u'the', u'farmer', u'who', u'propound', u'this', u'one', u'laugh', u'uproari', u'that', u'laughter', u'sound', u'strang', 'to', 'me', u'condit', u'beinga', u'they', u'were', u'but', 'i', u'had', u'seen', u'noth', u'yet', 'i', u'was', 'to', u'see', u'more', u'laugh', 'as', 'we', u'went', u'through', u'the', u'day', 'a', u'hard', u'desper', u'laughter', u'but', 'it', u'was', u'laughter', u'one', 'of', u'the', u'negro', u'boy', u'stay', 'on', u'the', u'run', u'board', u'and', u'direct', 'us', u'through', u'field', u'until', 'we', u'struck', u'anoth', u'littl', u'road', 'as', u'rut', u'and', u'danger', 'as', u'the', u'last', u'the', u'field', u'had', u'been', u'better', 'it', u'was', u'some', u'eight', u'mile', u'along', u'this', u'road', u'until', 'we', u'reach', u'earl', u'where', u'most', u'troubl', u'was', u'late', 'we', u'pass', u'the', u'holi', u'church', 'on', u'the', u'outskirt', u'where', u'the', u'meet', 'of', u'five', u'hundr', u'sharecropp', u'and', u'tenant', u'had', u'been', u'broken', 'up', u'some', u'week', u'befor', u'the', u'planter', u'had', u'come', u'broken', 'in', u'all', u'the', u'window', u'beaten', 'up', u'the', u'men', u'women', u'and', u'children', 'on', u'the', u'way', u'from', u'one', 'of', u'those', u'meet', u'two', 'of', u'the', u'men', u'had', u'been', u'shot', 'in', u'the', u'back', u'then', u'cart', u'off', 'to', u'jail', u'the', u'whole', u'town', u'had', u'been', u'arous', u'and', u'had', u'want', 'to', u'hold', u'anoth', u'meet', u'immedi', 'an', u'arm', u'meet', u'that', u'would', 'be', u'prepar', u'for', 'an', u'attack', u'but', u'the', u'union', 'is', u'oppos', 'to', u'violenc', u'that', u'whi', u'there', u'are', 'no', u'meet', u'be', u'held', u'right', u'now', u'except', u'back', 'in', u'the', u'wood', 'in', u'cabin', u'and', u'barn', u'behind', u'lock', u'door', u'with', u'guard', u'station', u'out', u'along', u'the', u'road', 'to', u'warn', u'them', 'of', u'the', u'come', 'of', u'the', u'planter', u'strang', u'enough', u'the', u'american', u'legion', 'in', u'that', u'town', 'is', u'all', u'with', u'the', u'union', u'the', u'small', u'town', u'want', u'the', u'union', u'want', u'justic', u'for', u'the', u'cropper', u'want', u'some', u'small', u'measur', 'of', u'freedom', u'for', u'the', u'farmer', 'as', 'it', 'is', u'now', u'the', u'plantat', u'owner', u'have', u'compani', u'store', u'and', u'furnish', u'the', u'tenant', u'dure', u'the', u'season', 'so', u'that', 'no', u'busi', u'ever', u'come', u'the', u'way', 'of', u'the', u'town', u'six', u'mile', u'out', 'of', u'earl', 'we', u'visit', u'anoth', u'littl', u'church', u'which', u'was', u'not', u'much', u'more', u'than', 'a', u'shed', u'you', u'could', u'not', u'even', u'dignifi', 'it', 'by', u'the', u'name', u'barn', 'in', u'this', u'church', u'five', u'famili', u'were', u'live', u'with', u'all', u'their', u'belong', u'they', u'too', u'had', u'been', u'evict', u'put', u'out', 'on', u'the', u'road', u'the', u'month', u'befor', u'the', u'men', u'were', u'out', u'when', 'we', u'came', u'forag', u'for', u'wood', u'but', u'the', u'church', u'was', u'pack', u'nevertheless', u'with', u'human', 'by', u'the', u'door', u'one', u'young', u'woman', u'was', u'wash', u'cloth', 'it', u'was', u'one', 'of', u'the', u'saddest', u'sight', 'to', u'see', u'this', u'attempt', 'at', u'cleanli', 'in', u'this', u'hovel', u'unfit', u'even', u'for', u'anim', u'furnitur', u'was', u'stack', u'everywher', u'and', u'bed', u'took', 'up', u'everi', u'avail', u'inch', 'of', u'space', u'except', u'around', u'the', u'fire', 'jr', u'butler', 'an', u'arkansan', u'all', u'his', u'life', u'presid', 'of', u'the', u'union', u'was', u'the', u'one', u'who', u'was', u'drive', u'around', u'distribut', u'the', u'relief', u'there', u'was', u'also', u'blain', u'treadway', 'a', u'printer', 'on', u'the', u'scripp', u'howard', u'paper', 'at', u'memphi', u'some', u'time', u'back', 'an', u'automobil', u'kill', 'a', u'hog', u'butler', u'said', '', u'and', 'of', u'cours', u'the', u'driver', u'didnt', u'stop', u'the', u'state', u'ranger', u'took', 'it', u'off', u'into', u'the', u'wood', u'and', 'it', u'was', u'rot', u'there', 'of', u'cours', 'he', u'could', u'have', u'given', 'it', 'to', u'the', u'poor', u'famili', 'in', u'the', u'first', u'place', u'but', 'he', u'didnt', u'this', u'crowd', u'found', 'it', u'when', 'it', u'was', u'all', u'bloat', u'and', u'swollen', u'and', u'stiff', u'and', u'they', u'ate', 'it', u'hunger', u'grip', u'them', u'everywher', '', 'it', u'wasnt', u'until', u'late', 'in', u'the', u'afternoon', u'that', 'we', u'reach', u'the', u'worst', u'place', 'of', u'all', u'just', u'outsid', u'parkin', u'arkansa', u'there', u'drawn', 'up', u'along', u'the', u'road', u'was', 'a', u'tent', u'coloni', u'which', u'hous', u'num', u'peopl', u'four', u'infant', u'among', u'them', u'and', u'god', u'know', u'how', u'mani', u'children', u'three', u'littl', u'girl', u'giggl', u'and', u'laugh', u'with', u'their', u'arm', u'around', u'each', u'other', u'while', 'we', u'talk', 'to', u'this', u'evict', u'crowd', 'of', u'sharecropp', u'onli', u'one', 'of', u'them', u'had', 'on', 'a', u'sweater', u'and', u'the', u'heel', u'and', u'toe', 'of', u'all', 'of', u'them', u'were', u'come', u'out', 'of', u'their', u'shoe', u'and', 'it', u'was', u'seventeen', u'abov', u'their', u'giggl', u'start', u'them', u'cough', u'and', u'woke', u'one', 'of', u'the', u'babi', u'who', u'cri', u'fret', u'weak', 'we', u'aint', u'got', u'much', 'to', u'complain', 'of', u'about', u'health', u'one', u'man', u'said', u'all', u'the', u'childern', u'got', u'cold', u'and', u'two', 'of', u'the', u'young', u'one', u'got', u'burn', u'against', u'the', u'stove', u'pretti', u'bad', u'too', u'mine', u'was', u'one', u'hes', u'four', u'year', u'old', u'one', u'man', u'die', u'last', u'night', 'he', u'was', u'taken', u'desol', u'sick', u'out', u'here', 'he', u'die', u'last', u'night', u'between', u'three', u'and', u'four', 'he', u'shore', u'suffer', u'afor', 'he', u'die', u'the', u'littl', u'tent', u'where', 'we', u'stood', 'on', u'the', u'frozen', u'earth', u'was', u'fill', u'with', u'fourteen', u'children', u'and', u'there', u'were', u'thirteen', u'more', 'in', u'the', u'camp', u'here', u'too', u'there', u'were', u'four', u'infant', 'in', u'scanti', u'cotton', u'blanket', u'the', u'troubl', u'with', u'arkansa', u'mrbutler', u'said', 'is', u'that', 'it', 'is', u'planterown', u'most', 'of', u'them', u'are', u'absente', u'landlord', 'at', u'that', u'they', u'put', 'in', 'a', u'bossdriv', u'and', u'they', 'go', u'off', u'and', u'live', 'in', u'the', u'town', u'the', u'deweychapman', u'plantat', u'has', u'about', u'num', u'acr', u'wilson', 'is', u'the', u'largest', u'with', u'num', u'the', u'boss', u'driver', u'allow', u'about', u'fifteen', u'acr', 'to', u'the', u'famili', 'so', u'you', u'see', u'how', u'mani', u'famili', u'are', u'affect', u'they', u'alway', u'end', 'up', u'the', u'year', u'owe', u'the', u'boss', u'there', 'is', 'no', u'money', u'for', u'shoe', u'for', u'cloth', u'they', u'grow', u'cotton', u'but', u'they', u'dress', 'in', u'flour', u'sack', u'it', u'the', u'richest', u'land', 'in', u'the', u'countri', u'but', u'they', u'arent', u'allow', 'to', u'put', 'in', 'a', u'garden', 'or', u'keep', 'a', u'pig', u'they', u'cant', 'go', 'to', u'school', u'everi', u'child', u'that', u'can', u'work', 'in', u'the', u'field', u'the', u'govern', u'has', u'start', u'rehabilit', u'farm', u'but', u'all', u'that', u'happen', 'is', u'that', u'the', u'owner', u'rent', u'his', u'land', 'to', u'the', u'govern', u'the', u'bossdriv', 'is', u'left', 'on', 'to', u'direct', u'the', u'job', u'and', u'the', u'work', u'run', u'along', u'same', 'as', u'usual', u'what', u'the', u'sharecropp', u'want', 'is', u'small', u'farm', 'of', u'their', u'own', u'and', u'some', u'help', u'from', u'the', u'govern', u'the', u'first', u'year', 'to', u'buy', u'equip', u'and', u'food', u'until', u'they', u'get', u'go', u'what', u'the', u'union', u'want', 'to', 'do', 'is', 'to', u'start', u'cooper', u'farm', 'of', u'num', u'acr', u'compar', 'to', u'the', u'planter', u'num', u'but', u'noth', u'can', 'be', u'done', u'eighti', u'per', u'cent', 'of', u'the', u'peopl', u'are', u'disfranchis', 'on', u'account', 'of', u'the', u'poll', u'tax', u'and', u'legislatur', u'relief', u'and', u'the', u'press', u'are', u'run', 'by', u'the', u'planter', u'this', 'is', u'the', u'situat', 'in', u'one', u'state', u'but', u'all', u'the', u'other', u'southern', u'state', u'are', u'affect', 'in', u'the', u'same', u'way', u'there', u'are', u'num', u'acr', u'devot', 'to', 'a', u'singl', u'crop', u'near', 'as', u'much', u'land', 'as', 'is', u'given', 'to', u'all', u'other', u'crop', 'in', u'the', u'whole', u'region', u'negro', 'no', u'longer', u'make', 'up', u'the', u'bulk', 'of', u'cotton', u'tenant', u'white', u'worker', u'outnumb', u'the', u'black', u'five', 'to', u'three', 'a', u'seventypag', u'studi', 'of', u'the', u'collaps', 'of', u'cotton', u'tenanc', 'is', u'publish', 'by', u'the', u'univers', 'of', u'north', u'carolina', u'press', u'the', u'prefac', 'of', u'this', 'is', u'sign', 'by', 'rt', u'revmsgr', u'john', 'a', u'ryan', u'and', u'revedgar', u'schmiedel', u'with', u'fifteen', u'other', u'leader', u'while', u'survey', u'are', u'be', u'made', u'and', u'written', u'the', u'southern', u'tenant', u'farmer', u'union', u'carri', 'on', u'they', u'are', u'organ', u'the', u'sharecropp', u'into', 'a', u'union', u'they', u'are', u'hold', u'meet', 'on', u'cooper', u'they', u'are', u'bring', u'relief', u'they', u'have', u'had', 'a', u'hard', u'struggl', 'in', u'the', u'past', u'and', u'the', u'futur', u'look', u'dark', u'but', u'combin', u'with', u'faith', u'and', u'chariti', u'they', u'have', u'hope', u'and', u'the', u'terror', u'that', u'walk', 'by', u'day', u'and', 'by', u'night', 'in', u'arkansa', u'doe', u'not', u'daunt', u'them', u'the', u'cathol', u'worker', u'decemb', u'num', u'num', u'summari', u'defin', u'christian', u'person', 'as', u'the', u'realize', 'of', u'the', u'digniti', 'of', u'the', u'other', u'fellow', 'of', u'our', u'oblig', 'to', u'him', u'the', u'willing', 'to', u'work', u'with', u'him', 'on', u'those', u'element', 'of', u'truth', 'he', u'has', u'seiz', u'hold', 'of', u'accept', u'his', u'cooper', 'as', u'far', 'as', 'he', u'will', u'give', 'it', u'and', u'the', u'refus', 'to', u'admit', u'disappoint', u'when', 'he', u'doesnt', 'go', 'as', u'far', 'as', 'we', u'think', 'he', u'might', u'argu', u'that', u'the', u'marshal', u'plan', u'has', u'deni', u'this', u'definit', u'and', 'is', 'an', u'extens', 'of', u'industri', u'capit', u'and', u'abrog', u'our', u'person', u'respons', u'ddlw', u'num', u'the', u'marshal', u'plan', 'an', u'editori', u'who', u'then', 'is', u'our', u'brother', u'the', u'opposit', 'of', u'the', u'cathol', u'worker', 'to', u'the', u'marshal', u'plan', 'is', u'base', u'upon', u'it', u'violat', 'of', u'the', u'christian', u'concept', 'of', u'chariti', u'the', u'fact', u'that', 'it', u'has', u'for', u'it', u'prime', u'purpos', u'the', u'extens', u'and', u'propag', 'of', 'an', u'econom', u'system', 'we', u'believ', 'to', 'be', u'unjust', u'and', u'immor', u'consist', u'the', u'cathol', u'worker', 'in', u'it', u'editori', u'polici', 'in', u'it', u'sign', u'articl', 'in', u'it', u'manner', 'of', u'oper', u'hous', 'of', u'hospit', u'has', u'emphas', u'christian', u'person', u'the', u'realize', 'of', u'the', u'digniti', 'of', u'the', u'other', u'fellow', 'of', u'our', u'oblig', 'to', u'him', u'the', u'willing', 'to', u'work', u'with', u'him', 'on', u'those', u'element', 'of', u'the', u'truth', 'he', u'has', u'seiz', u'hold', 'of', u'accept', u'his', u'cooper', 'as', u'far', 'as', 'he', u'will', u'give', 'it', u'and', u'the', u'refus', 'to', u'admit', u'disappoint', u'when', 'he', u'doesnt', 'go', 'as', u'far', 'as', 'we', u'think', 'he', u'might', u'consist', 'we', u'have', u'written', 'in', u'term', 'of', u'person', u'respons', u'and', u'the', u'need', u'for', u'all', u'cathol', 'to', u'exhaust', u'the', u'messag', 'of', u'christ', 'as', u'given', 'to', 'us', 'in', u'the', u'sermon', 'on', u'the', u'mount', 'i', u'have', 'a', u'new', u'command', 'to', u'give', u'you', u'that', u'you', u'are', 'to', u'love', u'one', u'anoth', u'that', u'your', u'love', u'for', u'one', u'anoth', 'is', 'to', 'be', u'like', u'the', u'love', 'i', u'have', u'born', u'you', u'like', u'the', u'love', 'i', u'have', u'born', u'youthat', 'is', u'the', u'essenc', 'of', u'the', u'lord', u'teach', u'his', u'love', u'for', 'us', u'was', u'not', u'selfright', u'but', u'full', 'of', u'deep', u'compass', u'his', u'love', u'was', u'base', u'upon', u'fratern', u'chariti', u'for', 'us', u'the', u'love', 'of', 'a', u'brother', u'for', u'brother', u'tri', u'and', u'imagin', u'our', u'lord', u'tell', u'the', u'nonbeliev', u'among', u'the', u'multitud', 'at', u'the', u'time', 'of', u'the', u'miracl', 'of', u'the', u'loav', u'and', u'fish', 'go', u'hungri', u'for', 'i', u'know', u'that', u'you', 'do', u'not', u'agre', u'with', 'me', u'now', u'and', 'i', u'know', u'and', 'he', u'did', u'know', u'that', 'on', 'a', u'certain', u'friday', u'you', u'will', u'demand', 'my', u'life', u'becaus', 'my', u'teach', u'affront', u'your', u'social', u'sensibl', u'this', u'then', 'is', u'the', u'essenc', 'of', u'the', u'marshal', u'plan', 'we', u'are', u'seiz', u'upon', u'the', u'starvat', u'status', 'of', u'our', u'brother', 'in', u'europ', u'and', u'tell', u'them', 'go', u'hungri', u'you', 'do', u'not', u'agre', u'with', u'our', u'ideolog', u'concept', 'if', 'i', u'feed', u'you', u'now', u'you', u'may', u'oppos', 'me', u'later', u'the', u'direct', u'antithesi', 'of', u'our', u'lord', u'teach', u'and', u'exampl', 'we', u'have', u'consist', 'in', u'the', u'cathol', u'worker', u'discuss', u'the', u'problem', 'of', u'modern', u'industri', u'capit', 'we', u'have', u'not', u'been', u'delud', u'nor', u'have', 'we', u'delud', u'ani', 'of', u'our', u'reader', u'into', u'think', u'that', u'with', u'the', u'applic', 'of', u'the', u'revolutionari', u'principl', 'of', u'christ', 'in', u'our', u'live', u'and', 'in', u'the', u'live', 'of', u'ani', u'great', u'number', 'of', u'peopl', u'that', u'the', u'present', u'social', u'and', u'industri', u'structur', u'can', u'continu', 'to', u'exist', u'insofar', 'as', 'we', u'appli', u'these', u'principl', 'in', u'our', u'live', 'to', u'that', u'extent', u'the', u'christian', u'revolut', u'has', u'begun', 'to', u'that', u'extent', u'the', u'present', u'system', u'has', u'been', u'chang', 'we', 'do', u'not', u'believ', u'that', u'europ', u'can', 'be', u'save', u'from', u'the', u'domin', 'of', u'one', u'servil', u'state', u'russia', 'by', u'the', u'extens', 'of', u'the', u'econom', u'polici', 'of', u'anoth', u'servil', u'state', u'the', u'unit', u'state', 'we', 'do', u'know', u'that', u'this', 'is', u'just', u'what', 'is', u'involv', 'in', u'the', u'marshal', u'plan', u'our', u'secretari', 'of', u'state', u'has', u'not', u'tri', 'to', u'conceal', 'it', 'he', u'has', 'in', u'fact', u'been', u'quit', u'explicit', 'on', u'this', u'score', 'we', 'do', u'know', u'that', u'alreadi', u'over', u'onehalf', 'of', u'the', u'moni', u'sent', 'to', u'greec', u'are', u'be', u'spent', 'on', u'the', u'armi', 'to', u'take', u'advantag', 'of', 'a', u'peopl', u'destitut', u'has', u'alway', u'been', 'a', u'fundament', u'polici', 'of', u'histor', u'capit', u'our', u'present', u'system', u'base', 'as', 'it', 'is', 'on', u'the', u'profit', u'motiv', u'can', u'never', u'give', u'the', u'underdog', 'an', u'even', u'break', 'it', 'is', u'that', u'form', 'of', u'capit', u'which', u'has', u'brought', u'about', u'the', u'present', u'situat', 'in', u'europ', 'it', 'is', u'that', u'form', 'of', u'capit', u'which', u'should', u'bear', u'the', u'cost', 'of', u'repair', u'the', u'damag', 'if', 'we', u'would', u'begin', u'here', u'and', u'now', 'to', u'produc', u'for', u'use', u'and', u'not', u'for', u'profit', 'we', u'could', u'meet', u'the', u'present', u'crisi', u'with', u'small', u'inconveni', 'to', u'ourselv', u'for', u'the', u'past', u'three', u'year', 'we', u'have', u'been', u'run', u'the', u'name', u'and', u'address', 'of', u'needi', u'person', 'in', u'europ', 'we', u'felt', u'that', 'it', u'was', 'an', u'opportun', u'for', u'our', u'reader', 'to', u'practic', u'person', u'what', 'we', u'have', u'been', u'write', u'about', u'all', u'these', u'year', u'our', u'reader', u'have', u'answer', u'these', u'appeal', u'for', u'help', u'magnific', 'we', u'pray', u'that', u'they', u'may', u'continu', 'to', 'do', 'so', 'we', u'hope', u'that', u'they', u'will', u'join', 'us', 'in', u'offer', u'our', u'prayer', 'in', u'increas', u'our', u'penanc', 'in', u'implor', u'our', u'lord', u'dure', u'the', u'recept', 'of', u'the', u'sacrament', 'to', u'smile', u'upon', u'his', u'children', u'throughout', u'the', u'world', u'regardless', 'of', u'race', 'or', u'creed', 'or', u'polit', u'belief', u'the', u'spiritu', u'weapon', u'are', u'our', u'strength', 'as', u'well', 'as', u'our', u'joy', u'and', u'upon', u'them', 'we', u'must', u'reli', u'constant', u'the', u'cathol', u'worker', u'januari', u'num', u'num', u'summari', u'explain', u'that', u'the', u'basic', u'unit', 'of', u'societi', u'for', u'cathol', u'sociolog', 'is', u'the', u'famili', u'and', u'when', u'the', u'famili', u'fall', 'so', u'doe', u'catholic', u'this', 'is', u'the', u'reason', u'for', u'hostil', 'to', u'communism', u'and', u'the', u'same', u'should', 'be', u'true', u'for', u'capit', u'sinc', 'it', u'creat', 'a', u'class', u'that', u'make', 'it', u'difficult', 'to', u'sustain', 'a', u'famili', u'encourag', u'famili', u'support', u'program', u'ddlw', u'num', u'the', u'paper', u'have', u'been', u'tell', 'us', u'recent', 'of', u'the', u'new', u'trend', 'in', u'soviet', u'russia', 'to', u'glorifi', u'the', u'woman', u'who', u'rais', 'a', u'famili', u'rather', u'than', u'the', u'woman', 'in', u'the', u'factori', 'or', u'drive', 'a', u'tractor', u'even', u'there', 'it', u'seem', u'the', u'experiment', 'in', u'social', u'relat', u'has', u'confirm', u'what', u'cathol', u'sociolog', u'has', u'alway', u'taught', u'that', u'there', u'are', 'so', u'mani', u'thing', u'need', u'for', u'the', u'reconstruct', 'of', u'social', u'order', u'out', 'of', u'chao', u'today', u'that', 'we', u'scarc', u'know', u'where', 'to', u'begin', u'cathol', u'trade', u'union', u'cooper', u'farm', u'commune', u'school', u'and', u'colleg', u'person', u'respons', u'all', 'of', u'these', 'of', u'coursebut', u'person', u'respons', 'on', u'the', u'part', 'of', u'the', u'individu', u'alon', u'doe', u'not', u'perpetu', 'a', u'new', u'mode', 'of', u'life', u'school', u'and', u'colleg', u'are', u'direct', 'by', u'those', u'brought', 'up', 'in', u'the', u'old', u'way', u'communiti', 'of', u'individu', u'die', u'out', 'or', u'chang', u'their', u'charact', u'cathol', u'cooper', u'and', u'trade', u'union', u'can', u'not', u'begin', 'to', u'exist', u'until', 'we', u'have', u'cathol', u'live', u'their', u'faith', u'and', u'creat', 'a', u'truli', u'cathol', u'cultur', u'and', u'tradit', 'so', 'it', 'is', 'to', u'the', u'recreat', 'of', u'the', u'cathol', u'famili', u'that', u'microcosm', 'of', u'societi', u'and', u'type', 'of', u'the', u'mystic', u'bodi', u'that', 'we', u'must', u'look', u'now', 'if', 'we', u'expect', u'later', u'generat', 'in', u'this', u'countri', 'to', u'see', u'catholic', u'instead', 'of', u'marxism', 'a', u'vital', u'forc', 'in', u'the', u'nation', 'if', 'we', u'hope', 'to', u'maintain', u'the', u'freedom', 'of', u'the', u'children', 'of', u'god', u'against', u'the', u'diabol', u'forc', 'of', u'fascism', 'if', 'we', u'wish', u'our', u'poster', 'to', u'enjoy', u'the', u'peac', u'which', 'is', u'their', u'heritag', u'from', u'christ', u'instead', 'of', u'the', u'strife', 'of', u'war', u'and', u'hatr', u'between', u'nation', u'and', u'class', u'marxist', u'communism', u'has', u'been', u'condemn', 'by', u'cathol', u'for', u'it', u'destruct', 'of', u'famili', u'life', u'what', u'then', 'of', u'capit', u'which', u'creat', 'an', u'evergrow', u'proletariat', u'ground', u'down', 'to', u'such', 'a', u'level', 'of', u'insecur', u'and', u'miseri', u'that', u'decent', u'famili', u'life', 'is', u'almost', u'imposs', u'probabl', 'we', u'have', u'all', u'wonder', u'about', u'the', u'dismal', u'futur', 'of', u'young', u'peopl', u'finish', u'school', u'these', u'day', u'with', u'year', 'of', u'steril', u'learn', u'behind', u'them', u'and', 'no', u'prospect', 'of', 'a', u'job', u'ahead', u'there', u'are', u'countless', u'thousand', 'of', u'them', u'well', 'on', 'in', u'their', u'twenti', u'today', u'who', u'have', u'never', u'had', 'a', u'job', u'and', u'whose', u'chanc', u'decreas', 'as', u'the', u'year', u'pass', u'capit', u'bar', u'them', u'from', u'live', 'by', u'the', u'fruit', 'of', u'the', u'labor', u'they', u'would', u'will', u'give', u'and', u'marriag', 'is', u'out', 'of', u'the', u'question', u'even', 'if', u'they', u'have', u'job', u'cathol', u'young', u'peopl', u'dare', u'not', u'assum', u'the', u'respons', 'of', u'marriag', u'becaus', u'they', u'cant', u'afford', 'a', u'babi', u'economist', u'tell', 'us', u'that', 'in', u'day', 'of', u'prosper', u'children', u'are', 'an', u'econom', u'asset', 'we', u'assert', u'that', 'in', u'these', u'day', 'of', u'revolut', u'when', 'we', u'cathol', u'must', u'arm', u'ourselv', u'with', u'the', u'holi', u'spirit', 'to', u'renew', u'the', u'face', 'of', u'the', u'earth', 'in', u'this', u'struggl', u'too', u'children', u'are', 'an', u'asset', u'and', u'famili', u'life', u'the', u'most', u'power', u'and', u'endur', u'form', 'of', u'propaganda', 'we', u'can', u'not', u'understand', u'then', u'the', u'shortsighted', 'of', u'cathol', u'who', u'contribut', u'generous', u'for', u'cathol', u'school', u'and', u'neglect', 'to', u'aid', u'the', u'cathol', u'famili', u'life', u'from', u'which', u'the', u'school', u'must', u'draw', u'their', u'student', u'longrang', u'action', 'is', u'need', u'here', u'but', u'there', 'is', u'also', 'an', u'immedi', u'and', u'urgent', u'need', 'in', u'the', u'new', u'york', u'dioces', u'especi', 'we', u'have', u'often', u'comment', 'in', u'the', u'cathol', u'worker', 'on', u'the', u'high', u'cost', 'of', u'matern', u'care', u'here', u'and', u'the', u'lack', 'of', u'facil', u'for', u'such', u'care', u'under', u'cathol', u'auspic', u'chicago', u'has', u'it', u'larg', u'cathol', u'matern', u'hospit', u'where', 'in', u'the', u'short', u'time', 'it', u'has', u'exist', u'over', u'ten', u'thousand', u'mother', u'have', u'been', u'care', u'for', u'mani', u'without', u'charg', u'other', u'dioces', u'have', u'establish', u'matern', u'guild', 'on', 'a', u'parish', 'or', u'diocesan', u'basi', u'but', 'in', u'new', u'york', u'where', u'last', u'year', u'misericordia', u'hospitalth', u'onli', u'cathol', u'hospit', 'in', u'the', u'citi', u'which', u'accept', u'free', u'matern', u'casescar', u'for', u'near', u'num', u'free', u'matern', u'patient', u'there', 'is', 'no', u'diocesan', u'provis', u'whatsoev', u'for', u'the', u'poor', u'who', u'are', u'tri', 'to', u'observ', u'the', u'teach', 'of', u'the', u'church', u'against', u'birth', u'control', u'and', u'rais', u'famili', u'despit', u'their', u'econom', u'circumst', 'we', u'would', u'suggest', u'that', 'as', 'a', u'first', u'even', u'though', u'inadequ', u'step', u'toward', u'realiz', u'out', u'ideal', 'of', u'famili', u'life', u'our', u'cathol', u'laypeopl', u'and', u'clergi', u'show', u'the', u'sincer', 'of', u'their', u'convict', 'by', u'rais', u'their', u'voic', 'in', u'loud', u'agit', u'for', u'cathol', u'provis', 'of', u'such', u'care', 'we', u'offer', 'up', 'to', u'thee', 'o', u'lord', 'a', u'victim', 'of', u'ransom', u'humbl', u'entreat', u'that', u'through', u'the', u'intercess', 'of', u'the', u'virgin', u'mother', 'of', u'god', u'and', u'bless', u'joseph', u'thou', u'wouldst', u'strong', u'establish', u'our', u'famili', 'in', u'thi', u'peac', u'and', u'thi', u'grace', u'secret', u'feast', 'of', u'the', u'holi', u'famili', u'the', u'cathol', u'worker', u'septemb', u'num', u'num', u'summari', u'discuss', 'in', u'length', u'the', u'modern', u'industri', u'problem', 'of', u'the', u'machin', u'and', u'it', u'relat', 'to', u'factori', u'land', u'and', u'worker', u'explain', u'the', u'cws', u'attempt', 'to', u'gain', u'the', u'worker', u'back', 'to', u'christ', 'by', u'explic', 'a', u'philosophi', 'of', u'work', u'that', u'distinguish', u'between', u'those', u'machin', u'that', u'are', u'the', u'extend', u'hand', 'of', u'man', u'and', u'those', u'that', u'make', u'man', u'the', u'extend', u'hand', 'of', u'the', u'machin', u'such', 'a', u'philosophi', u'see', u'peopl', 'as', u'cooper', u'with', u'their', u'creator', u'and', 'to', u'labor', 'is', 'to', u'pray', u'critic', u'american', u'cathol', u'for', u'not', u'appli', u'papal', u'teach', 'to', u'the', u'work', u'area', u'and', u'show', 'a', u'particular', u'acrimoni', 'to', 'a', u'priest', u'who', u'tell', u'worker', 'to', u'sanctifi', u'their', u'surround', u'instead', 'of', u'chang', 'it', u'ddlw', u'num', 'i', u'have', u'befor', 'me', u'canon', u'cardijn', u'pamphlet', u'the', u'spirit', 'of', u'the', u'young', u'christian', u'worker', u'and', 'on', u'page', u'num', 'of', u'that', u'pamphlet', 'he', u'say', 'it', 'is', u'useless', 'to', u'steer', u'clear', 'of', u'the', u'vital', u'and', u'therefor', u'most', u'difficult', u'problem', u'under', u'the', u'pretext', 'of', u'cathol', u'uniti', 'so', 'i', u'shall', u'tri', 'to', u'write', u'about', u'most', u'difficult', u'and', u'vital', u'thing', u'such', 'as', u'the', u'factori', u'and', u'the', u'land', u'and', u'the', u'worker', u'cathol', u'action', u'the', u'cathol', u'worker', 'is', u'not', u'part', 'of', u'cathol', u'action', 'as', u'such', u'have', 'no', u'mandat', u'from', u'the', u'hierarchi', u'for', u'this', u'work', 'it', 'is', u'lay', u'activ', 'so', u'well', u'describ', 'by', u'maritain', 'in', u'his', u'book', u'true', u'human', u'the', u'actu', u'associ', 'of', u'cathol', u'trade', u'unionist', u'the', u'young', u'christian', u'worker', u'the', u'cathol', u'action', u'cell', u'that', u'are', u'spring', 'up', u'all', u'through', u'manhattan', u'and', u'brooklyn', u'are', u'definit', u'part', 'of', u'cathol', u'action', u'there', u'are', u'chaplain', 'in', u'charg', u'and', u'whether', u'the', u'work', 'is', 'in', u'the', u'guis', 'of', u'cyo', u'cathol', u'youth', u'organ', 'or', u'ycw', 'it', 'is', u'the', u'attempt', 'to', u'reach', u'the', u'worker', 'to', u'tri', 'to', u'gain', u'back', u'the', u'worker', 'to', u'christ', u'canon', u'cardijn', u'quot', u'the', u'holi', u'father', u'pope', u'pius', 'xi', 'as', u'have', u'said', 'to', u'him', u'the', u'worker', 'of', u'the', u'world', u'are', u'lost', 'to', u'the', u'church', u'and', 'he', u'has', u'had', u'what', 'is', 'in', u'effect', 'a', u'mandat', u'from', u'the', u'holi', u'father', u'himself', 'to', u'tri', 'to', u'reach', u'the', u'worker', 'of', u'the', u'world', u'lost', u'philosophi', u'begin', 'at', u'the', u'begin', 'of', u'the', u'pamphlet', u'there', 'is', u'that', u'much', u'quot', u'line', u'without', u'work', u'there', u'will', 'be', 'no', u'host', 'no', u'wine', 'no', u'chalic', 'no', u'altar', u'and', 'no', u'church', u'and', 'i', u'wish', 'to', u'fling', u'down', u'the', u'challeng', 'at', u'onc', u'that', u'what', 'is', u'the', u'great', u'disast', 'is', u'that', u'priest', u'and', u'laiti', u'alik', u'have', u'lost', u'the', u'concept', 'of', u'work', u'they', u'have', u'lost', 'a', u'philosophi', 'of', u'labor', 'as', u'peter', u'maurin', u'has', u'alway', u'said', u'they', u'have', u'lost', u'the', u'concept', 'of', u'work', u'and', u'those', u'who', 'do', u'not', u'know', u'what', u'work', 'in', u'the', u'factori', 'is', u'have', u'romantic', u'both', 'it', u'and', u'the', u'worker', u'and', 'in', u'emphas', u'the', u'digniti', 'of', u'the', u'worker', u'have', u'perhap', u'unconsci', u'emphas', u'the', u'digniti', 'of', u'work', u'which', 'is', u'slaveri', u'and', u'which', u'degrad', u'and', u'dehuman', u'man', u'sanctifi', u'their', u'surround', u'can', u'one', u'sanctifi', 'a', u'saloon', 'a', u'hous', 'of', u'ill', u'fame', u'when', u'one', 'is', 'in', u'the', u'occas', 'of', u'sin', 'is', 'it', u'not', u'necessari', 'to', u'remov', u'oneself', u'from', 'it', 'if', u'the', u'citi', 'is', u'the', u'occas', 'of', u'sin', 'as', u'father', u'vincent', u'mcnabb', u'point', u'out', u'should', u'not', u'famili', u'men', u'and', u'women', u'begin', 'to', u'aim', 'at', 'an', u'exodus', 'a', u'new', u'migrat', 'a', u'go', u'out', u'from', u'egypt', u'with', u'it', u'flesh', u'pot', u'subtl', u'sin', u'and', u'when', 'we', u'are', u'talk', u'about', u'sin', 'im', u'not', u'talk', u'about', u'adulteri', u'fornic', u'theft', u'drunken', 'as', u'such', 'in', u'the', u'great', u'clean', u'shine', u'factori', u'with', u'good', u'light', u'and', u'air', u'and', u'the', u'most', u'sanitari', u'condit', 'an', u'eighthour', u'day', u'fiveday', u'week', u'with', u'the', u'worker', u'chain', 'to', u'the', u'belt', 'to', u'the', u'machin', u'there', 'is', 'no', u'opportun', u'for', u'sin', 'as', u'the', u'outsid', u'think', 'of', u'sin', 'no', 'it', 'is', u'far', u'more', u'subtl', u'than', u'that', 'it', 'is', u'submit', u'oneself', 'to', 'a', u'process', u'which', u'degrad', u'dehuman', 'to', 'be', 'an', u'effici', u'factori', u'worker', u'one', u'must', u'becom', 'a', u'hand', u'and', u'the', u'more', u'effici', u'one', 'is', u'the', u'less', u'one', u'think', u'take', u'typewrit', u'for', u'instanc', 'as', 'an', u'exampl', 'we', u'all', u'know', 'or', u'drive', 'a', u'car', 'or', 'a', u'sew', u'machin', u'these', u'machin', u'may', 'be', u'consid', u'good', u'tool', 'an', u'extens', 'of', u'the', u'hand', 'of', u'man', 'we', u'are', u'not', u'chain', 'to', u'them', 'as', 'to', 'a', u'belt', u'but', u'even', 'so', 'we', u'all', u'know', u'that', 'as', u'soon', 'as', u'one', u'start', 'to', u'think', 'of', u'what', u'one', 'is', u'do', 'we', u'slip', u'and', u'make', u'mistak', u'one', 'is', u'not', u'suppos', 'to', u'think', 'to', u'think', 'is', u'danger', 'at', 'a', u'machin', u'one', 'is', u'liabl', 'to', u'lose', 'a', u'finger', 'or', 'a', u'hand', u'and', u'then', 'go', 'on', u'the', u'scrap', u'heap', u'and', u'spend', u'the', u'rest', 'of', u'one', u'life', u'fight', u'for', u'compens', u'for', u'one', u'own', u'careless', 'as', u'the', u'factori', u'owner', u'say', u'for', u'not', u'use', u'the', u'safeti', u'devic', u'invent', u'and', 'so', u'plenti', u'for', u'the', u'benefit', 'of', u'the', u'worker', u'the', u'exist', 'of', u'those', u'same', u'safeti', u'devic', 'is', 'an', u'exampl', 'of', u'the', u'truth', 'of', u'what', 'i', u'write', u'the', u'danger', u'and', u'here', 'is', u'the', u'dancer', u'part', 'it', 'is', u'not', 'so', u'much', u'the', u'loss', 'of', u'the', u'hand', 'or', u'the', u'arm', u'but', u'the', u'loss', 'of', u'one', u'soul', u'when', u'one', u'give', u'one', u'self', 'up', 'to', u'one', u'work', u'when', u'one', u'ceas', 'to', u'think', u'and', u'becom', 'a', u'machin', u'himself', u'the', u'devil', u'enter', 'in', 'we', u'can', u'not', u'lose', u'ourselv', 'in', u'our', u'work', u'without', u'grave', u'danger', 'de', u'rougement', u'brought', u'this', u'out', 'in', u'his', u'last', u'book', u'the', u'devil', u'share', 'as', u'soon', 'as', u'one', u'becom', u'besid', u'one', u'self', 'as', u'soon', 'as', 'we', u'lose', u'ourselv', 'as', u'soon', 'as', 'we', u'give', u'ourselv', 'up', 'to', u'anyth', u'whether', 'it', 'is', u'sex', 'or', u'drink', 'or', u'work', 'at', u'the', u'machin', u'there', 'is', u'the', u'danger', 'of', u'the', u'devil', u'enter', 'in', 'he', u'look', u'for', u'just', u'such', u'opportun', u'and', u'modern', u'life', 'is', u'full', 'of', u'them', u'see', u'our', u'recreat', u'footbal', u'basebal', u'these', u'are', u'suppos', 'to', 'be', u'recreat', u'and', u'yet', u'they', u'enjoy', u'themselv', u'most', u'who', u'most', u'thorough', u'lose', u'themselv', 'in', u'the', u'mob', u'and', u'the', u'mob', 'is', 'a', u'mob', u'whether', u'friend', 'or', u'hostil', 'as', u'eugen', u'deb', u'the', u'great', u'socialist', u'labor', u'leader', u'point', u'out', 'i', u'have', u'onli', u'experienc', u'the', u'friend', u'mob', 'be', u'said', u'onc', u'when', 'i', u'was', u'releas', u'from', u'jail', u'and', u'they', u'met', 'me', u'and', u'bore', 'me', 'on', u'their', u'shoulder', 'it', u'was', 'a', u'friend', u'mob', u'but', 'it', u'smelt', u'like', 'a', u'beast', u'the', u'beast', u'was', u'there', '', u'work', 'is', u'not', 'a', u'punish', 'a', u'curs', 'or', u'enslav', u'but', u'the', u'cooper', 'of', u'the', u'labor', u'with', u'his', u'creator', u'and', u'redeem', u'canon', u'cardijn', u'write', u'but', u'what', u'kind', 'of', u'work', u'without', u'the', u'worker', u'there', u'will', 'he', 'no', u'host', 'no', u'wine', u'etc', u'noth', u'amidst', u'much', 'in', u'the', u'last', u'few', u'issu', 'of', u'the', u'farm', u'labor', u'news', u'publish', 'by', u'the', u'farm', u'labor', u'union', u'which', u'has', u'it', u'headquart', 'in', u'memphi', u'tennesse', 'it', 'is', u'state', u'there', u'are', u'num', u'famili', u'worker', u'migratori', u'worker', 'on', u'the', u'land', u'these', u'are', u'homeless', u'landless', u'peopl', u'wander', u'around', u'harvest', u'crop', 'in', u'that', u'same', u'issu', u'they', u'speak', 'of', 'a', u'num', u'acr', u'plantat', u'where', u'num', u'human', u'be', u'work', u'they', u'deal', u'with', u'the', u'machin', u'the', u'cotton', u'picker', u'and', u'the', u'flamethrow', u'weed', u'killer', u'which', 'is', u'about', 'to', u'displac', u'one', u'half', u'the', u'present', u'popul', u'engag', 'in', u'cotton', u'farm', u'accord', 'to', u'the', u'depart', 'of', u'agricultur', u'the', u'hous', 'of', u'have', u'has', u'just', u'about', u'gotten', u'farm', u'fix', 'r', 'e', u'pari', 'of', u'florenc', u'arkansa', u'write', 'to', u'the', u'editor', 'i', u'can', u'not', u'find', u'ani', u'place', u'where', u'the', u'tenant', u'farmer', u'and', u'sharecropp', u'fit', 'in', u'this', u'machin', u'age', 'i', 'am', u'go', 'to', u'liken', u'the', u'machin', u'age', 'in', u'farm', 'to', u'the', u'man', u'who', u'built', u'his', u'hous', u'upon', u'the', u'sand', u'the', u'storm', u'came', u'and', u'the', u'wind', u'blew', u'and', u'the', u'hous', u'fell', u'and', u'great', u'was', u'the', u'fall', u'thereof', u'the', u'depart', 'of', u'agricultur', u'express', u'the', u'pious', u'hope', u'that', u'the', u'displac', u'worker', u'will', u'find', u'job', 'in', u'industri', u'accord', 'to', u'the', u'stori', u'the', u'veri', u'use', 'of', u'the', u'word', u'pious', u'show', u'the', u'undercurr', 'of', u'bitter', 'to', u'religion', u'though', u'the', u'union', u'work', u'with', u'minist', u'all', u'through', u'the', u'south', u'widespread', 'i', u'have', u'these', u'paper', u'befor', 'me', u'and', 'so', 'i', u'quot', u'from', u'them', u'but', u'the', u'same', u'goe', u'for', u'wheat', u'grower', u'who', u'have', u'num', u'acr', u'farm', u'and', u'gambl', 'in', u'their', u'one', u'crop', u'farm', u'and', u'live', 'in', u'the', u'citi', u'they', u'grow', u'the', u'wheat', u'and', u'homeless', u'labor', u'harvest', 'it', 'it', 'is', u'mill', 'in', u'big', u'citi', u'and', u'all', u'the', u'good', u'taken', u'out', 'of', 'it', 'in', u'the', u'way', 'of', u'wheat', u'germ', u'and', u'the', u'dead', u'product', u'sold', 'to', u'the', u'consum', 'is', u'this', u'the', u'kind', 'of', u'wheat', u'our', u'lord', u'took', u'into', u'his', u'hand', u'when', 'he', u'consecr', 'it', u'and', u'said', u'take', u'eat', u'sanctifi', u'can', 'we', u'sanctifi', u'such', u'work', u'too', u'much', u'cotton', u'and', u'none', 'to', u'wear', u'read', u'anoth', u'stori', u'there', u'are', u'twentytwo', u'million', u'bale', 'in', u'the', u'warehous', 'of', u'the', u'world', u'and', u'the', u'price', 'is', u'over', u'twentysix', u'cent', 'a', u'pound', u'the', u'highest', 'it', u'has', u'been', 'in', u'twentytwo', u'year', u'the', u'onli', u'way', u'worker', u'will', u'get', u'cotton', 'in', u'cloth', u'work', u'cloth', u'comfort', u'curtain', u'sheet', u'towel', u'one', u'cant', u'use', u'silk', 'or', u'rayon', 'or', u'nylon', u'for', u'these', 'is', 'to', u'wait', u'for', u'the', 'u', 's', u'govern', 'to', u'work', u'out', 'a', u'world', u'agreement', u'with', u'other', u'countri', 'to', u'limit', u'the', u'acreag', u'and', u'set', u'the', u'price', u'meanwhil', u'the', u'cotton', 'is', u'held', 'in', u'the', u'storehous', u'can', u'one', u'sanctifi', u'such', u'practic', 'as', u'these', 'oh', u'the', u'effici', 'of', u'modern', u'busi', u'which', u'lead', 'to', u'war', 'i', u'accus', u'yes', 'i', u'accus', u'the', u'leader', u'the', u'teacher', u'the', u'intellectu', u'the', u'clergi', 'of', u'have', 'a', u'romant', u'attitud', u'toward', u'the', u'worker', u'they', u'write', u'with', u'fervor', u'and', u'glow', u'wordsthey', u'dramat', u'the', u'struggl', u'they', u'are', u'walk', 'on', u'picket', u'line', u'they', u'love', u'the', u'man', 'in', u'the', u'dungare', u'and', u'the', u'blue', 'or', u'plaid', u'shirt', u'they', u'write', u'glow', 'of', u'his', u'callus', u'handsthey', u'take', u'these', u'leav', u'from', u'the', u'communist', u'notebookthey', u'are', u'glorifi', u'the', u'proletariat', u'the', u'dispossess', u'the', u'propertyless', u'the', u'homeless', u'and', u'the', u'worker', u'can', u'hang', 'a', u'holi', u'medal', 'on', u'their', u'machin', 'or', u'over', u'their', u'bunk', 'in', u'the', u'focastl', u'and', u'pray', 'as', u'they', u'begin', u'and', u'finish', u'their', u'work', u'and', 'go', u'home', 'to', u'their', u'tworoom', 'or', u'threeroom', u'apart', u'and', u'surround', 'by', u'children', u'and', 'an', u'exhaust', u'wife', u'sanctifi', u'their', u'surround', 'or', u'forget', u'them', 'in', u'the', u'nearest', u'tavern', u'with', u'pollut', u'beer', u'adulter', u'wine', 'or', u'hard', u'liquor', u'the', u'ycw', u'comment', 'at', u'the', u'end', 'of', u'canon', u'cardijn', u'pamphlet', 'is', u'this', u'the', u'ycw', u'show', u'the', u'worker', u'that', u'whatev', 'a', u'man', u'state', u'which', u'after', u'all', 'is', u'accord', 'to', u'the', u'dispens', 'of', u'provid', u'his', u'primari', u'struggl', 'is', u'against', u'himself', u'and', u'when', u'that', 'is', u'well', 'in', u'hand', u'the', u'rest', 'is', u'larg', 'a', u'question', 'of', u'adapt', u'himself', 'to', u'his', u'surround', 'in', u'order', 'to', u'get', u'the', u'veri', u'best', u'out', 'of', u'them', u'both', u'spiritu', u'and', u'tempor', u'accept', u'then', 'in', u'other', u'word', u'accept', u'the', u'machin', u'accept', u'the', u'factori', u'the', u'speedup', u'system', u'the', u'work', 'in', u'which', u'you', u'are', u'spentso', u'spent', u'inde', u'that', 'on', u'come', u'out', 'of', u'the', u'factori', u'exhaust', u'though', u'not', u'physic', u'the', u'work', 'is', u'light', u'mani', u'say', u'one', u'seek', u'the', u'anodyn', 'of', u'drink', 'or', u'sex', 'no', u'the', u'immor', 'is', u'not', 'in', u'the', u'factori', 'so', u'much', 'it', 'is', u'after', 'it', 'a', u'result', 'of', 'it', u'for', u'some', u'month', 'i', u'live', 'in', u'farmingdal', u'long', u'island', u'where', u'there', u'was', 'a', u'gigant', u'airplan', u'factori', u'where', u'they', u'were', u'construct', u'bomber', 'it', u'was', u'flung', 'up', u'dure', u'the', u'war', u'and', u'transport', u'was', u'difficult', u'what', u'with', u'gasolin', u'and', u'tire', u'and', u'poor', u'bus', u'servic', 'so', u'girl', u'were', u'alway', u'accept', u'lift', u'from', u'the', u'men', 'to', u'and', u'from', u'work', u'even', 'on', u'the', u'bus', u'there', u'was', u'crude', u'horseplay', u'and', u'immodest', u'talk', u'effect', 'a', u'man', 'is', u'spent', 'in', u'his', u'work', 'he', 'is', u'empti', 'he', 'is', u'emascul', u'and', u'vital', u'statist', u'from', u'drbaker', u'figur', u'from', u'the', u'depart', 'of', u'agricultur', u'show', u'that', 'it', 'is', u'not', u'onli', u'citi', u'life', u'but', u'life', 'in', u'the', u'factori', u'that', 'is', u'cut', u'down', u'the', u'size', 'of', u'the', u'famili', 'so', u'that', 'at', u'present', 'we', u'are', 'a', u'middleag', u'countri', 'i', u'speak', u'frank', 'as', 'a', u'lay', u'person', 'to', u'priest', 'of', u'this', u'vain', u'attempt', 'of', u'priest', 'to', u'urg', u'the', u'worker', 'to', u'sanctifi', u'one', u'surround', 'to', u'adapt', u'oneself', 'to', u'one', u'surround', u'what', u'kind', 'of', u'home', 'do', u'priest', u'live', 'in', u'how', u'larg', u'are', u'their', u'rectori', u'their', u'monasteri', u'their', u'hous', 'of', u'studi', u'how', u'can', u'they', u'speak', 'of', u'home', 'so', u'glow', u'how', u'can', u'they', u'talk', 'of', u'the', u'larg', u'famili', u'with', u'such', u'unction', u'when', u'they', u'see', u'the', u'tworoom', u'apart', u'the', u'fourroom', u'apart', 'on', u'mulberri', u'street', 'on', u'mott', u'street', u'trash', u'should', u'the', u'worker', u'sanctifi', u'his', u'surround', 'in', u'the', u'lumber', u'camp', u'where', u'huge', u'forest', u'are', u'be', u'denud', u'all', u'over', u'the', u'countri', u'for', u'profit', 'do', u'they', u'bring', u'him', u'home', u'where', u'did', u'all', u'the', u'lumber', 'go', 'it', 'is', u'tragic', 'as', 'i', u'write', 'on', u'this', u'white', u'paper', 'to', u'think', u'that', u'tree', u'were', u'cut', u'down', 'to', u'make', u'this', u'paper', u'and', u'the', u'billion', 'of', u'daili', u'news', u'true', u'stori', u'detect', u'fiction', u'movi', u'magazin', u'that', u'fill', u'the', u'drug', u'store', u'and', u'the', u'bus', u'station', u'pennsylvania', u'use', 'to', u'have', u'num', u'acr', 'of', u'forest', u'now', u'less', u'than', u'num', u'acr', 'of', u'forest', u'remain', u'within', u'the', u'commonwealth', u'and', u'pennsylvania', u'has', u'drop', 'in', u'lumber', u'product', u'from', u'the', u'first', 'to', u'twentyfourth', u'place', u'among', u'the', u'state', u'circular', u'from', u'the', u'depart', 'of', u'forest', 'of', 'pa', 'do', 'we', u'just', u'adapt', u'ourselv', 'to', u'this', u'evil', 'of', u'destruct', u'and', u'wast', u'not', u'onli', 'of', u'men', u'but', 'of', u'raw', u'materi', 'we', u'can', 'no', u'more', u'bless', 'it', u'sanctifi', 'it', u'than', u'the', u'priest', u'can', u'bless', u'the', u'scrap', u'iron', u'which', 'he', u'sprinkl', u'with', u'holi', u'water', 'in', u'the', u'church', u'yard', u'befor', 'it', u'set', u'off', 'to', u'kill', u'japanes', 'or', u'german', 'a', u'pictur', 'of', u'this', u'was', 'in', u'the', u'hearst', u'paper', 'in', u'chicago', u'and', u'yet', u'that', 'is', u'what', u'the', u'clergi', u'are', u'do', u'when', u'they', u'accept', u'this', u'system', u'this', u'industri', u'capit', u'canon', u'cardijn', 'is', u'oppos', 'to', u'the', u'materi', 'of', u'the', u'day', u'but', u'most', u'priest', u'when', u'they', u'sensit', u'use', u'the', u'word', u'materi', u'rather', u'than', u'communism', 'or', u'red', u'fascism', u'are', u'mean', u'the', u'same', u'thing', u'temporari', u'lift', u'where', u'christ', 'is', u'present', u'among', u'the', u'mass', 'in', u'all', u'their', u'circumst', u'condit', u'mode', 'of', u'life', u'there', u'christian', u'mystic', u'which', 'is', u'union', u'with', u'christ', u'take', 'on', 'a', u'deeper', u'and', u'more', u'extens', u'realiti', 'i', 'am', u'still', u'quot', u'from', u'the', u'pamphlet', u'but', 'i', u'deni', u'that', u'christ', 'is', u'present', 'in', u'the', u'factori', u'among', u'the', u'worker', u'that', u'they', u'take', u'him', u'there', u'the', u'worker', 'of', u'the', u'world', u'are', u'lost', 'to', u'the', u'church', u'they', u'are', u'still', u'lost', u'for', u'all', u'the', u'jocist', u'ycw', u'cyo', u'and', u'cathol', u'action', u'that', 'is', u'go', 'on', 'it', 'is', u'imposs', 'to', u'sanctifi', 'a', u'hous', 'of', u'shame', 'it', 'is', 'a', u'delus', 'a', u'decept', u'and', u'onc', u'more', u'the', u'worker', u'are', u'be', u'betray', u'for', 'a', u'while', u'they', u'may', 'be', u'arous', 'to', u'hope', u'they', u'may', u'lose', u'themselv', 'in', u'meet', 'in', u'mass', u'demonstr', 'in', u'pilgrimag', 'to', u'the', u'holi', u'father', u'and', u'fill', u'with', u'the', u'afterglow', 'of', u'these', u'glorious', u'experi', u'warm', u'their', u'factori', u'with', u'them', u'but', 'it', u'will', u'not', 'be', u'last', u'communism', u'creep', u'inexor', 'on', u'and', u'show', u'it', u'influenc', u'one', u'way', 'or', u'anoth', u'the', u'popular', u'republican', u'movement', 'of', u'franc', u'which', u'has', 'as', u'part', 'of', u'it', u'econom', u'program', u'nation', 'or', u'nation', u'control', u'has', u'elicit', u'this', u'comment', u'from', u'the', u'holi', u'father', 'in', u'his', u'letter', 'to', u'the', u'social', u'studi', u'week', u'held', 'by', u'french', u'cathol', 'in', u'strasbourg', u'recent', 'it', 'is', u'evid', u'that', u'instead', 'of', u'attenu', u'the', u'mechan', u'natur', 'of', u'life', u'and', u'labor', 'in', u'common', u'nation', u'even', u'when', u'licit', u'threaten', 'to', u'accentu', u'these', u'still', u'more', u'and', u'consequ', u'the', u'benefit', u'deriv', u'from', 'it', 'to', 'a', u'true', u'communiti', 'is', u'veri', u'much', u'open', 'to', u'question', u'the', u'pope', u'word', 'in', u'num', 'in', 'an', u'address', 'to', u'the', u'intern', u'congress', 'of', u'cathol', u'women', u'leagu', u'the', u'holi', u'father', u'said', 'in', u'this', u'age', 'of', u'mechan', u'the', u'human', u'person', u'becom', u'mere', 'a', u'more', u'perfect', u'tool', 'in', u'industri', u'product', u'and', u'how', u'sad', 'it', 'is', 'to', u'say', 'it', 'a', u'perfect', u'tool', u'for', u'mechan', u'warfar', u'and', 'at', u'the', u'same', u'time', u'materi', u'and', u'readymad', u'amus', 'is', u'onli', u'thing', u'which', u'stir', u'and', u'set', u'the', u'limit', 'to', u'the', u'aspir', 'of', u'the', u'mass', 'in', u'this', u'disintegr', 'of', u'human', u'person', u'effort', u'are', u'be', u'made', 'to', u'restor', u'uniti', u'but', u'the', u'plan', u'propos', u'are', u'vitiat', u'from', u'the', u'start', u'becaus', u'they', u'set', u'out', u'from', u'the', u'selfsam', u'principl', 'as', u'the', u'evil', u'they', u'intend', 'to', u'cure', u'the', u'wound', u'and', u'bruis', 'of', u'individualist', u'and', u'matererialist', u'mankind', u'can', u'not', 'be', u'heal', 'in', 'a', u'system', u'which', 'is', u'materialist', 'in', u'it', u'own', u'principl', u'and', u'mechanist', 'in', u'the', u'applic', 'of', u'it', u'principl', 'to', u'heal', u'the', u'wound', u'there', 'is', u'one', u'sovereign', u'remedi', u'that', 'is', 'a', u'return', 'of', u'the', u'heart', u'and', u'mind', 'of', u'mankind', 'to', u'knowledg', u'and', u'love', 'of', u'god', u'the', u'common', u'father', u'and', 'of', u'him', u'whom', u'god', u'has', u'sent', 'to', u'save', u'the', u'world', u'jesus', u'christ', u'the', u'mass', u'continu', u'this', u'charg', 'of', u'sentiment', 'or', u'romantic', 'of', u'the', u'proletariat', 'we', u'would', u'like', 'to', u'call', u'attent', 'to', u'the', u'use', 'of', u'the', u'word', u'mass', 'i', u'use', 'to', u'work', u'for', u'the', u'old', u'mass', u'and', 'i', u'had', u'quit', 'a', u'few', u'articl', 'in', u'the', u'new', u'mass', u'publish', 'by', u'the', u'communist', 'in', u'new', u'york', u'citi', u'one', 'of', 'my', u'articl', 'on', u'the', u'birth', 'of', 'my', u'daughter', 'in', 'a', u'public', u'hospit', u'was', u'reprint', u'all', u'over', u'russia', u'and', u'accord', 'to', 'my', u'old', u'friend', u'michael', u'gold', 'of', u'the', u'daili', u'worker', u'staff', 'i', u'have', u'quit', 'a', u'bit', 'of', u'royalti', u'come', 'to', 'me', 'in', u'russia', u'sinc', 'it', u'was', u'translat', u'and', u'reprint', 'in', u'mani', u'languag', u'diego', u'rivera', u'the', u'mexican', u'artist', u'told', 'me', 'in', u'num', u'that', 'he', u'had', u'read', 'it', u'when', 'he', u'was', 'in', u'russia', 'it', u'was', 'he', 'in', u'fact', u'that', u'told', 'me', 'it', u'had', u'been', u'reprint', u'the', u'mass', u'what', 'a', u'glow', u'word', 'it', u'was', 'to', 'us', u'then', 'to', u'speak', 'to', u'the', u'mass', 'to', u'write', 'to', u'the', u'mass', 'to', 'be', 'a', u'part', 'of', u'the', u'massesour', u'heart', u'dure', u'those', u'year', u'glow', u'within', 'us', 'at', u'the', u'idea', u'the', u'peopl', u'and', u'now', u'the', u'holi', u'father', 'in', u'one', 'of', u'his', u'great', u'christma', u'messag', u'point', u'out', u'the', u'differ', u'between', u'the', u'mass', u'and', u'the', u'peopl', 'he', u'tell', 'us', u'that', u'the', u'express', u'the', u'mass', u'convey', u'the', u'idea', 'of', u'the', u'inert', u'mass', u'be', u'move', u'this', u'way', u'and', u'that', u'sway', 'as', 'by', 'a', u'wind', u'control', 'by', u'the', u'demagogu', u'wherea', u'the', u'peopl', u'convey', u'the', u'idea', 'of', u'person', u'creatur', 'of', u'bodi', u'and', u'soul', u'templ', 'of', u'the', u'holi', u'ghost', u'men', u'and', u'women', u'and', u'children', u'made', 'to', u'the', u'like', 'of', u'god', u'divin', 'by', u'their', u'baptism', u'and', u'made', u'son', 'of', u'god', u'sinc', u'that', u'great', u'messag', u'the', u'holi', u'father', u'these', u'comment', 'of', u'his', u'were', u'onli', 'a', u'part', 'of', 'it', 'i', u'have', u'hesit', u'ever', u'again', 'to', u'use', u'the', u'word', u'mass', u'canon', u'cardijn', 'or', u'his', u'translat', u'use', 'it', u'thirteen', u'time', 'in', 'a', u'twentypag', u'pamphlet', 'he', u'use', u'crowd', u'onc', 'to', u'captur', u'the', u'crowd', u'and', u'multitud', u'onc', u'long', u'rang', u'view', u'this', 'is', 'an', u'old', u'controversi', 'we', u'start', 'to', u'read', u'about', 'it', 'in', u'the', u'page', 'of', u'blackfriar', u'the', u'english', u'dominican', u'month', 'it', u'certain', u'went', 'on', u'when', 'g', 'k', u'chesterton', u'and', u'eric', u'gill', u'were', u'still', u'aliv', 'it', 'is', u'their', u'point', 'of', u'view', 'we', u'repres', 'we', u'gave', 'up', u'page', 'of', u'the', u'cathol', u'worker', 'to', 'it', u'when', u'father', u'paul', u'sanley', u'furfey', u'and', u'father', u'john', 'j', u'hugo', u'wrote', 'on', 'it', u'back', 'in', u'num', 'it', u'often', u'end', 'in', u'acrimoni', u'and', 'i', u'hope', u'that', u'our', u'friend', u'will', u'not', u'credit', 'us', u'with', u'ani', u'but', u'the', u'most', u'love', u'desir', 'to', u'gain', u'them', u'for', 'a', u'longrang', u'program', 'of', u'action', 'we', u'need', 'to', u'start', 'in', u'the', u'factori', 'of', u'cours', u'here', 'is', u'what', u'eric', u'gill', u'said', 'in', u'some', u'letter', 'to', u'the', u'cathol', u'worker', 'in', u'num', 'i', u'should', u'like', 'to', u'say', u'simpli', u'that', u'fundament', u'the', u'problem', 'of', u'the', u'machin', 'is', u'one', u'which', u'should', 'be', u'dealt', u'with', 'by', u'those', u'who', u'actual', u'use', u'machin', 'at', u'present', 'as', u'you', u'know', u'the', u'respons', u'for', u'use', 'or', u'not', u'use', u'machin', 'is', u'entir', u'that', 'of', u'men', 'of', u'busi', u'whose', u'interest', u'are', 'of', u'cours', u'simpli', 'in', u'buy', u'and', u'sell', u'and', u'not', 'in', u'make', u'and', u'therefor', 'in', 'a', u'broad', u'way', 'it', u'may', 'be', u'said', u'that', u'the', u'first', u'thing', 'to', 'be', u'done', u'first', 'in', u'the', u'sens', 'of', u'most', u'import', 'is', u'for', u'the', u'worker', 'to', u'recaptur', u'the', u'control', 'of', u'industri', u'mad', u'commerci', u'this', 'of', u'cours', 'is', u'the', u'communist', u'idea', u'but', u'unfortun', u'the', u'communist', u'coupl', u'with', u'this', u'their', u'veri', u'crude', u'materialist', u'philosophi', u'and', u'their', u'equal', u'crude', u'idolatri', 'of', u'the', u'machin', u'for', u'the', u'rest', 'it', u'should', 'be', u'obvious', u'that', u'some', u'thing', u'are', u'better', u'made', 'by', u'machin', u'than', 'by', u'hand', u'for', u'instanc', 'it', u'would', 'be', u'ridicul', 'to', u'make', u'typewrit', u'except', 'by', u'mass', u'product', u'otherwis', u'they', u'would', 'be', u'absolut', u'prohibit', 'in', u'price', u'and', u'the', u'whole', u'point', 'of', 'a', u'typewrit', 'is', 'to', u'save', u'money', u'and', u'time', u'but', u'again', 'it', u'should', 'be', u'obvious', u'that', u'the', u'whole', u'idea', 'of', u'save', u'time', u'and', u'money', 'to', u'such', 'an', u'extent', 'as', 'we', u'have', u'develop', 'it', 'is', 'a', u'product', 'of', u'our', u'quit', u'mad', u'unholi', u'commerci', u'competit', u'rush', u'hand', u'man', u'holi', u'then', u'again', u'such', u'thing', 'as', u'watermil', u'and', u'windmil', u'which', u'save', u'human', u'labor', u'grind', u'corn', u'saw', u'wood', u'etc', u'are', u'obvious', u'proper', u'instrument', u'and', u'this', u'bring', 'us', 'to', u'the', u'point', 'of', u'distinguish', u'between', u'those', u'machin', u'which', u'simpli', u'save', u'human', u'muscular', u'labor', u'and', u'those', u'which', u'displac', u'human', u'creativ', u'skill', u'the', u'worker', 'is', 'a', u'man', u'and', u'not', u'simpli', 'a', u'hand', u'work', u'done', 'by', u'man', 'is', u'human', u'work', 'to', 'be', u'valu', u'and', u'thought', 'of', 'as', u'such', u'and', u'not', u'mere', 'as', 'a', u'cost', 'in', u'the', u'account', u'book', 'to', u'labor', 'is', 'to', u'pray', u'that', 'is', u'the', u'central', u'point', 'of', u'the', u'christian', u'doctrin', 'of', u'work', u'henc', 'it', 'is', u'that', u'while', u'both', u'communism', u'and', u'christian', u'are', u'move', 'by', u'compass', u'for', u'the', u'multitud', u'the', u'object', 'of', u'communism', 'is', 'to', u'make', u'the', u'poor', u'richer', u'but', u'the', u'object', 'of', u'christian', 'is', 'to', u'make', u'the', u'rich', u'poor', u'and', u'the', u'poor', u'holi', u'rich', 'or', u'respons', u'the', u'trade', u'union', u'movement', u'seem', 'to', 'be', u'move', u'sole', 'by', u'the', u'ambit', 'to', u'rais', u'wage', u'and', u'shorten', u'hour', 'of', u'labor', u'howev', u'just', u'such', 'an', u'ambit', u'may', 'be', 'it', u'can', u'never', 'be', u'the', u'rule', u'motiv', 'of', u'the', u'cathol', u'worker', u'the', u'poverti', u'induc', 'by', u'industri', u'stink', u'but', u'poverti', 'as', u'seen', 'by', u'christian', 'is', u'actual', 'a', u'holi', u'condit', u'therefor', u'what', 'is', u'demand', 'by', u'the', u'cathol', u'worker', 'is', u'not', 'so', u'much', u'money', 'as', u'control', u'not', u'rich', u'but', u'respons', u'and', u'this', 'is', 'in', u'line', u'with', u'the', u'christian', u'doctrin', 'of', u'privat', u'properti', u'the', u'individu', u'appropri', 'of', u'the', u'mean', 'of', u'product', u'for', 'it', 'is', 'as', u'workmen', u'that', u'man', u'primarili', u'need', u'properti', u'how', u'els', u'can', 'he', u'maintain', u'complet', u'respons', u'the', u'hirel', u'flieth', u'becaus', 'he', 'is', 'a', u'hirel', u'but', u'this', 'is', u'not', 'to', u'say', u'that', u'the', u'wage', u'slave', 'is', 'a', u'bad', u'man', u'but', u'that', 'he', u'has', u'not', u'the', u'respons', 'of', 'an', u'owner', u'and', u'onli', 'he', u'who', 'is', u'fulli', u'respons', u'can', u'truli', u'serv', u'his', u'fellow', u'the', u'lord', u'knew', 'i', u'would', u'like', 'to', 'go', 'on', u'with', u'comment', 'on', u'passag', u'from', u'the', u'pamphlet', u'the', u'young', u'christian', u'worker', u'the', u'lay', u'apostl', u'canon', u'cardijn', u'say', u'must', u'aim', 'at', u'get', u'hold', 'of', u'the', u'crowd', 'i', u'say', u'with', u'dom', u'chautard', u'that', 'he', u'must', u'aim', 'at', u'the', u'hand', u'the', u'few', 'as', u'canon', u'cardijn', u'himself', u'did', u'when', 'he', u'start', u'his', u'parish', u'work', u'with', u'six', u'young', u'peopl', 'to', u'train', 'as', u'lay', u'apostl', u'bewar', 'of', u'the', u'mass', 'of', u'the', u'crowd', u'rememb', u'deb', u'word', u'about', u'the', u'beast', u'there', 'is', 'in', 'it', u'our', u'lord', u'knew', u'what', u'was', 'in', u'men', u'and', u'when', u'the', u'crowd', u'cheer', u'him', 'as', 'he', u'came', u'into', u'jerusalem', 'he', u'knew', u'that', u'the', u'follow', u'week', u'they', u'would', u'crucifi', u'him', u'whole', u'and', u'holi', u'the', u'worker', u'life', 'is', 'a', u'prayer', u'and', 'a', u'sacrific', u'can', 'a', u'hand', 'or', u'tool', u'pray', 'or', u'sacrific', u'can', 'a', u'man', u'who', 'is', u'not', u'whole', 'be', u'holi', 'we', u'may', u'speak', 'of', u'the', u'mystic', 'of', u'labor', u'without', u'exagger', u'but', u'when', 'a', u'man', u'lose', u'himself', 'is', 'it', u'god', 'or', u'the', u'devil', u'who', 'is', u'go', 'to', u'enter', 'in', 'de', u'rougement', u'book', 'is', u'import', u'read', 'it', u'was', u'review', 'by', u'the', u'cathol', u'world', u'and', u'though', u'written', 'by', 'a', u'noncathol', 'it', u'contain', u'onli', u'cathol', u'truth', u'accord', 'to', u'the', u'review', 'de', u'rougement', 'is', u'the', u'protest', u'leader', 'a', u'swiss', 'of', u'the', u'personalist', u'and', u'communitarian', u'movement', 'in', u'pari', 'of', u'which', u'emmanuel', u'mounier', 'is', u'the', u'cathol', u'leader', u'supernatur', u'this', u'supernatur', u'ideal', 'of', u'labor', u'must', u'need', 'be', u'accompani', 'by', 'a', u'supernatur', u'ethic', 'of', u'labor', 'by', 'a', u'proper', u'moral', 'in', u'work', u'condit', u'such', u'influenc', 'as', u'self', u'interest', u'hatr', u'and', u'violenc', u'have', 'no', u'place', 'in', 'it', u'cathol', u'teach', 'on', u'this', u'point', 'is', 'in', u'direct', u'opposit', 'to', u'that', 'of', u'the', u'atheist', u'the', u'agnost', u'and', u'the', u'materialist', u'and', 'it', 'is', u'these', u'who', u'have', u'the', u'ear', 'of', u'the', u'labor', u'class', 'in', u'the', u'matter', 'of', u'work', u'here', u'again', 'is', u'the', u'pacifist', 'in', u'the', u'class', u'struggl', u'yes', 'we', u'are', u'all', u'pacifist', 'in', u'the', u'class', u'struggl', u'though', u'not', 'in', u'intern', u'war', u'ala', u'although', 'if', u'ever', u'war', u'was', u'justifi', u'one', u'would', u'think', u'that', u'peopl', u'could', u'fight', u'for', u'their', u'home', u'for', u'their', u'land', u'for', u'their', u'famili', u'their', u'work', u'half', 'of', u'the', u'tenant', u'farmer', u'and', u'sharecropp', 'of', u'the', u'south', u'are', u'go', 'to', 'be', u'evict', 'in', u'the', u'interest', 'of', u'big', u'busi', u'rememb', u'within', u'the', u'next', u'five', u'year', u'accord', 'to', u'the', u'depart', 'of', u'agricultur', 'of', u'the', 'u', 's', u'holi', u'resist', u'supernatur', u'selfish', 'is', 'in', u'order', u'here', u'and', 'a', u'hatr', 'of', u'such', u'work', u'that', u'degrad', u'and', u'dehuman', 'we', u'must', u'hate', u'evil', 'i', 'am', u'not', u'argu', u'for', u'class', u'war', u'for', u'resist', 'at', u'the', u'point', 'of', u'machin', u'gun', u'for', u'barricad', u'which', 'go', 'by', u'the', u'way', u'with', u'the', u'citi', u'street', u'there', u'must', 'be', u'some', u'probabl', 'of', u'success', u'that', 'is', u'one', 'of', u'the', u'condit', u'for', 'a', u'just', u'war', u'but', 'i', 'am', 'in', u'favor', 'of', u'the', u'nonviol', u'techniqu', 'of', u'the', u'strike', u'the', u'withdraw', 'of', u'labor', u'violenc', u'will', 'be', u'inflict', u'but', u'let', 'it', u'not', 'be', 'on', u'the', u'part', 'of', u'the', u'worker', 'in', u'steinbeck', u'book', 'in', u'dubious', u'battl', u'about', 'a', u'strike', 'of', u'the', u'migratori', u'worker', 'in', u'california', u'the', u'close', u'page', u'tell', 'of', u'the', u'kill', 'of', u'the', u'organ', 'of', u'the', u'worker', u'and', u'the', u'satisfact', 'of', u'the', u'communist', 'at', u'this', u'shed', 'of', u'blood', u'one', u'must', u'lay', u'down', u'one', u'life', u'for', u'one', u'brother', u'they', u'say', 'we', u'must', u'sacrific', u'even', 'to', u'the', u'shed', 'of', u'blood', 'of', u'cours', u'they', 'do', u'not', u'stop', u'there', u'but', u'they', 'do', u'regard', u'the', u'shed', 'of', u'one', u'blood', 'as', 'so', u'essenti', u'that', 'i', u'have', u'seen', u'the', u'worker', u'goad', u'the', u'polic', 'to', u'bring', u'about', u'this', u'violenc', 'so', u'satisfi', 'to', u'man', u'who', u'has', u'not', u'been', u'taught', u'the', u'use', 'of', u'his', u'spiritu', u'weapon', 'so', u'that', 'he', u'may', u'take', u'heaven', 'by', u'violenc', u'and', u'make', 'it', u'more', u'possibl', 'to', u'prais', u'and', u'worship', u'god', u'here', u'and', u'now', u'thus', u'begin', u'one', u'heaven', 'on', u'earth', 'to', u'what', u'extent', 'it', 'is', u'possibl', u'saint', u'and', u'worker', u'the', u'basic', u'ideal', 'of', u'famili', u'life', 'is', u'tomultipli', u'the', u'number', 'of', u'the', u'elect', u'let', u'this', 'be', u'brought', u'home', 'to', u'the', u'work', u'class', u'for', u'they', 'in', u'particular', u'are', u'equal', 'to', u'the', u'act', 'of', u'generos', u'devot', u'and', u'self', u'denial', u'which', u'such', 'an', u'ideal', u'demand', u'what', u'about', u'wage', u'hous', u'the', u'mother', u'work', u'the', u'father', u'withdrawn', u'for', u'long', u'hour', u'from', u'the', u'home', 'so', u'that', u'under', u'our', u'present', u'civil', 'a', u'child', u'might', 'as', u'well', u'not', u'have', 'a', u'father', u'and', u'are', u'not', u'the', u'clergi', u'lay', u'burden', u'too', u'heavi', u'for', u'them', 'to', u'bear', 'on', u'the', u'shoulder', 'of', u'the', u'worker', u'and', u'themselv', u'not', u'will', 'to', u'lighten', u'them', 'by', u'lift', 'a', u'finger', 'to', u'chang', u'the', u'condit', 'of', u'their', u'life', 'it', 'is', u'not', u'right', u'that', u'heroic', u'sanctiti', 'be', u'demand', 'of', u'the', u'worker', u'and', u'the', u'women', 'of', u'his', u'famili', 'we', u'are', u'all', u'call', 'to', 'be', u'saint', u'stpaul', u'say', u'and', u'pope', u'pius', 'xi', u'has', u'repeat', 'in', u'his', u'encycl', 'on', u'stfranci', 'de', u'sale', 'it', u'should', 'be', 'an', u'ordinari', u'thing', u'not', 'a', u'heroic', u'thing', u'what', 'is', u'be', u'done', 'to', u'make', 'it', u'possibl', u'for', u'the', u'worker', 'to', 'be', 'a', u'saint', 'a', u'good', u'ordinari', u'saint', u'follow', u'the', u'littl', u'way', u'sound', u'doctrin', u'mayb', u'the', u'worker', u'life', 'is', 'a', u'way', 'of', u'the', u'cross', u'but', 'it', 'is', 'a', u'continu', 'of', u'the', u'mission', 'of', u'christ', u'the', u'worker', u'let', u'these', u'his', u'successor', u'grasp', u'the', u'possibl', u'and', u'the', u'digniti', 'of', u'their', u'state', u'and', u'they', u'will', 'be', u'proud', 'of', 'it', u'instead', 'of', u'tri', 'to', u'chang', 'it', u'woe', u'woe', 'to', u'the', u'shepherd', u'who', 'do', u'not', u'feed', u'the', u'sheep', u'good', u'sound', u'teach', u'woe', 'to', u'the', u'shepherd', u'who', u'feed', u'themselv', u'and', 'do', u'not', u'feed', u'the', u'sheep', 'in', u'ezeki', u'there', u'are', u'terribl', u'warn', u'and', 'a', u'terribl', u'threat', 'if', u'the', u'worker', u'are', u'lost', 'to', u'the', u'church', u'who', u'will', 'be', u'held', u'respons', u'priest', u'and', u'worker', u'cardin', u'oconnel', 'of', u'boston', u'work', 'in', 'a', u'cotton', u'mill', u'when', 'he', u'was', 'a', u'littl', u'boy', u'but', 'he', u'onli', u'stay', 'a', u'week', 'or', 'so', 'he', u'soon', u'got', u'out', 'of', 'it', u'ive', u'heard', u'mani', 'a', u'worker', u'say', u'laugh', u'canon', u'cardijn', u'work', u'too', u'was', 'it', 'in', 'a', u'factori', u'when', 'he', u'was', 'a', u'child', u'and', u'his', u'father', u'had', u'work', u'befor', u'him', u'but', 'he', u'did', u'not', u'stay', u'there', u'our', u'lord', u'was', 'a', u'worker', u'yes', 'as', u'well', 'as', 'a', u'priest', u'and', u'stpaul', u'work', u'with', u'his', u'hand', 'as', 'a', u'weaver', 'of', u'goat', u'hair', u'but', u'they', u'did', u'not', u'work', 'in', u'factori', u'tend', 'a', u'machin', u'sit', u'half', u'senseless', 'in', u'front', 'of', 'it', u'their', u'hand', u'unus', 'as', 'i', u'have', u'seen', u'men', 'do', 'in', u'the', u'pepperil', u'mill', 'in', u'main', u'they', u'saw', u'their', u'work', u'they', u'were', u'respons', u'for', u'their', u'work', 'so', u'they', u'could', u'regard', u'their', u'work', 'as', u'holi', 'it', u'was', u'whole', u'work', 'i', 'am', u'tire', 'of', u'hear', u'our', u'lord', u'compar', 'to', 'a', u'modern', u'factori', u'worker', u'philip', u'hagren', u'has', 'a', u'bite', u'and', u'strong', u'articl', 'in', 'a', u'recent', u'issu', 'of', u'the', u'cathol', u'worker', 'it', u'had', u'been', u'print', 'in', u'the', u'cross', u'and', u'the', u'plough', u'and', 'in', 'it', 'he', u'wonder', u'how', u'priest', u'would', u'like', 'it', 'if', u'there', u'was', u'subdivis', 'of', u'labor', 'in', u'offer', 'up', u'the', u'holi', u'mass', u'thousand', 'of', u'them', u'recit', u'the', u'gloria', u'thousand', u'recit', u'the', u'introibo', u'but', 'to', 'be', u'consist', 'we', u'should', u'break', 'it', u'down', 'to', u'one', u'senseless', u'syllabl', u'rather', u'than', 'to', 'a', u'prayer', 'or', u'canticl', u'the', u'priest', u'work', u'has', u'remain', u'unchang', u'but', u'not', u'the', u'factori', u'worker', u'what', 'to', 'do', 'it', 'is', u'all', u'veri', u'well', 'to', u'critic', u'peopl', u'will', u'say', 'we', u'will', u'agre', 'to', u'much', 'of', u'what', u'you', u'say', u'but', u'what', 'is', u'there', 'to', 'do', u'but', 'i', u'say', 'we', u'have', u'been', u'told', u'what', 'to', 'do', u'but', u'will', 'we', 'do', 'it', 'do', 'we', u'listen', 'to', u'the', u'holi', u'father', u'himself', u'did', 'we', u'listen', u'dure', u'the', u'war', u'when', 'he', u'urg', 'us', u'not', 'to', u'extend', u'the', u'scene', 'of', u'carnag', u'and', 'we', u'immedi', u'here', 'in', u'america', u'enter', 'in', u'when', 'he', u'urg', u'the', u'sister', u'and', u'priest', 'to', u'keep', u'the', u'war', u'out', 'of', u'the', u'school', u'room', u'and', u'they', u'went', 'on', u'sell', u'war', u'stamp', u'when', 'he', u'urg', u'set', 'up', u'citi', 'of', u'refug', u'while', 'we', u'justifi', u'the', u'bomb', 'of', u'citi', u'and', u'innoc', u'peopl', u'writer', 'g', 'k', u'chesterton', u'has', u'written', u'two', u'bookswhat', u'wrong', u'with', u'the', u'world', u'and', u'the', u'outlin', 'of', u'saniti', u'eric', u'gill', u'has', u'money', u'and', u'moral', u'and', u'cloth', 'if', u'our', u'priest', u'friend', u'would', u'studi', u'these', u'four', u'book', u'and', u'then', u'start', u'think', 'in', u'term', 'of', u'the', u'few', u'rather', u'than', 'of', u'the', u'mass', 'we', u'would', u'have', 'a', u'begin', 'if', u'they', u'would', u'onli', u'read', u'father', u'vincent', u'mcnabb', 'a', u'priest', u'like', u'themselv', u'and', u'prayer', u'consid', u'what', 'he', u'has', 'to', u'tell', u'them', 'he', u'who', u'live', 'in', u'the', u'largest', u'citi', 'of', u'the', u'world', u'kind', 'of', u'work', u'all', u'worker', 'do', u'not', u'have', 'to', 'be', u'farmer', u'there', u'are', u'mani', u'occup', 'on', u'the', u'land', 'a', u'communiti', u'need', u'librari', u'bookbind', u'engrav', u'letter', u'craftsmen', 'of', u'all', u'kind', 'a', u'communiti', u'need', u'bread', u'and', u'bakeri', u'and', u'carpent', u'and', u'cabinet', u'maker', u'and', u'silversmith', u'and', u'laundri', u'and', u'blacksmith', u'shop', u'teacher', u'are', u'need', u'and', u'doctor', u'are', u'need', u'and', u'nurs', u'there', u'are', u'mani', u'small', u'town', u'and', u'villag', u'that', u'need', u'apostl', u'where', u'are', u'the', u'men', u'who', u'will', u'like', u'mose', 'so', u'have', u'compass', 'on', u'the', u'slaveri', 'of', u'their', u'brother', u'that', u'they', u'will', u'lead', u'them', u'out', 'of', u'the', u'land', 'of', u'bondag', u'liter', u'first', 'by', u'word', u'and', u'then', 'by', u'deed', u'there', u'are', u'not', u'onli', u'the', u'pharaoh', 'to', 'be', u'consid', u'but', u'the', u'bishop', u'help', 'or', u'hinder', 'i', u'attend', 'a', u'meet', 'in', u'new', u'york', u'when', u'canon', u'cardijn', u'spoke', 'i', u'wonder', u'were', u'there', u'ani', u'worker', u'there', 'or', u'where', u'they', u'not', u'all', 'of', u'them', u'white', u'collar', u'worker', u'offic', u'worker', u'desk', u'worker', u'paper', u'worker', u'did', u'they', u'not', u'associ', u'themselv', u'with', u'their', u'employ', u'rather', u'than', u'with', u'the', u'worker', u'and', u'did', u'they', u'not', 'in', u'subtl', u'way', 'do', u'the', u'work', u'too', 'of', u'their', u'employ', 'in', u'sanctifi', u'their', u'surround', u'and', u'make', u'the', u'worker', u'realiz', u'the', u'digniti', 'of', u'his', u'surround', u'and', u'his', u'work', u'rather', u'than', 'to', u'attempt', 'to', u'chang', u'them', u'the', u'cathol', u'worker', u'septemb', u'num', u'num', u'summari', u'describ', u'her', u'trip', 'to', u'antigonish', u'nova', u'scotia', u'and', u'her', u'stay', u'with', u'the', u'communiti', u'discuss', u'her', u'meet', u'with', u'the', u'unit', u'mine', u'worker', u'and', u'how', u'cooper', u'store', u'there', u'have', u'built', 'a', u'spiritu', u'foundat', u'for', u'their', u'materi', u'need', u'distribut', u'comment', 'on', u'the', u'communiti', u'independ', u'and', u'it', u'interdepend', 'on', u'one', u'other', u'ddlw', u'num', u'speak', 'of', u'steel', u'worker', u'hous', u'mine', u'worker', u'day', u'after', u'day', u'there', u'are', u'all', u'sort', 'of', u'odd', u'and', u'end', 'in', 'my', u'suitcas', 'as', 'i', u'unpack', u'from', 'my', u'three', u'week', u'trip', 'to', u'nova', u'scotia', u'where', 'i', u'cover', u'the', u'rural', u'and', u'industri', u'confer', 'at', u'antigonish', u'there', u'are', u'the', u'pair', 'of', u'overal', u'which', u'father', u'oneil', u'from', u'georgetown', u'gave', 'me', u'after', 'he', u'had', u'worn', u'them', u'down', u'into', 'a', u'mine', u'one', u'morn', 'at', u'reserv', u'mine', u'cape', u'breton', u'there', u'are', u'some', u'pouch', 'of', u'tobacco', u'from', u'the', u'cooper', u'store', 'at', u'reserv', u'and', 'a', u'huge', u'pair', 'of', u'white', u'mitten', u'big', u'enough', u'for', 'a', u'sevenfoot', u'knit', 'by', u'the', u'fishermen', 'at', u'dover', u'the', u'onli', u'person', u'they', u'match', 'in', u'ani', u'way', 'is', u'big', u'dan', u'ort', u'and', 'im', u'go', 'to', u'give', u'them', 'to', u'him', 'to', u'wear', 'in', u'the', u'dead', 'of', u'winter', u'when', 'he', 'is', u'sell', u'the', u'cathol', u'worker', 'up', 'on', u'fortysecond', u'street', u'hes', 'a', u'good', u'indoctrin', u'big', u'dan', 'is', u'and', 'as', 'he', u'sell', u'the', u'paper', u'hell', 'be', u'tell', u'everyon', 'of', u'the', u'cooper', u'movement', 'at', u'antigonish', u'and', u'therel', 'be', 'a', u'few', u'more', u'street', u'corner', u'discuss', u'round', u'tabl', u'discuss', u'after', u'their', u'fashion', 'on', 'a', u'christian', u'social', u'order', u'there', u'are', u'some', u'handspun', u'wool', u'and', u'book', 'on', u'craft', u'and', u'two', u'carder', u'which', u'teresa', u'get', u'great', u'enjoy', 'in', u'play', u'with', u'she', 'is', u'look', u'forward', 'to', u'the', u'sheep', 'on', u'the', u'farm', u'and', u'wash', u'and', u'card', u'the', u'wool', u'herself', 'to', u'make', u'comfort', u'with', u'two', u'sheep', u'not', u'purchas', u'yet', u'but', u'one', 'of', u'our', u'reader', u'sent', 'us', u'the', u'money', u'for', u'them', u'how', u'long', u'will', 'it', u'take', 'to', u'make', u'seventyf', u'comfort', 'to', u'warm', u'the', u'new', u'york', u'and', u'easton', u'cathol', u'worker', u'gang', 'it', 'is', u'good', 'to', 'be', u'back', u'and', 'i', u'shall', u'not', 'go', u'away', u'again', u'for', 'a', u'year', 'my', u'mail', 'is', u'full', 'of', u'invit', 'to', u'speak', u'throughout', u'the', u'middl', u'west', u'but', 'i', u'have', u'been', u'wander', 'so', u'much', u'here', 'a', u'month', u'gone', 'a', u'month', u'and', u'now', 'i', u'shall', u'stay', u'home', u'for', 'a', u'while', u'out', 'in', u'front', u'scotti', u'clark', u'and', u'shorti', u'are', u'manag', u'the', u'breadlin', u'with', u'their', u'usual', u'kindli', u'and', u'effici', u'and', u'the', u'coffe', 'we', u'still', u'insist', 'is', u'the', u'best', u'anywher', u'weve', u'been', u'drink', u'strong', u'strong', u'tea', u'for', u'the', u'last', u'three', u'weeksgood', u'but', u'not', 'so', u'comfort', 'as', u'coffe', u'scotti', u'just', u'ran', 'in', 'to', u'say', 'we', u'had', u'run', u'out', 'of', u'sugarw', u'need', u'four', u'pound', u'more', 'to', u'get', u'through', u'the', u'morn', u'scottv', 'is', u'not', 'a', u'cathol', u'and', 'i', u'must', u'tell', u'him', u'about', u'all', u'the', u'scotch', u'cathol', 'i', u'met', 'in', u'nova', u'scotia', u'and', u'the', u'scotch', u'game', 'we', u'saw', u'and', u'the', u'highland', u'fling', u'and', u'the', u'sword', u'danc', u'and', u'the', u'music', u'which', u'sound', u'like', u'the', u'chines', u'music', 'in', u'the', u'club', u'room', u'down', 'at', u'the', u'corner', u'joe', u'zarella', u'tell', 'me', u'that', u'dure', u'the', u'festa', 'to', u'san', u'rocco', u'the', u'process', u'has', u'bagpip', u'trumpet', u'and', u'good', u'know', u'what', u'other', u'music', u'instrument', u'eastern', u'steamship', u'line', u'the', u'boat', u'was', u'crowd', u'go', 'up', u'and', u'come', u'back', u'and', u'one', u'cant', u'say', u'much', u'for', u'the', u'servic', 'on', u'the', u'eastern', u'steamship', u'line', u'they', u'have', u'some', u'kind', 'of', u'ineffici', u'arrang', 'in', u'the', u'dine', u'room', 'so', u'that', u'everyon', u'has', 'to', u'wait', 'in', u'line', u'for', 'a', u'seat', u'anoth', u'bread', u'line', u'come', u'back', u'one', u'not', u'onli', u'wait', u'for', u'meal', u'but', u'wait', u'for', u'land', u'card', u'wait', u'again', 'to', u'get', u'them', u'stamp', u'wait', u'again', 'to', u'get', u'off', 'so', u'that', u'the', u'land', u'took', u'almost', u'two', u'and', 'a', u'half', u'hour', u'one', u'man', u'who', u'had', u'been', u'abroad', u'fifti', u'three', u'time', u'said', u'that', 'it', u'was', u'easier', 'to', u'get', u'into', u'germani', u'than', u'back', u'into', u'the', u'unit', u'state', u'but', u'the', u'trip', u'was', u'enjoy', u'with', 'a', u'day', u'and', 'a', u'night', 'on', u'the', u'ocean', u'go', 'up', 'it', u'was', u'foggi', u'and', u'the', u'fog', u'horn', u'mourn', u'but', u'come', u'back', 'it', u'was', u'clear', u'and', u'warm', u'land', 'at', u'yarmouth', u'one', u'had', 'to', u'ride', u'from', u'eleven', 'in', u'the', u'morn', u'till', u'onethirti', 'at', u'night', 'to', u'get', 'to', u'antigonish', 'so', 'we', u'stop', u'off', 'at', u'truro', u'that', u'night', u'and', u'proceed', 'on', u'again', u'the', u'next', u'morn', u'unit', u'mine', u'worker', 'no', u'sooner', u'had', 'we', u'land', 'at', u'the', u'extens', u'offic', 'of', u'franci', u'xavier', u'univers', u'which', 'is', u'the', u'seat', 'of', u'the', u'cooper', u'movement', u'than', u'father', u'coadi', u'announc', 'we', u'were', u'go', u'right', u'back', 'to', u'truro', 'to', u'address', 'a', u'convent', 'of', u'the', u'unit', u'mine', u'worker', u'delight', 'at', u'the', u'close', 'of', u'the', u'organ', u'labor', u'movement', 'to', u'the', u'cooper', u'movement', 'i', u'was', u'glad', 'to', 'go', 'i', u'had', u'gone', 'up', u'there', 'to', u'learn', u'about', u'what', u'was', u'be', u'done', 'to', u'sit', 'in', 'at', u'the', u'confer', 'of', u'stfranci', u'xavier', u'univers', u'but', u'found', 'i', u'had', 'to', u'begin', u'speak', u'onc', u'again', u'befor', 'i', u'was', u'through', 'i', u'had', u'talk', 'to', u'miner', u'steel', u'worker', u'farmer', u'and', u'fishermen', u'and', u'mani', 'of', u'them', u'have', u'been', u'take', u'the', u'cathol', u'worker', u'for', u'year', u'and', u'were', u'close', u'support', 'of', u'the', u'movement', 'it', u'made', 'me', u'feel', u'again', u'how', u'necessari', 'a', u'penni', u'paper', 'is', u'that', u'can', 'be', u'sent', u'out', 'by', u'the', u'bundl', 'it', 'is', u'all', u'the', u'better', u'that', 'it', u'come', u'onli', u'onc', 'a', u'month', u'and', 'is', u'onli', u'eight', u'page', u'the', u'worker', 'do', u'not', u'read', 'a', u'great', u'deal', u'and', u'the', u'reiter', 'of', u'general', u'principl', u'and', u'the', u'idea', u'for', 'a', u'new', u'social', u'order', 'we', u'are', u'stress', u'come', u'with', 'so', u'much', u'the', u'more', u'forc', u'miner', u'and', u'steel', u'worker', u'the', u'cooper', u'store', u'that', u'the', u'miner', u'and', u'steel', u'worker', 'in', u'sidney', u'and', u'reserv', u'and', u'glare', u'bay', u'have', u'start', u'are', u'model', 'of', u'effici', u'and', u'cleanli', 'it', u'sound', 'so', u'paltri', 'to', u'say', u'thatto', u'say', u'that', u'they', u'are', u'the', u'equal', 'of', u'ani', u'wellrun', u'chain', u'store', u'when', u'they', u'mean', 'so', u'much', u'more', u'they', u'mean', u'that', u'group', 'of', u'the', u'worker', u'have', u'gotten', u'togeth', 'to', u'studi', u'condit', u'they', u'have', u'put', u'into', 'it', u'their', u'earn', 'to', u'buy', u'share', u'when', u'they', u'were', u'work', u'onli', u'two', u'and', u'three', u'day', 'a', u'week', u'with', u'larg', u'famili', 'to', u'support', u'they', u'have', u'scrub', u'and', u'paint', u'and', u'scheme', u'and', u'figur', 'to', u'make', 'it', 'a', u'success', u'it', 'a', u'store', 'in', u'the', u'offic', 'of', u'which', u'you', u'see', u'missal', u'lie', 'on', u'the', u'desk', u'it', 'a', u'store', u'where', u'the', u'board', u'think', 'in', u'term', 'of', u'prayer', 'to', u'straighten', u'out', u'difficulti', u'the', u'old', u'store', u'use', 'to', u'give', u'credit', u'and', u'goug', u'the', u'custom', u'but', u'that', u'credit', u'meant', u'enslav', 'it', u'meant', u'that', u'store', u'shut', u'off', u'credit', 'in', u'time', 'of', u'strike', u'but', 'we', u'know', u'what', 'it', u'mean', 'to', 'go', u'hungri', u'one', 'of', u'the', u'board', u'said', 'we', u'have', u'known', u'the', u'time', 'in', u'our', u'own', u'childhood', u'when', u'our', u'father', u'came', u'home', u'with', u'noth', u'but', 'a', u'loaf', 'of', u'bread', u'for', u'the', u'whole', u'famili', 'to', u'eat', 'we', u'know', u'that', u'voluntari', u'poverti', u'and', u'the', u'work', 'of', u'merci', u'which', u'the', u'cathol', u'worker', u'stress', u'must', u'have', u'their', u'place', 'in', u'this', u'movement', u'too', 'we', u'know', u'that', u'the', u'spiritu', u'foundat', 'is', u'everyth', u'that', u'without', 'it', u'materi', u'success', u'mean', u'noth', 'we', u'are', u'work', u'becaus', 'we', u'are', u'member', 'of', u'the', u'mystic', u'bodi', u'and', 'we', u'know', u'that', u'when', u'the', u'health', 'of', u'one', u'member', u'suffer', u'the', u'health', 'of', u'the', u'whole', u'bodi', 'is', u'lower', 'so', 'we', u'are', u'start', 'a', u'stvincent', 'de', u'paul', u'societi', 'to', u'build', 'up', u'the', u'idea', 'of', u'mutual', u'chariti', u'and', u'mutual', u'aid', u'everyon', u'bring', 'in', u'what', 'he', u'can', u'spare', u'one', 'of', u'the', u'member', 'of', u'the', u'cooper', u'board', 'is', u'the', u'prime', u'mover', u'hous', 'it', 'is', u'hard', u'tri', 'to', u'write', u'this', 'at', u'the', u'last', u'minut', u'befor', u'go', 'to', u'press', u'one', u'can', u'onli', u'touch', 'on', 'a', u'few', 'of', u'the', u'highlight', 'of', u'the', u'antigonish', u'movement', u'but', u'after', u'all', u'our', u'reader', u'can', u'get', u'pamphlet', u'and', u'materi', 'by', u'write', 'to', u'the', u'extens', u'depart', u'stfranci', u'xavier', u'univers', u'antigonish', u'nova', u'scotia', u'tell', u'them', u'you', u'are', 'a', u'cathol', u'worker', u'reader', u'and', u'theyll', u'send', 'on', u'the', u'pamphlet', u'tell', u'how', u'they', u'got', u'start', u'the', u'hous', u'project', u'that', 'is', u'start', 'by', u'the', u'miner', u'deserv', 'a', u'whole', u'articl', u'they', u'start', u'with', u'ten', u'famili', 'in', 'a', u'studi', u'club', u'and', u'now', u'they', u'have', 'a', u'villag', u'they', u'call', u'tompkinsvill', u'after', 'fr', 'j', 'j', u'tompkin', u'who', u'with', u'father', u'coadi', u'are', u'the', u'prime', u'mover', 'of', u'the', u'new', u'order', 'in', u'maritim', u'provinc', u'which', u'includ', u'nova', u'scotia', u'cape', u'breton', u'new', u'brunswick', u'new', u'foundland', u'all', u'the', u'men', 'in', u'the', u'group', u'are', u'miner', u'and', u'they', u'have', u'known', u'each', u'other', u'sinc', u'childhood', u'they', u'averag', 'in', u'age', u'num', u'year', u'the', u'oldest', 'is', u'around', u'fortyf', u'and', u'the', u'youngest', u'twentyon', u'there', u'are', u'num', u'member', 'of', u'the', u'communiti', u'count', u'the', u'children', u'they', u'studi', u'their', u'problem', u'for', u'six', u'month', u'befor', u'they', u'start', u'they', u'design', u'their', u'hous', u'even', 'to', u'the', u'extent', 'of', u'build', u'them', 'of', u'cardboard', u'first', u'the', u'hous', u'are', u'all', u'num', 'by', u'num', u'feet', u'and', u'the', u'ten', u'acr', 'of', u'land', u'cost', u'fifti', u'dollar', 'an', u'acr', u'they', u'have', 'an', u'option', 'on', 'an', u'addit', u'ten', u'acr', u'for', 'a', u'communal', u'farm', u'adjoin', 'to', u'work', 'in', u'their', u'spare', u'time', u'then', u'when', u'the', u'whole', u'plan', u'was', u'work', u'out', u'they', u'were', u'abl', 'to', u'get', u'allow', u'from', u'thc', u'govern', 'of', u'num', u'five', u'hundr', 'of', u'which', 'is', u'for', u'their', u'labor', u'sinc', u'they', u'are', u'do', u'the', u'work', u'themselv', u'none', 'of', u'the', u'men', u'are', u'carpent', u'they', u'are', u'all', u'miner', u'but', 'by', u'call', 'in', u'occasion', 'a', u'skill', u'carpent', u'mason', u'and', u'plumber', u'they', u'are', u'abl', 'to', u'put', 'up', u'the', u'hous', u'themselv', u'one', 'is', u'complet', u'bath', u'and', u'all', 'as', 'a', u'sampl', 'to', u'work', u'out', u'the', u'budget', u'and', u'the', u'other', u'ten', u'are', u'go', 'up', u'with', u'all', u'the', u'men', u'work', 'on', u'all', u'the', u'hous', 'at', u'onc', 'to', u'work', 'so', u'that', u'all', u'the', u'hous', u'will', 'be', u'readi', u'for', u'them', 'at', u'the', u'same', u'time', u'they', u'dug', u'all', u'the', u'foundat', u'first', u'then', u'did', u'the', u'cement', u'work', u'then', u'the', u'framework', u'and', 'so', 'on', u'they', u'have', u'appoint', 'a', u'foreman', u'out', 'of', u'their', u'number', u'and', u'they', u'keep', 'a', u'time', u'sheet', 'on', u'the', u'day', u'when', u'the', u'mine', u'are', u'not', u'work', u'the', u'men', u'put', 'in', u'ten', u'hour', 'a', u'day', u'the', u'day', u'they', u'put', 'in', u'eight', u'hour', 'at', u'the', u'mine', u'they', u'put', 'in', u'four', 'on', u'the', u'hous', u'build', u'one', u'man', u'said', u'that', 'he', u'had', u'been', u'forc', 'to', 'be', u'absent', u'for', u'one', u'day', u'becaus', 'he', u'had', u'put', 'in', u'sixteen', u'hour', u'straight', 'at', u'the', u'mine', u'fill', 'in', u'for', u'one', 'of', u'the', u'men', u'who', u'was', u'off', u'mainadieu', u'the', u'fish', u'and', u'farm', u'communiti', u'where', 'i', u'spoke', u'was', u'down', 'on', u'the', u'top', 'of', u'cape', u'breton', u'there', u'are', u'not', u'mani', u'hous', 'in', u'the', u'littl', u'settlement', u'but', u'the', u'men', u'came', u'from', u'along', u'the', u'shore', u'and', u'from', u'inland', u'and', u'famili', u'came', u'with', u'their', u'children', 'we', u'had', u'oil', u'lamp', u'lit', u'four', 'of', u'them', u'hang', 'in', 'a', u'chandeli', 'in', u'the', u'middl', 'of', u'the', u'room', u'even', u'the', u'glebe', u'hous', u'which', 'is', u'what', u'they', u'call', u'the', u'rectori', u'has', 'no', u'electr', 'or', u'run', u'water', 'it', u'was', 'a', u'serious', u'sympathet', u'crowd', u'which', u'gather', u'togeth', 'to', u'listen', u'that', u'night', u'everywher', u'one', u'found', u'reader', 'of', u'the', u'cathol', u'worker', u'who', u'felt', u'that', u'they', u'knew', 'us', u'all', u'here', 'in', u'new', u'york', u'and', 'in', u'our', u'other', u'branch', 'it', u'was', u'good', 'to', u'talk', 'to', u'them', u'next', u'morn', u'teresa', 'fr', u'poiriet', u'and', 'i', u'got', 'up', 'at', u'sixthirti', u'and', u'took', u'one', 'of', u'the', u'fish', u'boat', u'with', 'an', u'old', u'automobil', u'engin', 'in', 'it', u'out', 'to', u'scateri', u'island', u'which', 'is', u'eight', u'mile', u'out', u'from', u'the', u'mainland', u'and', u'extend', u'seven', u'mile', u'out', u'into', u'the', u'ocean', 'it', 'as', u'the', u'last', u'tip', 'of', u'land', u'lindbergh', u'saw', 'as', 'he', u'made', u'his', u'first', u'flight', u'cross', u'the', u'ocean', u'when', 'we', u'got', 'to', u'the', u'furthest', u'tip', 'of', u'the', u'island', u'and', u'pull', 'at', u'the', u'littl', u'dock', 'we', u'still', u'had', 'a', u'walk', 'of', 'a', u'mile', u'across', u'the', u'island', 'to', u'get', 'to', u'the', u'church', u'there', u'are', u'onli', u'twenti', u'famili', 'on', u'the', u'island', u'live', 'on', u'either', u'side', 'of', 'it', u'and', u'father', u'poiriet', u'get', u'over', u'there', u'onc', 'a', u'month', 'he', u'had', 'to', u'hear', 'a', u'great', u'mani', u'confess', u'befor', u'mass', u'began', u'even', u'the', u'littlest', u'girl', u'and', u'boy', u'all', u'dress', 'up', u'for', u'the', u'great', u'occas', u'prepar', 'to', u'receiv', u'him', u'whom', u'the', u'heaven', u'and', u'sea', u'can', u'not', u'contain', u'there', u'was', 'a', u'sermon', u'and', u'after', u'the', u'mass', 'we', u'had', 'a', u'littl', u'meet', u'out', 'in', u'front', 'of', u'the', u'church', u'from', u'the', u'step', u'look', u'out', u'over', u'the', u'sea', u'where', u'the', u'fish', u'boat', u'were', u'out', u'alreadi', u'the', u'men', 'on', u'scateri', u'had', u'stay', u'late', u'ashor', 'to', u'receiv', u'communion', 'we', u'had', u'breakfast', 'in', u'the', u'littl', u'threeroom', u'hous', 'of', u'one', 'of', u'the', u'familiesa', u'meal', 'of', u'porridg', u'tea', u'and', u'toast', u'the', u'young', u'mother', 'of', u'nine', u'children', u'two', 'of', u'them', u'had', u'die', u'serv', 'us', u'daintili', u'hold', 'a', u'babi', 'in', u'her', u'arm', u'who', u'crow', 'at', 'us', u'bliss', 'it', 'is', 'a', u'hard', u'life', u'for', u'the', u'women', u'there', 'on', u'the', u'island', u'what', u'with', u'the', u'long', u'winter', u'when', u'the', u'children', u'are', u'within', u'door', u'it', u'next', 'to', u'imposs', 'to', u'get', 'a', u'doctor', u'there', 'in', u'the', u'winter', u'the', u'sea', 'is', 'so', u'full', 'of', u'float', u'ice', u'and', u'last', u'winter', u'she', u'had', u'been', u'ill', u'with', u'scarlet', u'fever', u'when', u'the', u'babi', u'was', u'born', u'and', u'the', u'other', u'children', u'down', u'with', 'it', u'too', 'a', u'gallant', u'and', u'brave', u'woman', u'young', u'mrswadden', u'none', 'of', u'the', u'peopl', u'consid', u'themselv', 'to', 'be', u'lead', u'heroic', u'live', u'heroic', 'in', u'their', u'patienc', u'and', u'endur', 'of', u'poverti', u'they', u'are', u'independ', u'and', u'sturdi', u'and', u'they', u'have', u'hope', u'start', 'a', u'studi', u'and', 'a', u'credit', u'union', u'when', u'the', u'fish', 'is', u'good', u'and', u'the', u'market', u'good', u'they', u'get', u'ten', u'cent', 'a', u'pound', u'for', u'the', u'swordfish', u'which', u'come', 'to', 'as', u'big', 'as', u'five', u'hundr', u'pound', u'then', u'they', u'have', 'a', u'good', u'winter', u'but', u'they', u'must', u'organ', u'their', u'market', u'and', u'besid', u'that', u'they', u'must', u'studi', u'farm', u'and', u'handicraft', u'there', 'is', u'not', 'a', u'plough', 'on', u'the', u'island', u'and', u'onli', u'one', u'hors', u'there', u'are', u'cow', u'and', u'chicken', u'but', 'no', u'sheep', u'more', 'to', u'come', 'we', u'are', u'go', 'to', u'write', u'more', u'and', u'more', u'about', u'this', u'work', u'dure', u'the', u'come', u'winter', 'we', u'wish', u'our', u'reader', 'to', u'know', 'of', u'this', u'power', u'hous', u'which', 'is', u'antigonish', u'which', 'is', u'send', u'light', u'over', u'the', u'contin', u'they', u'are', 'in', u'their', u'begin', u'after', u'year', 'of', u'patient', u'endur', u'and', u'studi', u'they', u'are', u'work', u'the', u'littl', u'way', u'and', u'littl', u'sttheres', u'whose', u'statu', u'stand', u'over', u'the', u'altar', 'in', u'the', u'church', 'on', u'scateri', u'island', u'must', u'love', u'them', u'the', u'cathol', u'worker', u'may', u'num', u'num', u'summari', u'outlin', 'p', u'maurin', u'program', u'for', u'social', u'reorder', u'call', u'for', 'a', u'green', u'revolut', 'a', u'return', 'to', u'the', u'villag', u'find', u'his', u'whole', u'messag', u'embodi', 'in', u'person', u'which', u'begin', u'with', u'oneself', u'blame', u'the', u'cws', u'problem', 'in', u'it', u'lack', 'of', u'abil', 'to', u'limit', u'itself', u'ddlw', u'num', u'with', u'the', u'may', u'issu', 'of', u'the', u'cathol', u'worker', 'we', u'begin', u'now', u'our', u'numnd', u'year', u'peter', u'maurin', u'the', u'founder', 'of', u'the', u'cathol', u'worker', u'movement', u'die', 'in', u'num', u'may', u'num', 'on', u'the', u'feast', 'of', u'stjohn', u'baptist', 'de', 'la', u'sall', 'in', u'whose', u'order', 'of', u'christian', u'brother', 'he', u'had', u'taught', u'for', u'five', u'year', 'as', 'a', u'young', u'man', 'in', u'pari', 'he', u'was', u'preemin', 'a', u'teacher', 'an', u'agit', 'he', u'like', 'to', u'call', u'himself', u'and', 'he', u'brought', 'to', 'us', u'great', u'book', u'and', u'great', u'idea', u'and', u'great', u'men', 'so', u'that', u'over', u'the', u'year', 'we', u'have', u'becom', 'a', u'school', u'for', u'the', u'servic', 'of', u'god', u'here', u'and', u'now', u'mani', u'have', u'come', u'and', u'gone', 'in', u'this', u'work', u'find', u'their', u'vocat', 'in', u'religion', 'or', 'in', u'the', u'world', 'as', u'usual', 'we', u'went', u'out', u'into', u'union', u'squar', u'this', u'may', u'day', 'to', u'distribut', u'some', u'two', u'thousand', u'copi', 'of', u'the', u'paper', u'and', 'to', u'meet', u'with', u'old', u'friend', u'with', u'whom', 'we', u'can', u'talk', 'of', u'philosoph', u'differ', u'and', u'with', u'whom', 'we', u'are', u'unit', 'in', 'a', u'passion', u'for', u'peac', u'and', u'justic', u'peter', u'book', 'on', u'this', u'anniversari', 'of', u'peter', u'death', 'we', u'announc', u'anoth', u'edit', 'of', u'peter', u'maurin', u'easi', u'essay', 'to', 'be', u'publish', 'by', u'the', u'thistl', u'press', u'which', 'is', u'also', u'bring', u'out', u'anoth', u'one', 'of', u'the', u'fritz', u'eichenberg', u'album', u'short', u'there', u'may', 'be', 'a', u'few', u'copi', 'of', u'the', u'first', u'one', u'left', u'this', u'book', u'will', u'not', u'onli', u'contain', u'peter', u'essay', 'in', u'new', u'format', u'but', u'also', u'some', u'essay', u'about', u'peter', u'written', 'by', u'his', u'friend', u'includ', u'one', u'editori', u'from', u'blackfriar', u'about', u'peter', 'as', u'prophet', u'when', 'we', u'listen', 'to', u'abb', u'pierr', u'this', u'month', u'and', u'his', u'talk', 'of', u'the', u'need', u'for', u'prophet', 'to', u'bring', 'to', u'men', 'in', u'power', u'the', u'need', 'of', u'the', u'poor', 'to', u'cri', u'out', u'unceas', u'for', u'justic', 'we', u'thought', 'of', u'peter', u'peter', u'use', 'to', u'love', 'to', u'quot', u'eric', u'gill', u'who', u'said', u'that', u'jesus', u'christ', u'came', 'to', u'make', u'the', u'rich', u'poor', u'and', u'the', u'poor', u'holi', 'as', u'for', u'the', u'destitut', 'we', u'can', u'onli', u'reach', u'them', u'with', u'love', u'and', u'the', u'work', 'of', u'merci', u'perform', u'person', 'at', 'a', u'sacrific', u'you', u'can', u'not', u'preach', 'to', u'men', u'with', u'empti', u'stomach', u'our', u'temptat', u'becaus', u'peter', u'program', u'call', u'for', u'such', u'practic', u'thing', 'as', u'hous', 'of', u'hospit', u'and', u'farm', u'commune', 'or', u'agronom', u'univers', 'we', u'have', u'often', u'forgotten', u'the', u'first', u'point', 'in', u'his', u'program', u'which', u'was', u'the', u'need', u'for', u'clarif', 'of', u'thought', u'the', u'need', 'to', u'clarifi', u'the', u'theori', 'of', u'revolut', 'he', u'use', 'to', u'quot', u'lenin', 'as', u'say', u'there', u'can', 'be', 'no', u'revolut', u'without', 'a', u'theori', 'of', u'revolut', u'but', u'peter', u'was', u'the', u'green', u'revolut', 'a', u'call', u'for', 'a', u'return', 'to', u'the', u'villag', u'and', u'the', u'land', 'to', u'make', u'that', u'kind', 'of', u'societi', u'where', 'it', 'is', u'easier', u'for', u'men', 'to', 'be', u'good', u'realiz', u'that', 'we', u'had', u'all', u'too', u'often', u'leap', u'into', u'the', u'activ', u'work', 'of', u'tri', 'to', u'initi', u'these', u'farm', u'commune', u'and', u'agronom', u'univers', u'when', u'our', u'vocat', u'was', 'to', u'write', u'and', u'speak', u'and', 'go', u'out', u'into', u'the', u'highway', u'and', u'byway', u'and', u'that', u'even', u'the', u'model', u'societi', u'wasnt', u'the', u'first', u'step', 'in', u'chang', u'men', u'heart', u'toward', u'each', u'other', 'we', u'start', 'in', u'num', 'a', u'retreat', u'hous', u'where', u'all', u'could', u'come', u'and', u'make', u'five', u'day', u'silent', u'retreat', 'to', u'begin', u'again', u'the', u'work', 'of', u'put', u'off', u'the', u'old', u'man', u'and', u'put', 'on', u'our', u'lord', u'jesus', u'christ', u'with', u'him', 'we', u'could', 'do', u'all', u'thing', u'and', u'without', u'him', 'we', u'could', 'do', u'noth', u'our', u'farm', u'maryfarm', u'and', u'peter', u'maurin', u'farm', u'becam', u'onc', u'again', u'hous', 'of', u'hospit', 'on', u'the', u'land', 'as', u'well', 'as', u'place', u'where', 'we', u'could', u'have', u'retreat', u'and', u'day', 'of', u'recollect', u'peopl', u'who', u'need', u'hospit', u'and', u'who', u'are', u'suffer', 'in', u'bodi', u'mind', u'and', u'soul', u'are', u'not', u'the', u'one', 'to', 'be', u'start', u'agronom', u'univers', u'even', u'though', u'god', u'has', u'alway', u'use', u'the', u'mean', u'and', u'lowli', u'the', u'weak', u'and', u'powerless', 'to', 'do', u'his', u'work', 'we', u'will', u'get', u'the', u'work', u'initi', u'that', 'we', u'know', 'as', 'it', 'is', u'now', u'all', u'over', u'the', u'countri', u'thing', u'are', u'be', u'done', u'that', u'never', u'would', u'have', u'been', u'done', 'if', u'there', u'had', u'been', 'no', u'peter', u'maurin', u'back', 'in', u'num', u'broadcast', u'his', u'idea', u'through', u'the', u'new', u'medium', 'of', u'the', u'cathol', u'worker', u'befor', u'that', 'he', u'had', u'work', 'as', 'an', u'individu', u'spread', u'mimeograph', u'sheet', 'or', u'even', u'hand', u'written', u'one', 'to', u'all', u'who', u'would', u'read', u'and', u'stop', 'in', u'the', u'public', u'squar', u'all', u'those', u'who', u'would', u'listen', u'person', u'anarch', u'libertarian', u'his', u'whole', u'messag', u'was', u'that', u'everyth', u'began', u'with', u'one', u'self', 'he', u'term', u'his', u'messag', 'a', u'personalist', u'one', u'and', u'was', u'much', u'avers', 'to', u'the', u'word', u'socialist', u'sinc', 'it', u'had', u'alway', u'been', u'associ', u'with', u'the', u'idea', 'of', u'polit', u'action', u'the', u'action', 'of', u'the', u'citi', 'or', u'the', u'state', 'he', u'want', 'us', u'all', 'to', 'be', u'what', 'we', u'want', u'the', u'other', u'fellow', 'to', 'be', 'if', u'everi', u'man', u'becam', u'poor', u'there', u'would', u'not', 'be', u'ani', u'destitut', 'he', u'said', 'if', u'everyon', u'becam', u'better', u'everyon', u'would', 'be', u'better', u'off', 'he', u'want', 'us', u'all', 'to', u'quit', u'pass', u'the', u'buck', u'and', u'tri', 'to', u'pass', 'on', u'the', u'work', 'to', u'georg', 'to', 'do', 'he', u'love', u'use', u'american', u'slang', 'in', u'his', u'french', u'peasant', u'accent', u'which', u'made', 'it', u'veri', u'funni', u'but', 'it', u'has', u'kept', u'his', u'most', u'popular', u'essay', u'from', u'be', u'appreci', 'in', u'his', u'nativ', u'countri', u'franc', u'freedom', u'abov', u'all', 'it', u'was', 'in', u'the', u'name', 'of', u'man', u'freedom', u'that', u'peter', u'oppos', u'all', u'govern', u'ownership', 'of', u'the', u'indig', 'as', u'one', u'bishop', u'put', 'it', u'men', u'who', u'were', u'truli', u'brother', u'would', u'share', u'what', u'they', u'had', u'and', u'that', u'was', u'the', u'begin', 'of', u'simpl', u'communiti', u'two', 'is', u'make', 'a', 'we', 'he', u'use', 'to', u'say', u'and', 'we', 'is', 'a', u'communiti', u'and', u'they', 'is', 'a', u'crowd', 'a', u'lone', u'crowd', 'he', u'would', u'have', u'ad', 'if', 'he', u'had', u'read', u'reisman', u'book', u'men', u'were', u'free', u'and', u'they', u'were', u'alway', u'reject', u'their', u'freedom', u'which', u'brought', u'with', 'it', 'so', u'mani', u'respons', 'he', u'want', 'no', u'organ', 'so', u'the', u'cathol', u'worker', u'group', u'have', u'alway', u'been', u'free', u'associ', 'of', u'peopl', u'who', u'are', u'work', u'togeth', 'to', u'get', u'out', 'a', u'paper', 'to', u'run', u'hous', 'of', u'hospit', u'for', u'themselv', u'and', u'for', u'other', u'who', u'come', 'in', u'off', u'the', u'road', 'no', u'class', u'war', 'in', u'addit', 'to', u'be', u'oppos', 'to', u'intern', u'and', u'civil', u'war', 'he', u'was', u'oppos', 'to', u'race', u'war', u'and', u'class', u'war', 'he', u'had', u'taken', 'to', u'himself', u'that', u'new', u'constitut', u'that', u'new', u'rule', 'of', u'the', u'sermon', 'on', u'the', u'mount', u'and', u'truli', u'love', u'his', u'enemi', u'and', u'want', 'to', 'do', u'good', 'to', u'all', u'men', u'includ', u'those', u'who', u'injur', u'him', 'or', u'tri', 'to', u'enslav', u'him', 'he', u'liter', u'believ', 'in', u'overcom', u'evil', u'with', u'good', u'hatr', u'with', u'love', 'he', u'love', u'the', u'rich', 'as', u'well', 'as', u'the', u'poor', u'and', 'he', u'want', 'to', u'make', u'the', u'rich', u'envi', u'the', u'poor', u'who', u'were', 'so', u'close', 'to', u'christ', u'and', 'to', u'tri', 'to', u'becom', u'closer', 'to', u'them', 'by', u'give', 'of', u'their', u'mean', 'to', u'start', u'these', u'school', u'farm', u'commune', u'and', u'agronom', u'univers', u'hous', 'of', u'hospit', u'are', u'alway', u'run', 'by', u'the', u'generos', 'of', u'the', u'poor', u'who', u'work', 'in', u'them', u'and', 'by', u'the', u'donat', 'of', u'the', u'more', u'comfort', u'off', u'who', u'send', u'what', u'they', u'can', 'to', u'keep', u'them', u'go', u'but', u'one', u'realiz', u'more', u'and', u'more', u'that', u'farmer', u'and', u'agronomist', u'and', u'craftsmen', 'do', u'not', u'seek', u'hospit', 'do', u'not', u'come', 'in', u'off', u'the', u'road', u'they', u'might', u'give', 'a', u'year', 'or', 'so', 'of', u'their', u'live', 'if', u'there', u'were', u'the', u'tool', 'to', u'work', u'with', u'even', u'the', u'hous', 'to', u'live', 'in', 'it', 'is', 'a', u'piti', u'thing', 'to', u'hous', u'priest', 'in', u'chicken', u'coop', u'even', u'though', u'they', u'have', u'the', u'privaci', 'of', u'one', u'room', 'in', u'these', u'convert', u'shelter', 'it', 'is', u'hard', 'to', u'expect', 'a', u'craftsman', 'to', u'work', u'when', 'he', 'is', u'coop', 'up', 'in', 'a', u'dormitori', u'and', u'there', 'is', 'no', u'space', u'for', u'his', u'tool', 'we', u'are', u'all', u'greedi', 'no', u'anoth', u'one', 'of', u'our', u'mistak', 'in', u'the', u'past', 'is', u'that', 'we', u'have', u'want', 'to', 'be', u'all', u'thing', 'to', 'do', u'all', u'thing', u'and', u'while', 'we', u'have', u'learn', 'by', u'do', 'we', u'have', u'also', u'learn', u'what', 'we', u'can', u'not', 'do', 'we', u'can', u'agit', 'we', u'can', u'initi', 'we', u'can', u'arous', u'the', u'conscienc', u'but', 'we', u'can', u'not', u'start', 'a', u'hous', u'project', u'for', u'the', u'destitut', 'as', u'abb', u'pierr', u'has', 'in', u'pari', 'or', 'a', u'model', u'villag', 'or', 'an', u'agronom', u'univers', u'either', u'part', 'of', u'abb', u'pierr', u'great', u'wisdom', 'is', u'that', 'he', u'limit', u'himself', 'to', u'that', u'most', u'import', u'work', 'of', u'the', u'day', u'shelter', u'the', u'harborless', u'without', u'question', u'with', u'the', u'love', 'of', u'his', u'fellow', u'poor', 'he', u'himself', u'had', u'gone', u'out', 'to', u'sleep', 'in', u'the', u'doorway', 'on', u'the', u'hard', u'pavement', 'in', u'order', 'to', u'give', u'his', u'bed', 'to', 'a', u'destitut', u'woman', u'and', u'child', u'and', 'in', u'reward', u'for', u'this', u'folli', 'of', u'love', 'he', u'had', u'been', u'enabl', 'to', u'arous', u'the', u'peopl', 'of', u'franc', 'so', u'that', 'in', 'a', u'brief', u'year', u'more', u'was', u'accomplish', u'than', 'he', u'had', u'ever', u'been', u'abl', 'to', u'accomplish', 'by', u'his', u'seven', u'year', 'in', u'the', u'hous', 'of', u'deputi', 'in', u'pari', u'how', u'peter', u'would', u'have', u'love', u'his', u'singl', u'minded', u'his', u'puriti', 'of', u'vision', 'we', u'have', u'had', u'mani', u'with', 'us', u'who', u'could', u'not', u'find', u'their', u'vocat', u'there', u'have', u'been', u'the', u'wander', u'monk', u'that', u'stbenedict', u'talk', 'of', u'they', u'want', u'religi', u'life', u'and', u'life', 'in', u'the', u'world', u'they', u'want', 'to', u'have', u'famili', u'and', 'to', u'preach', u'not', u'teach', u'they', u'want', 'so', u'much', u'not', u'recogn', 'it', u'was', u'god', u'himself', u'they', u'want', u'that', u'they', u'could', u'not', u'develop', u'the', u'talent', u'god', u'gave', u'them', u'and', u'wander', u'year', u'after', u'year', u'wonder', u'what', u'god', u'want', u'them', 'to', 'do', u'peter', u'would', u'tell', u'them', u'first', 'of', u'all', u'earn', 'a', u'live', 'by', u'the', u'sweat', 'of', u'your', u'own', u'brow', u'not', u'some', u'one', u'els', u'choos', 'a', u'work', u'that', u'can', 'be', u'consid', u'honor', u'and', u'can', 'be', u'class', u'under', u'the', u'head', 'of', 'a', u'work', 'of', u'merci', u'serv', u'your', u'brother', u'not', u'exploit', u'them', u'man', u'work', 'is', 'as', u'import', 'to', u'him', 'as', u'bread', u'and', 'by', 'it', 'he', u'gain', u'his', u'bread', u'and', 'by', 'it', 'he', u'gain', u'love', u'too', u'becaus', 'he', u'serv', u'his', u'brother', u'and', u'love', 'is', 'an', u'exchang', 'of', u'gift', u'how', u'often', 'i', u'have', u'seen', u'peopl', u'begin', 'to', u'love', u'each', u'other', u'becaus', u'they', u'work', u'togeth', u'they', u'began', u'toknow', '', u'each', u'other', u'through', u'the', u'work', u'they', u'share', u'stbenedict', u'how', u'peter', u'love', u'stbenedict', u'whose', u'motto', u'was', u'work', u'and', u'pray', 'he', 'is', u'happi', 'no', u'doubt', u'that', 'i', u'his', u'cowork', u'was', u'profess', u'last', u'month', 'as', 'a', u'full', u'oblat', 'of', u'stbenedict', u'attach', 'to', u'stprocopius', u'abbey', u'the', u'mission', 'of', u'which', 'is', 'to', u'work', u'for', u'uniti', u'between', u'east', u'and', u'west', u'and', u'which', u'aim', 'to', u'set', 'up', 'a', u'shrine', 'to', u'the', u'eastern', u'saint', 'at', u'the', u'monasteri', 'at', u'lisl', u'illinoi', 'he', u'love', u'stbenedict', u'becaus', 'he', u'said', u'that', u'what', u'the', u'worker', u'need', u'most', u'was', 'a', u'philosophi', 'of', u'work', 'he', u'love', u'stfranci', u'becaus', 'he', u'said', u'stfranci', u'through', u'his', u'voluntari', u'poverti', u'was', u'free', 'as', 'a', u'bird', u'stfranci', u'was', u'the', u'personalist', u'stbenedict', u'the', u'communitarian', u'and', u'now', u'sad', u'news', u'this', u'issu', 'of', u'the', u'paper', 'is', u'be', u'gotten', u'out', 'by', u'charl', u'mccormack', u'roger', u'oneil', u'ammon', u'hennaci', u'pete', u'asaro', u'peter', u'carey', u'bob', u'stewart', u'and', u'isador', u'fazio', u'the', u'sad', u'news', 'we', u'have', 'to', u'tell', 'is', u'that', u'tom', u'sullivan', 'is', u'take', 'an', u'indefinit', u'leav', 'of', u'absenc', u'the', u'decis', u'came', u'veri', u'sudden', u'for', u'the', u'rest', 'of', 'us', u'and', u'there', u'has', u'been', u'great', u'grief', u'around', u'the', u'place', 'it', u'was', u'like', u'tom', 'to', u'wait', u'until', u'the', u'death', 'of', u'shorti', u'befor', 'he', u'made', 'up', u'his', u'mind', 'he', u'had', u'want', 'to', 'go', 'he', u'said', u'for', 'a', u'year', u'but', u'such', u'human', u'need', 'as', u'shorti', u'kept', u'him', u'like', u'iron', u'chain', u'when', u'shorti', u'die', u'last', u'month', 'he', u'made', u'his', u'decis', u'the', u'lord', u'doe', u'strang', u'thing', u'with', 'us', u'like', u'habbukuk', 'we', u'are', u'pluck', 'by', u'the', u'hair', 'of', u'the', u'head', u'and', u'deposit', u'here', u'and', u'there', 'in', u'the', u'apostol', 'we', u'had', u'all', u'thought', u'that', u'tom', u'like', u'charli', u'orourk', u'was', u'with', 'us', u'for', u'life', u'vain', u'assumpt', u'the', u'lord', u'give', u'and', u'the', u'lord', u'take', u'away', u'tom', u'was', u'with', u'the', u'chicago', u'hous', u'and', u'chicago', u'cathol', u'worker', u'until', u'the', u'war', u'and', u'after', u'the', u'war', 'he', u'came', 'to', u'the', u'new', u'york', u'hous', u'where', u'asid', u'from', u'the', u'interv', 'of', 'a', u'yearh', u'went', 'to', u'chicago', u'for', 'a', u'wed', u'and', u'didnt', u'come', u'backwhen', 'he', u'went', 'to', u'loyola', u'and', u'work', u'for', u'today', u'magazineh', u'has', u'been', u'with', 'us', u'sinc', u'write', u'for', u'and', u'make', u'upth', u'cathol', u'worker', u'each', u'month', u'head', u'the', u'hous', u'have', u'charg', 'of', u'the', u'financ', u'and', 'in', u'general', u'perform', u'all', u'the', u'work', 'of', u'merci', u'who', u'knowsit', u'may', 'be', u'like', u'the', u'last', u'trip', u'this', u'time', 'he', u'went', u'for', 'a', u'vacat', u'and', u'write', 'to', u'tell', 'us', 'he', 'is', u'not', u'come', u'back', u'perhap', 'in', 'a', u'year', u'after', 'a', u'retreat', u'after', 'a', u'sabbat', u'year', 'he', u'will', 'be', u'back', 'it', 'is', u'whatev', u'god', u'want', u'there', 'is', u'some', u'mean', 'to', 'it', u'all', u'charli', u'mccormack', u'said', u'with', 'a', u'veri', u'great', u'sigh', u'never', u'mind', u'said', u'joe', u'motyka', u'who', u'has', u'been', u'with', 'us', u'almost', 'as', u'long', 'as', u'tom', 'we', u'got', u'the', u'paper', u'out', u'dure', u'the', u'war', u'and', u'well', u'get', 'it', u'out', u'again', u'anyon', u'familiar', u'with', u'joe', u'can', u'hear', u'his', u'hardboil', u'accent', 'as', 'he', u'said', 'it', u'but', u'they', u'know', u'too', u'the', u'sad', 'in', u'his', u'heart', u'and', 'in', u'john', u'pohl', u'heart', u'and', u'the', u'rest', 'of', u'the', u'chrysti', u'street', u'gang', 'at', u'the', u'absenc', 'of', 'a', u'friend']
last = len(ddt)
print(ddt[last])

IndexError: list index out of range

In [19]:
drop_pronouns(ddt)

IndexError: string index out of range

In [23]:
tagger.tag(['naw','I','do', 'Know'])

[('naw', 'NN'), ('I', 'PRP'), ('do', 'VBP'), ('Know', 'VB')]

In [28]:
from nltk.probability import FreqDist
import string
import re

In [32]:
freq = FreqDist(ddt) 
columns_obj = ["term", "freq"]
freqDF = pd.DataFrame(freq.items(), columns=columns_obj) # convert it to a data frame
#freqDF = freqDF.set_index('term')

In [38]:
words = freqDF['term'].values.tolist()
print(len(words))


2846


In [42]:
noPro = drop_pronouns(words[1:])
print(len(noPro))

2823


In [43]:
freqDF = freqDF.set_index('term')
print(freqDF.shape)

(2846, 1)


In [45]:
freqDF.head()

,freq
term,
,11
childern,1
vermin,1
four,11
hath,2


In [49]:
freqDFnp = freqDF.ix[noPro]
print(len(freqDFnp))
freqDFnp.head()

2823


,freq
term,
vermin,1
four,11
hath,2
protest,1
poiriet,2
